# Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
import statsmodels.formula.api as smf

# Data Sample
> 2014-2024; 

> NA & Global Firms;

> Interested in returns from 2016-01 onwards

## Emissions Data

### Load

In [4]:
""" Read in the CSV """
raw_emissions_df = pd.read_csv(
    "emissions_2014to2024.csv", 
    dtype={"companyid": "str", "gvkey": "str"}, #identifiers
    parse_dates=["periodenddate"], 
)

raw_emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,gvkey,companyname,country
0,11485,2019,2019-12-31,1.171531,4.551169,56.416747,NaN,Allegany Co-op Insurance Company,United States
1,11489,2023,2023-12-31,4154.828402,12742.671094,176205.682880,NaN,Factory Mutual Insurance Company,United States
2,11489,2021,2021-12-31,3809.540653,16339.416941,203773.712750,NaN,Factory Mutual Insurance Company,United States
3,11489,2022,2022-12-31,1859.947024,5704.373536,78880.089343,NaN,Factory Mutual Insurance Company,United States
4,11489,2020,2020-12-31,3356.875454,12529.659338,159075.046820,NaN,Factory Mutual Insurance Company,United States
...,...,...,...,...,...,...,...,...,...
5929322,119043496,2021,2021-12-31,47.475921,146.879648,280.011919,NaN,Geo Engineering Consulting,Italy
5929323,119043496,2020,2020-12-31,30.736150,87.774611,170.360967,NaN,Geo Engineering Consulting,Italy
5929324,119063782,2021,2021-12-31,25.679523,90.062691,149.618703,NaN,"LA CARPETA I EL PAPER, SA",Spain
5929325,119063782,2020,2020-12-31,19.478006,61.830186,105.704910,NaN,"LA CARPETA I EL PAPER, SA",Spain


### EDA (Can be skipped)

**Inspect the loaded datatypes**

In [1514]:
print(raw_emissions_df["gvkey"].isnull().sum())
print((raw_emissions_df["gvkey"] == "").sum())

1769291
0


In [1511]:
raw_emissions_df.dtypes

institutionid             int64
fiscalyear                int64
periodenddate    datetime64[ns]
di_319413               float64
di_319414               float64
di_319415               float64
companyid                object
gvkey                    object
companyname              object
country                  object
dtype: object

**The company ids are extracted and written out to a file**

In [680]:
""" Utility functions regarding validity, uniqueness, and writing out 
unique and valid ids. """

def keep_valid(data, colname=None):
    """ Only keeps the rows/items from the given dataframe/array-like which 
    have valid values - for the specified column in the case of a dataframe.

    Args:
        data: Dataframe or array-like.
        colname: Name of the dataframe column whose valid values we are using to 
            filter.

    Returns:
        The dataframe/array-like with rows/items that have invalid values, w.r.t 
        the specified column if applicable, filtered out.
    """

    # Array whose values we are interested in
    col = data[colname] if colname is not None else data
    # Values are considered valid as long as they are not NaN
    return data[pd.notnull(col)]


def extract_unique(df, colname):
    """ Extract the unique and non-NaN values from a dataframe column.
    
    Args:
        df: Dataframe.
        colname: Column name w.r.t the dataframe.

    Returns:
        Unique and non-NaN column values.
    """

    return keep_valid(pd.unique(df[colname]))


def write_ids(df, idname, filename):
    """ Writes the unique, non-NaN instances of the indicated identifier, within the 
    indicated dataframe, on separate lines of a new file, whose filename should 
    be specified.

    Args:
        df: Dataframe.
        idname: Column name of the identifier with respect to the dataframe.
        filename: The name to use for the newly created file.
    """
    with open(filename, "w") as fh:
        for idval in extract_unique(df, idname):
            fh.write(f"{idval}\n")

In [4]:
""" Export CIQ company ids """
write_ids(raw_emissions_df, "companyid", "companyids.txt")

**The number of unique company ids and gvkeys are reported for this raw emissions data**

In [681]:
""" Utility function """
def report_unique(df, colname):
    """ Reports unique, non-NaN values of a dataframe column
    
    Args:
        df: Dataframe.
        colname: Column name.
    """

    print(f"Number of unique, non-null values of \'{colname}\': {len(extract_unique(df, colname))}")

In [682]:
print("-- Raw Emissions Data")
report_unique(raw_emissions_df, "institutionid")
report_unique(raw_emissions_df, "companyid")
report_unique(raw_emissions_df, "gvkey")

-- Raw Emissions Data
Number of unique, non-null values of 'institutionid': 1720932
Number of unique, non-null values of 'companyid': 1725359
Number of unique, non-null values of 'gvkey': 24304


**Check whether every gvkey entry has a corresponding company id (non-null -> non-null)**

In [684]:
def is_backed(df, src_colname, ref_colname):
    """ Check if one column is backed by another in a dataframe. This means that 
    when the first column is non-null, the second column cannot be null as 
    otherwise it would be backing the first column.

    Args:
        df: Dataframe.
        src_colname: Name of column that should be backed.
        ref_colname: Name of backing column.
    
    Returns:
        True/False depending on whether the first column is backed by the second
    """

    # check for 0 invalid cases for valid backing
    return (df[src_colname].notnull() & df[ref_colname].isnull()).sum() == 0

def report_backed(df, src_colname, ref_colname):
    """ Reports whether every non-null value of src is backed by a non-null 
    value of ref.
    
    Args:
        df: Dataframe.
        src_colname: Name of column that should be backed.
        ref_colname: Name of backing column.
    """
    print(f"Is \'{src_colname}\' backed by \'{ref_colname}\': {'YES' if is_backed(df, src_colname, ref_colname) else 'NO'}")

In [685]:
print("-- Raw Emissions Data")
report_backed(
    raw_emissions_df, "gvkey", "companyid"
)
report_backed(
    raw_emissions_df, "companyid", "institutionid"
)

-- Raw Emissions Data
Is 'gvkey' backed by 'companyid': YES
Is 'companyid' backed by 'institutionid': YES


**Check for duplicates and null values in the 3 identifiers - companyid, gvkey and institutionid**

In [686]:
""" Utilty functions """
def report_null(df, colname):
    """ Indicate the number of nulls in the specified column in the 
    given dataframe.
    
    Args:
        df: Dataframe.
        src_colname: Name of column that we are interested in.
    """
    print(f"Number of nulls in \'{colname}\': {df[colname].isnull().sum()}")

def report_dup(df, colname):
    """ Indicate the presence of duplicates in the specified column in the 
    given dataframe.
    
    Args:
        df: Dataframe.
        src_colname: Name of column that we are interested in.
    """

    print(f"Are there duplicates in \'{colname}\': {'YES' if df[colname].duplicated().any() else 'NO'}")

In [687]:
print("-- Raw Emissions Data")
print("> Null checks")
report_null(raw_emissions_df, "companyid")
report_null(raw_emissions_df, "institutionid")
report_null(raw_emissions_df, "gvkey")
print("> Dup checks")
report_dup(raw_emissions_df, "companyid")
report_dup(raw_emissions_df, "institutionid")
report_dup(raw_emissions_df, "gvkey")

-- Raw Emissions Data
> Null checks
Number of nulls in 'companyid': 747
Number of nulls in 'institutionid': 0
Number of nulls in 'gvkey': 1769291
> Dup checks
Are there duplicates in 'companyid': YES
Are there duplicates in 'institutionid': YES
Are there duplicates in 'gvkey': YES


**Utilities for numerical value conflicts on grouping by a key. Check for one to one mapping conflicts between identifiers.**

In [688]:
""" Utility functions """

# aggregates series values, NaN if all equal, else string representation with space delimiter
# TODO: assumes no NaN in vals and also the existence of at least 1 element due 
# to the retrieval at index 0.
numer_conflict_aggr = lambda vals: np.nan if (vals.iloc[0] == vals).all() else " ".join(vals.astype(str))

def detect_numerical_conflicts(df, key, numer_cols):
    """ Detects numerical conflicts when grouping on a dataframe by the given 
    keys - filling in the multiple conflict values for inspection.
    
    Args:
        df: Dataframe.
        key: Key or keys that are used for grouping.
        numer_cols: Name of numerical columns which are checked for conflicts
    
    Returns:
        Dataframe with values as NaN if no conflicts, else the conflict values 
        delimited by spaces are filled in.
    """

    return df.groupby(key).agg(
        {
            numer_col : numer_conflict_aggr for numer_col in numer_cols
        }
    )

def one_to_one_conflict_aggr(vals):
    """ Aggregator for the 1t1 conflicts, NaN indicates valid as 1 mapped, else 
    the conflicting values of the second identifier are returned 
    (nothing or multiple).
    
    Args:
        vals: Series that is aggregated.
    Returns:
        NaN if valid, else the conflict values joined together in a string, 
        separated by spaces.
    """
    # simple short-circuit on apparent validity
    if vals.count() == 1:
        return np.nan
    # checking if there is conflict by there being no mapping
    if vals.count() == 0:
        return ""

    # otherwise, we have multiple entries (>1)
    vals = vals.dropna() # remove null bloat to leave just the entries

    if (vals.iloc[0] == vals).all(): # reduces to 1 actual mapping, hence valid
        return np.nan
    else: # otherwise, multiple actual mappings, which creates conflicts
        return " ".join(vals.astype(str))

def detect_1t1_conflicts(df, key, oth_id):
    """ Detects one to one mapping conflicts when grouping on a dataframe by the 
    first identifier and seeing how many of the second identifier this maps to.
    
    Args:
        df: Dataframe.
        key: Key for the first identifier.
        oth_id: Name of the column for the second identifier
    
    Returns:
        Dataframe with values as NaN if no conflicts, else the conflict values, 
        for the second identifier, delimited by spaces are filled in.
    """

    return df.groupby(key).agg(
        {
            oth_id : one_to_one_conflict_aggr, 
        }
    )

**Numerical value conflicts in environmental metrics when grouping by gvkey and 
fiscal year.**

In [689]:
raw_emissions_gvkey_numer_conflicts = detect_numerical_conflicts(
    raw_emissions_df, 
    ["gvkey", "fiscalyear", "periodenddate"], 
    ["di_319413", "di_319414", "di_319415"]
)

raw_emissions_gvkey_numer_conflicts

di_319413 di_319414 di_319415
gvkey  fiscalyear periodenddate                              
001004 2022       2022-05-31          NaN       NaN       NaN
001045 2022       2022-12-31          NaN       NaN       NaN
       2023       2023-12-31          NaN       NaN       NaN
001050 2022       2022-12-31          NaN       NaN       NaN
       2023       2023-12-31          NaN       NaN       NaN
...                                   ...       ...       ...
362683 2022       2022-03-31          NaN       NaN       NaN
362705 2022       2022-12-31          NaN       NaN       NaN
362758 2022       2022-03-31          NaN       NaN       NaN
362761 2022       2022-03-31          NaN       NaN       NaN
       2023       2023-03-31          NaN       NaN       NaN

[33896 rows x 3 columns]

In [690]:
raw_emissions_gvkey_numer_conflicts[
    pd.notnull(raw_emissions_gvkey_numer_conflicts["di_319413"])
    | pd.notnull(raw_emissions_gvkey_numer_conflicts["di_319414"])
    | pd.notnull(raw_emissions_gvkey_numer_conflicts["di_319415"])
]

di_319413  \
gvkey  fiscalyear periodenddate                              
003413 2022       2022-12-31        2867127.0 51062759.241   
003897 2022       2022-12-31     27114550.031 25358937.262   
004094 2022       2022-12-31      810.06 1410.524 7995.579   
005180 2022       2022-12-31        636360.11 53101.342601   
005600 2022       2022-12-31      1080082.1763 4073314.057   
...                                                    ...   
275535 2022       2022-03-31          734.626 112.41037212   
       2023       2023-03-31          611.138 80.405872624   
289724 2022       2022-12-31          101.653 0.0053786782   
295786 2022       2022-12-31       134627.057 0.9207597207   
318474 2022       2022-12-31            916.0 626.35933659   

                                               di_319414  \
gvkey  fiscalyear periodenddate                            
003413 2022       2022-12-31      1018275.0 3933.0872579   
003897 2022       2022-12-31       197000.0 1953.2613298   
004094 2022       2022-12-31     7696.0 2956.39 9723.273   
005180 2022       2022-12-31       251038.0 32063.589498   
005600 2022       2022-12-31        61712.663909 73116.0   
...                                                  ...   
275535 2022       2022-03-31        4498.412 111.2256901   
       2023       2023-03-31       3819.836 79.558482923   
289724 2022       2022-12-31       1642.732 0.0869208615   
295786 2022       2022-12-31     129177.195 3.5451698548   
318474 2022       2022-12-31         4813.0 1649.4464065   

                                                      di_319415  
gvkey  fiscalyear periodenddate                                  
003413 2022       2022-12-31           1650402.398 2024652.4121  
003897 2022       2022-12-31           4025432.601 1005488.8192  
004094 2022       2022-12-31     197193.363 84877.747 39817.706  
005180 2022       2022-12-31           2436669.603 254265.62927  
005600 2022       2022-12-31            850262.94718 353150.506  
...                                                         ...  
275535 2022       2022-03-31              8751.123 2720.5732053  
       2023       2023-03-31              6836.914 1945.9953604  
289724 2022       2022-12-31              7071.309 0.3741598245  
295786 2022       2022-12-31           5009667.962 64.639735408  
318474 2022       2022-12-31            109611.983 3010.0308692  

[95 rows x 3 columns]

It is evident from the above that the gvkeys are mapping to multiple company ids in 
the same period and this is leading to multiple instances of the environment metrics

**Gvkey <-> Companyid Mapping**

- *gvkey -> company id*

In [691]:
raw_emissions_gvkey_companyid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "gvkey", "companyid"
)

raw_emissions_gvkey_companyid_1t1_conflicts

,companyid
gvkey,
001004,NaN
001045,NaN
001050,NaN
001075,NaN
001076,NaN
...,...
362620,NaN
362683,NaN
362705,NaN


In [692]:
raw_emissions_gvkey_companyid_1t1_conflicts[
    pd.notnull(raw_emissions_gvkey_companyid_1t1_conflicts["companyid"])
]

,companyid
gvkey,
002856,259777 259777 3103613 3103613
003413,263295 6020983 6020983
003897,266598 1067744 1067744
004094,528325 27755 386000 386000
005180,275442 275442 5546439
...,...
275535,109366051 109366051 881331 881331
275839,53336883 5482350
289724,215005509 215005509 30941174


Notice that gvkey can map to multiple companyids, but remember that it 
maps to at least 1 company id because it backed by companyid.

- *company id -> gvkey*

In [693]:
raw_emissions_companyid_gvkey_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "companyid", "gvkey"
)

raw_emissions_companyid_gvkey_1t1_conflicts

,gvkey
companyid,
100000307,
100013,NaN
1000212,
1000277,
10004497,NaN
...,...
99996472,
99996476,
99996998,


In [694]:
raw_emissions_companyid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"] == "")
]

,gvkey
companyid,
100000307,
1000212,
1000277,
10004521,
10005029,
...,...
99996472,
99996476,
99996998,


In [695]:
raw_emissions_companyid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"] != "")
]

,gvkey
companyid,
104422749,037090 184982
106683684,285220 316585
1067744,003897 065089 003897 065089
1073371,100557 220942 100557 220942
112732,024616 145471 024616 145471
...,...
9683016,220688 247655
983017,007824 145270 007824 145270
9833116,242985 321595


Notice that companyid can both map to no or multiple gvkeys

**Institutionid <-> Companyid Mapping**

- *companyid -> institutionid*

In [696]:
raw_emissions_companyid_institutionid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "companyid", "institutionid"
)

raw_emissions_companyid_institutionid_1t1_conflicts

,institutionid
companyid,
100000307,NaN
100013,NaN
1000212,NaN
1000277,NaN
10004497,NaN
...,...
99996472,NaN
99996476,NaN
99996998,NaN


In [697]:
raw_emissions_companyid_institutionid_1t1_conflicts[
    pd.notnull(raw_emissions_companyid_institutionid_1t1_conflicts["institutionid"])
]

,institutionid
companyid,


Hence, we can see companyid maps to exactly one institutionid.

- *institutionid -> companyid*

In [698]:
raw_emissions_institutionid_companyid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "institutionid", "companyid"
)

raw_emissions_institutionid_companyid_1t1_conflicts

,companyid
institutionid,
11489,6520204 24951392 43964734 6520204 24951392 439...
11654,NaN
11679,8333444 24586834 8333444 24586834
11894,7925667 34873695 7925667 34873695
12062,NaN
...,...
118526590,NaN
118706040,NaN
118918440,NaN


In [699]:
raw_emissions_institutionid_companyid_1t1_conflicts[
    pd.notnull(raw_emissions_institutionid_companyid_1t1_conflicts["companyid"])
]

,companyid
institutionid,
11489,6520204 24951392 43964734 6520204 24951392 439...
11679,8333444 24586834 8333444 24586834
11894,7925667 34873695 7925667 34873695
13644,26465936 60478967 26465936 60478967
13959,246652 6167099 242354247 246652 6167099 242354247
...,...
109987465,
110365423,
111445983,


Here, we can see that while a company belongs to exactly one institution, an 
institution maps to many companys (none, one or potentially many).

**Gvkey <-> Institutionid Mapping**

- *gvkey -> institutionid*

In [700]:
raw_emissions_gvkey_institutionid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "gvkey", "institutionid"
)

raw_emissions_gvkey_institutionid_1t1_conflicts

,institutionid
gvkey,
001004,NaN
001045,NaN
001050,NaN
001075,NaN
001076,NaN
...,...
362620,NaN
362683,NaN
362705,NaN


In [701]:
raw_emissions_gvkey_institutionid_1t1_conflicts[
    pd.notnull(raw_emissions_gvkey_institutionid_1t1_conflicts["institutionid"])
]

,institutionid
gvkey,
002856,4057039 4057039 4057076 4057076
003413,4057041 4057080 4057080
003897,4057044 4057083 4057083
004094,108462 4021861 4388004 4388004
005180,4806213 4806213 5053995
...,...
275535,4265945 4265945 4326804 4326804
275839,4295672 6343205
289724,4772912 4772912 6393031


Noting that there are no null values in institutionid, this means that gvkey 
corresponds to at least one, and potentially many, institutions.

- *institutionid -> gvkey*

In [702]:
raw_emissions_institutionid_gvkey_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "institutionid", "gvkey"
)

raw_emissions_institutionid_gvkey_1t1_conflicts

,gvkey
institutionid,
11489,
11654,
11679,
11894,
12062,
...,...
118526590,
118706040,
118918440,


In [703]:
raw_emissions_institutionid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"] == "")
]

,gvkey
institutionid,
11489,
11654,
11679,
11894,
12062,
...,...
118526590,
118706040,
118918440,


In [704]:
raw_emissions_institutionid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"] != "")
]

,gvkey
institutionid,
100165,002001 002002
100259,004708 027665 004708 027665
100391,008119 017095 008119 017095
101674,005849 039571
103042,012124 027867
...,...
6618361,204440 247501
6626040,278266 340246
6932029,100787 326859


We can see that institutionids map to many gvkeys (can be zero, one or multiple)

**Determine the number of gvkey identifiers that exist across all fiscal years and also the number that exist in each fiscal year**

First check whether `fiscalyear` is ever null

In [705]:
print("Is fiscalyear ever null:")
print('YES' if raw_emissions_df["fiscalyear"].isnull().any() else 'NO')

Is fiscalyear ever null:
NO


Check if there are any duplicate fiscal year entries for gvkeys

In [706]:
raw_emissions_gvkey_fiscalyear_dup = raw_emissions_df.groupby("gvkey").agg(
    {
        "fiscalyear": lambda vals: vals.duplicated().any(),
    }
)

raw_emissions_gvkey_fiscalyear_dup[
    raw_emissions_gvkey_fiscalyear_dup["fiscalyear"]
]

,fiscalyear
gvkey,
002856,True
003413,True
003897,True
004094,True
005180,True
...,...
271134,True
275535,True
289724,True


We can see there are, and this is due to the duplication from the varying 
`companyid`'s and `institutionid`'s

### Data Preparation
> Mainly involves filtering

In [5]:
# cid_gvkey_df = pd.read_csv("cid_gvkey_map.csv")
cid_gvkey_df = pd.concat(
    [pd.read_csv(f"p{i}_gvkey_cids.csv") for i in range(7)],
    ignore_index=True
)

cid_gvkey_df

,companyid,gvkey,startdate,enddate,companyname
0,18511,210835,B,E,3i Group plc
1,18527,210418,B,E,ABB Ltd
2,18671,29751,B,E,Albemarle Corporation
3,18711,28349,B,E,The Allstate Corporation
4,18749,64768,B,E,"Amazon.com, Inc."
...,...,...,...,...,...
32326,1856268950,50370,B,E,Sucro Limited
32327,1859487646,359029,B,E,KET Inc.
32328,1863445043,362169,B,E,Chaosua Foods Industry Public Company Limited
32329,1866194559,42972,2024-01-01,E,"Eco Bright Future, Inc."


Inspect the loaded datatypes

In [5]:
cid_gvkey_df.dtypes

companyid       int64
gvkey           int64
startdate      object
enddate        object
companyname    object
dtype: object

In [6]:
print("-- Null Values:")
cid_gvkey_df.isnull().sum()

-- Null Values:


companyid      0
gvkey          0
startdate      0
enddate        0
companyname    0
dtype: int64

Filter the mappings to just those that are one-to-one, and keep note of these.

In [6]:
cid_gvkey_1t1_df = cid_gvkey_df[(cid_gvkey_df["startdate"] == "B") & (cid_gvkey_df["enddate"] == "E")]

cid_gvkey_1t1_df

,companyid,gvkey,startdate,enddate,companyname
0,18511,210835,B,E,3i Group plc
1,18527,210418,B,E,ABB Ltd
2,18671,29751,B,E,Albemarle Corporation
3,18711,28349,B,E,The Allstate Corporation
4,18749,64768,B,E,"Amazon.com, Inc."
...,...,...,...,...,...
32325,1855399529,358653,B,E,"SEIYU KOGYO Co.,Ltd."
32326,1856268950,50370,B,E,Sucro Limited
32327,1859487646,359029,B,E,KET Inc.
32328,1863445043,362169,B,E,Chaosua Foods Industry Public Company Limited


*Another approach for filtering mappings is considering where gvkey is not 
duplicated, as opposed to mappings that exist for all of time. This alternative 
approach may however lead to cases where the same company exists under 
different gvkeys (same companyid corresponds to multiple gvkeys). This was not 
chosen in our case.*

Verify that these mappings are indeed one-to-one.

In [8]:
print(f"Gvkey duplicates: {cid_gvkey_1t1_df['gvkey'].duplicated().any()}")
print(f"Companyid duplicates: {cid_gvkey_1t1_df['companyid'].duplicated().any()}")

Gvkey duplicates: False
Companyid duplicates: False


The lack of duplicates on both sides indicates that the mappings are indeed one to one.

>Notice filtering to one-to-one mappings reduces the number of unique gvkeys from ~24k to ~23k in the join table.

**Filter the main environment table using this join table, remembering the following facts:**
The raw main environment table has:
- ~1.8 million rows
- ~1.7 million unique companyid/institutionid values
- ~24k unique gvkey values

From the main environment table, filter entries to just those with gvkeys.

In [7]:
emissions_df = raw_emissions_df[pd.notnull(raw_emissions_df["gvkey"])]
emissions_df = emissions_df.copy()

emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,gvkey,companyname,country
15,11555,2021,2021-12-31,6896.779446,29580.824880,3.689112e+05,122954,"American Family Mutual Insurance Company, S.I.",United States
16,11555,2020,2020-12-31,6711.648272,25051.470514,3.180505e+05,122954,"American Family Mutual Insurance Company, S.I.",United States
161,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,122594,State Farm Mutual Automobile Insurance Company,United States
162,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,122594,State Farm Mutual Automobile Insurance Company,United States
163,13959,2021,2021-12-31,18241.016543,94615.486662,1.580437e+06,122594,State Farm Mutual Automobile Insurance Company,United States
...,...,...,...,...,...,...,...,...,...
5929180,113580486,2019,2019-12-31,17592.373778,1748.093285,5.985221e+03,050370,Sucro Limited,United States
5929209,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,359029,KET Inc.,Japan
5929246,114764179,2022,2022-12-31,1641.238617,1117.543920,4.313828e+04,362169,Chaosua Foods Industry Public Company Limited,Thailand
5929316,118989678,2021,2021-12-31,162.790078,201.931404,5.448875e+03,362779,Novamarine S.p.A.,Italy


In [10]:
emissions_df.dtypes

institutionid             int64
fiscalyear                int64
periodenddate    datetime64[ns]
di_319413               float64
di_319414               float64
di_319415               float64
gvkey                    object
companyname              object
country                  object
dtype: object

In [8]:
emissions_df["gvkey"] = emissions_df["gvkey"].astype(int)
emissions_df.dtypes

institutionid             int64
fiscalyear                int64
periodenddate    datetime64[ns]
di_319413               float64
di_319414               float64
di_319415               float64
gvkey                     int64
companyname              object
country                  object
dtype: object

**Further filter to just the gvkeys that 1 to 1 map with companyid.** 
This should leave one entry per fiscal year for each gvkey. 

*Note some gvkeys may not have an entry for certain fiscal years because that data is simply missing.*

In [9]:
emissions_df = emissions_df[
    np.isin(emissions_df["gvkey"], cid_gvkey_1t1_df["gvkey"])
]

emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,gvkey,companyname,country
15,11555,2021,2021-12-31,6896.779446,29580.824880,3.689112e+05,122954,"American Family Mutual Insurance Company, S.I.",United States
16,11555,2020,2020-12-31,6711.648272,25051.470514,3.180505e+05,122954,"American Family Mutual Insurance Company, S.I.",United States
161,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,122594,State Farm Mutual Automobile Insurance Company,United States
162,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,122594,State Farm Mutual Automobile Insurance Company,United States
163,13959,2021,2021-12-31,18241.016543,94615.486662,1.580437e+06,122594,State Farm Mutual Automobile Insurance Company,United States
...,...,...,...,...,...,...,...,...,...
5929180,113580486,2019,2019-12-31,17592.373778,1748.093285,5.985221e+03,50370,Sucro Limited,United States
5929209,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,359029,KET Inc.,Japan
5929246,114764179,2022,2022-12-31,1641.238617,1117.543920,4.313828e+04,362169,Chaosua Foods Industry Public Company Limited,Thailand
5929316,118989678,2021,2021-12-31,162.790078,201.931404,5.448875e+03,362779,Novamarine S.p.A.,Italy


Here, we check whether the above statement of exactly one entry per gvkey, 
fiscal year is true (barring the exception of missing data).

In [10]:
emissions_gvkey_fiscalyear_entries = emissions_df.groupby(["gvkey", "fiscalyear"]).size()

emissions_gvkey_fiscalyear_entries

gvkey   fiscalyear
1004    2016          1
        2017          1
        2018          1
        2019          1
        2020          1
                     ..
362761  2021          1
        2022          1
        2023          1
362779  2020          1
        2021          1
Length: 147049, dtype: int64

In [11]:
emissions_gvkey_fiscalyear_entries[
    emissions_gvkey_fiscalyear_entries != 1
]

Series([], dtype: int64)

The empty return for a number of entries different than 1 shows that 
we have indeed made gvkey, fiscal year unique.


In [12]:
emissions_df.nunique()["gvkey"]

29403

We can see that this process has reduced the number of unique `gvkey`'s to 
~19k now.

In [16]:
emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,gvkey,companyname,country
15,11555,2021,2021-12-31,6896.779446,29580.824880,3.689112e+05,122954,"American Family Mutual Insurance Company, S.I.",United States
16,11555,2020,2020-12-31,6711.648272,25051.470514,3.180505e+05,122954,"American Family Mutual Insurance Company, S.I.",United States
161,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,122594,State Farm Mutual Automobile Insurance Company,United States
162,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,122594,State Farm Mutual Automobile Insurance Company,United States
163,13959,2021,2021-12-31,18241.016543,94615.486662,1.580437e+06,122594,State Farm Mutual Automobile Insurance Company,United States
...,...,...,...,...,...,...,...,...,...
5929180,113580486,2019,2019-12-31,17592.373778,1748.093285,5.985221e+03,50370,Sucro Limited,United States
5929209,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,359029,KET Inc.,Japan
5929246,114764179,2022,2022-12-31,1641.238617,1117.543920,4.313828e+04,362169,Chaosua Foods Industry Public Company Limited,Thailand
5929316,118989678,2021,2021-12-31,162.790078,201.931404,5.448875e+03,362779,Novamarine S.p.A.,Italy


**Perform QC with respect to null values**

In [13]:
emissions_df.dropna()

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,gvkey,companyname,country
15,11555,2021,2021-12-31,6896.779446,29580.824880,3.689112e+05,122954,"American Family Mutual Insurance Company, S.I.",United States
16,11555,2020,2020-12-31,6711.648272,25051.470514,3.180505e+05,122954,"American Family Mutual Insurance Company, S.I.",United States
161,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,122594,State Farm Mutual Automobile Insurance Company,United States
162,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,122594,State Farm Mutual Automobile Insurance Company,United States
163,13959,2021,2021-12-31,18241.016543,94615.486662,1.580437e+06,122594,State Farm Mutual Automobile Insurance Company,United States
...,...,...,...,...,...,...,...,...,...
5929180,113580486,2019,2019-12-31,17592.373778,1748.093285,5.985221e+03,50370,Sucro Limited,United States
5929209,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,359029,KET Inc.,Japan
5929246,114764179,2022,2022-12-31,1641.238617,1117.543920,4.313828e+04,362169,Chaosua Foods Industry Public Company Limited,Thailand
5929316,118989678,2021,2021-12-31,162.790078,201.931404,5.448875e+03,362779,Novamarine S.p.A.,Italy


We can see that the number of rows is not reduced, therefore there are no 
null values and the data quality has been verified.

**Write out the gvkeys for linking with fundamentals and returns data**

In [1664]:
write_ids(emissions_df, "gvkey", "gvkeys.txt")

**Inspect the fiscal years and the year-month accounting ends which are associated with them**

Create an extra column to isolate the month and year from the entire `periodenddate`. 
This column will be called `periodend_ym` to denote that it just retains the 
year and month information.
- This new column will be of the `period[M]` type

In [15]:
emissions_df = emissions_df.copy()
emissions_df["periodend_ym"] = emissions_df['periodenddate'].dt.to_period('M')

print(f"New Column Type: {emissions_df['periodend_ym'].dtype}")
emissions_df

New Column Type: period[M]


,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,gvkey,companyname,country,periodend_ym
15,11555,2021,2021-12-31,6896.779446,29580.824880,3.689112e+05,122954,"American Family Mutual Insurance Company, S.I.",United States,2021-12
16,11555,2020,2020-12-31,6711.648272,25051.470514,3.180505e+05,122954,"American Family Mutual Insurance Company, S.I.",United States,2020-12
161,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,122594,State Farm Mutual Automobile Insurance Company,United States,2022-12
162,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,122594,State Farm Mutual Automobile Insurance Company,United States,2023-12
163,13959,2021,2021-12-31,18241.016543,94615.486662,1.580437e+06,122594,State Farm Mutual Automobile Insurance Company,United States,2021-12
...,...,...,...,...,...,...,...,...,...,...
5929180,113580486,2019,2019-12-31,17592.373778,1748.093285,5.985221e+03,50370,Sucro Limited,United States,2019-12
5929209,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,359029,KET Inc.,Japan,2022-03
5929246,114764179,2022,2022-12-31,1641.238617,1117.543920,4.313828e+04,362169,Chaosua Foods Industry Public Company Limited,Thailand,2022-12
5929316,118989678,2021,2021-12-31,162.790078,201.931404,5.448875e+03,362779,Novamarine S.p.A.,Italy,2021-12


Group by fiscal year and year-month ends to see which year-month ends are 
contained within each fiscal year.

In [16]:
emissions_df.groupby(["fiscalyear", "periodend_ym"]).size()

fiscalyear  periodend_ym
2013        2014-01            3
2014        2014-01           40
            2014-02           83
            2014-03          829
            2014-04           41
                            ... 
2023        2023-09          265
            2023-10           59
            2023-11           29
            2023-12         5628
            2024-01            5
Length: 131, dtype: int64

In [17]:
emissions_df.groupby("fiscalyear").size()

fiscalyear
2013        3
2014     5743
2015     5813
2016    13043
2017    13907
2018    15944
2019    16368
2020    22111
2021    22165
2022    22799
2023     9153
dtype: int64

**Finalise DataFrame**
- according to joining considerations

In [14]:
emissions_df = emissions_df.reset_index(drop=True)

emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,gvkey,companyname,country
0,11555,2021,2021-12-31,6896.779446,29580.824880,3.689112e+05,122954,"American Family Mutual Insurance Company, S.I.",United States
1,11555,2020,2020-12-31,6711.648272,25051.470514,3.180505e+05,122954,"American Family Mutual Insurance Company, S.I.",United States
2,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,122594,State Farm Mutual Automobile Insurance Company,United States
3,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,122594,State Farm Mutual Automobile Insurance Company,United States
4,13959,2021,2021-12-31,18241.016543,94615.486662,1.580437e+06,122594,State Farm Mutual Automobile Insurance Company,United States
...,...,...,...,...,...,...,...,...,...
147044,113580486,2019,2019-12-31,17592.373778,1748.093285,5.985221e+03,50370,Sucro Limited,United States
147045,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,359029,KET Inc.,Japan
147046,114764179,2022,2022-12-31,1641.238617,1117.543920,4.313828e+04,362169,Chaosua Foods Industry Public Company Limited,Thailand
147047,118989678,2021,2021-12-31,162.790078,201.931404,5.448875e+03,362779,Novamarine S.p.A.,Italy


In [15]:
emissions_df = emissions_df.set_index(
    ["gvkey", "fiscalyear"]
)
emissions_df = emissions_df.sort_index()

emissions_df

institutionid periodenddate     di_319413     di_319414  \
gvkey  fiscalyear                                                            
1004   2016              4157610    2016-05-31  56034.630253  28700.181128   
       2017              4157610    2017-05-31  59349.114332  33489.111842   
       2018              4157610    2018-05-31  54842.261413  30334.505186   
       2019              4157610    2019-05-31  62932.147241  32039.420175   
       2020              4157610    2020-05-31  62592.179000  32015.494000   
...                          ...           ...           ...           ...   
362761 2021              6627661    2021-03-31   5542.197413   4629.001659   
       2022              6627661    2022-03-31   7202.802447   4349.187602   
       2023              6627661    2023-03-31   6651.660029   4016.397443   
362779 2020            118989678    2020-12-31    105.640229    119.005359   
       2021            118989678    2021-12-31    162.790078    201.931404   

                       di_319415                              companyname  \
gvkey  fiscalyear                                                           
1004   2016        263831.791850                                AAR Corp.   
       2017        311638.624770                                AAR Corp.   
       2018        211206.593710                                AAR Corp.   
       2019        217486.646950                                AAR Corp.   
       2020        192435.070000                                AAR Corp.   
...                          ...                                      ...   
362761 2021         34334.587431  Akums Drugs and Pharmaceuticals Limited   
       2022         34489.242775  Akums Drugs and Pharmaceuticals Limited   
       2023         31850.202650  Akums Drugs and Pharmaceuticals Limited   
362779 2020          3249.154936                        Novamarine S.p.A.   
       2021          5448.875156                        Novamarine S.p.A.   

                         country  
gvkey  fiscalyear                 
1004   2016        United States  
       2017        United States  
       2018        United States  
       2019        United States  
       2020        United States  
...                          ...  
362761 2021                India  
       2022                India  
       2023                India  
362779 2020                Italy  
       2021                Italy  

[147049 rows x 7 columns]

In [17]:
emissions = emissions_df.drop(
    columns=[
        "institutionid", "periodenddate", "companyname", "periodend_ym"
    ]
)
# "companyid"

emissions

di_319413     di_319414      di_319415        country
gvkey  fiscalyear                                                          
1004   2016        56034.630253  28700.181128  263831.791850  United States
       2017        59349.114332  33489.111842  311638.624770  United States
       2018        54842.261413  30334.505186  211206.593710  United States
       2019        62932.147241  32039.420175  217486.646950  United States
       2020        62592.179000  32015.494000  192435.070000  United States
...                         ...           ...            ...            ...
362761 2021         5542.197413   4629.001659   34334.587431          India
       2022         7202.802447   4349.187602   34489.242775          India
       2023         6651.660029   4016.397443   31850.202650          India
362779 2020          105.640229    119.005359    3249.154936          Italy
       2021          162.790078    201.931404    5448.875156          Italy

[147049 rows x 4 columns]

**Inspect 0 GHG values across scopes 1, 2 and 3**

In [24]:
emissions_df[
    (emissions_df["di_319413"] == 0) 
    | (emissions_df["di_319414"] == 0)
]

institutionid periodenddate     di_319413  di_319414  \
gvkey  fiscalyear                                                         
2578   2022              4071224    2022-12-31  0.000000e+00    1112.00   
4093   2014              4121470    2014-12-31  1.260000e+08       0.00   
12689  2019              4094395    2019-11-30  0.000000e+00   24025.00   
       2020              4094395    2020-11-30  0.000000e+00   19744.00   
15647  2015              4144815    2015-12-31  0.000000e+00     426.00   
...                          ...           ...           ...        ...   
349529 2022             10691607    2022-12-31  0.000000e+00      80.48   
351514 2022             27663466    2022-12-31  0.000000e+00   25113.70   
       2023             27663466    2023-12-31  0.000000e+00   12902.78   
351587 2022            100596622    2022-12-31  0.000000e+00     311.79   
354657 2023            107457845    2023-12-31  0.000000e+00     253.31   

                      di_319415                                  companyname  \
gvkey  fiscalyear                                                              
2578   2022        8.428337e+03                            Telos Corporation   
4093   2014        9.218064e+06                      Duke Energy Corporation   
12689  2019        9.032422e+05                                      KB Home   
       2020        8.371673e+05                                      KB Home   
15647  2015        1.136394e+05                               Storebrand ASA   
...                         ...                                          ...   
349529 2022        1.263894e+03                     Brii Biosciences Limited   
351514 2022        8.387886e+04                       DR Corporation Limited   
       2023        4.470471e+04                       DR Corporation Limited   
351587 2022        1.197620e+05  Hangzhou SF Intra-city Industrial Co., Ltd.   
354657 2023        5.199414e+03                 Flowing Cloud Technology Ltd   

                         country periodend_ym  
gvkey  fiscalyear                              
2578   2022        United States      2022-12  
4093   2014        United States      2014-12  
12689  2019        United States      2019-11  
       2020        United States      2020-11  
15647  2015               Norway      2015-12  
...                          ...          ...  
349529 2022                China      2022-12  
351514 2022                China      2022-12  
       2023                China      2023-12  
351587 2022                China      2022-12  
354657 2023                China      2023-12  

[276 rows x 8 columns]

In [25]:
emissions_df[
    (emissions_df["di_319415"] == 0)
]

,,institutionid,periodenddate,di_319413,di_319414,di_319415,companyname,country,periodend_ym
gvkey,fiscalyear,,,,,,,,


- todo: extract fundamentals data of the same fiscal years (also see the link between periodenddate and fiscal year)
- todo: market data of one year ahead from the fiscal/fundamentals data
  - or at least one month

**Phase 1 Context Awareness**

Number of US based companies out of the ~19k unique gvkeys

In [18]:
(emissions_df.groupby(level="gvkey").first()["country"] == "United States").sum()

4655

In [47]:
# pd.unique(emissions_df["country"])

## Currency Data
- currency exchange fluctuations, when taking USD as the base currency, cause FX returns that should be accounted for
- market value and other quantities must be converted into a common currency (USD) to facilitate comparisons

Load daily currency exchange rates to USD & GBP. Sort by origin currency and then date.

In [19]:
exrts_df = pd.read_csv(
    "exrts_2014to2024.csv", 
    parse_dates=["datadate"], 
)
exrts_df = exrts_df.sort_values(["curd", "datadate"])

exrts_df

,curd,datadate,exratd_toGBP,exratd_toUSD
0,AED,2014-01-01,0.164541,0.272250
174,AED,2014-01-02,0.165645,0.272254
348,AED,2014-01-03,0.165590,0.272264
522,AED,2014-01-04,0.165590,0.272264
692,AED,2014-01-05,0.165590,0.272264
...,...,...,...,...
628762,ZWL,2024-08-17,0.002350,0.003032
628915,ZWL,2024-08-18,0.002350,0.003032
629068,ZWL,2024-08-19,0.002350,0.003051
629221,ZWL,2024-08-20,0.002350,0.003060


In [28]:
exrts_df.dtypes

curd                    object
datadate        datetime64[ns]
exratd_toGBP           float64
exratd_toUSD           float64
dtype: object

Compute monthly currency rates.
- Based on the exchange rates at the end of the months i.e. the last ones.

In [20]:
exrts_df["data_ym"] = exrts_df["datadate"].dt.to_period('M')

exrts_df

,curd,datadate,exratd_toGBP,exratd_toUSD,data_ym
0,AED,2014-01-01,0.164541,0.272250,2014-01
174,AED,2014-01-02,0.165645,0.272254,2014-01
348,AED,2014-01-03,0.165590,0.272264,2014-01
522,AED,2014-01-04,0.165590,0.272264,2014-01
692,AED,2014-01-05,0.165590,0.272264,2014-01
...,...,...,...,...,...
628762,ZWL,2024-08-17,0.002350,0.003032,2024-08
628915,ZWL,2024-08-18,0.002350,0.003032,2024-08
629068,ZWL,2024-08-19,0.002350,0.003051,2024-08
629221,ZWL,2024-08-20,0.002350,0.003060,2024-08


In [21]:
m_exrts = exrts_df.groupby(
    ["curd", "data_ym"]
).last(
).drop(
    columns="datadate"
)

m_exrts

exratd_toGBP  exratd_toUSD
curd data_ym                            
AED  2014-01      0.165667      0.272257
     2014-02      0.162459      0.272249
     2014-03      0.163316      0.272248
     2014-04      0.161238      0.272251
     2014-05      0.162301      0.272243
...                    ...           ...
ZWL  2024-04      0.002350      0.002944
     2024-05      0.002350      0.002991
     2024-06      0.002350      0.002970
     2024-07      0.002350      0.003018
     2024-08      0.002350      0.003072

[20787 rows x 2 columns]

## Market Data
- Global Market Index Data

In [22]:
mkt_df = pd.read_csv(
    "market_prices_2014to2024.csv", 
    parse_dates=["datadate"], 
)
# year-month index
mkt_df["data_ym"] = mkt_df["datadate"].dt.to_period("M")
mkt_df = mkt_df.set_index("data_ym")
# compute monthly returns from prices
mkt_df["m_mktret"] = mkt_df["prccm"].pct_change() * 100
mkt_df = mkt_df.dropna()

mkt_df

,gvkeyx,prccm,datadate,conm,tic,m_mktret
data_ym,,,,,,
2014-02,150918,1856.3682,2014-02-28,S&P Global 1200 Index,I6UNK112,4.694558
2014-03,150918,1858.1357,2014-03-31,S&P Global 1200 Index,I6UNK112,0.095213
2014-04,150918,1877.0979,2014-04-30,S&P Global 1200 Index,I6UNK112,1.020496
2014-05,150918,1906.7544,2014-05-31,S&P Global 1200 Index,I6UNK112,1.579912
2014-06,150918,1937.6656,2014-06-30,S&P Global 1200 Index,I6UNK112,1.621142
...,...,...,...,...,...,...
2024-03,150918,3774.8709,2024-03-31,S&P Global 1200 Index,I6UNK112,3.153459
2024-04,150918,3636.9339,2024-04-30,S&P Global 1200 Index,I6UNK112,-3.654085
2024-05,150918,3792.7481,2024-05-31,S&P Global 1200 Index,I6UNK112,4.284219


In [23]:
mkt_df = mkt_df[["m_mktret"]]

mkt_df

,m_mktret
data_ym,
2014-02,4.694558
2014-03,0.095213
2014-04,1.020496
2014-05,1.579912
2014-06,1.621142
...,...
2024-03,3.153459
2024-04,-3.654085
2024-05,4.284219


## Returns Data

### NA & Global - Loading and Preliminary Processing

**NA**

In [33]:
na_returns_df = pd.read_csv(
    "na_security_returns_2014to2024.csv", 
    dtype={"iid": "str", "tpci": "str"}, 
    parse_dates=["datadate"], 
)
print(na_returns_df.dtypes)
na_returns_df = na_returns_df.sort_values(["gvkey", "iid", "datadate"])

na_returns_df

KeyboardInterrupt: 

In [168]:
na_returns = na_returns_df[
    (na_returns_df["tpci"] == "0")
]
na_returns = na_returns.sort_values(["gvkey", "iid", "datadate"])
na_returns["data_ym"] = na_returns["datadate"].dt.to_period('M')

print(na_returns.isnull().sum())
na_returns

gvkey             0
iid               0
datadate          0
conm              0
curcdd         6091
ajexdi         6091
cshoc         37292
prccd          6245
trfd        4148160
tpci              0
data_ym           0
dtype: int64


,gvkey,iid,datadate,conm,curcdd,ajexdi,cshoc,prccd,trfd,tpci,data_ym
0,1004,01,2014-01-02,AAR CORP,USD,1.0,39600000.0,27.360,1.619454,0,2014-01
1,1004,01,2014-01-03,AAR CORP,USD,1.0,39600000.0,26.880,1.619454,0,2014-01
2,1004,01,2014-01-06,AAR CORP,USD,1.0,39600000.0,26.780,1.619454,0,2014-01
3,1004,01,2014-01-07,AAR CORP,USD,1.0,39600000.0,26.510,1.619454,0,2014-01
4,1004,01,2014-01-08,AAR CORP,USD,1.0,39600000.0,26.730,1.619454,0,2014-01
...,...,...,...,...,...,...,...,...,...,...,...
17873581,351590,01,2024-08-19,DAIMLER TRUCK HOLDING AG,USD,1.0,822952000.0,37.374,NaN,0,2024-08
17873582,351590,01,2024-08-20,DAIMLER TRUCK HOLDING AG,USD,1.0,822952000.0,37.270,NaN,0,2024-08
17873583,351590,01,2024-08-21,DAIMLER TRUCK HOLDING AG,USD,1.0,822952000.0,37.270,NaN,0,2024-08
17873584,351590,01,2024-08-22,DAIMLER TRUCK HOLDING AG,USD,1.0,822952000.0,37.540,NaN,0,2024-08


In [193]:
na_returns = na_returns.dropna(subset="prccd")

print(na_returns.isnull().sum())
na_returns

gvkey             0
iid               0
datadate          0
conm              0
curcdd            0
ajexdi            0
cshoc         31201
prccd             0
trfd        4147467
tpci              0
data_ym           0
dtype: int64


,gvkey,iid,datadate,conm,curcdd,ajexdi,cshoc,prccd,trfd,tpci,data_ym
0,1004,01,2014-01-02,AAR CORP,USD,1.0,39600000.0,27.360,1.619454,0,2014-01
1,1004,01,2014-01-03,AAR CORP,USD,1.0,39600000.0,26.880,1.619454,0,2014-01
2,1004,01,2014-01-06,AAR CORP,USD,1.0,39600000.0,26.780,1.619454,0,2014-01
3,1004,01,2014-01-07,AAR CORP,USD,1.0,39600000.0,26.510,1.619454,0,2014-01
4,1004,01,2014-01-08,AAR CORP,USD,1.0,39600000.0,26.730,1.619454,0,2014-01
...,...,...,...,...,...,...,...,...,...,...,...
17873581,351590,01,2024-08-19,DAIMLER TRUCK HOLDING AG,USD,1.0,822952000.0,37.374,NaN,0,2024-08
17873582,351590,01,2024-08-20,DAIMLER TRUCK HOLDING AG,USD,1.0,822952000.0,37.270,NaN,0,2024-08
17873583,351590,01,2024-08-21,DAIMLER TRUCK HOLDING AG,USD,1.0,822952000.0,37.270,NaN,0,2024-08
17873584,351590,01,2024-08-22,DAIMLER TRUCK HOLDING AG,USD,1.0,822952000.0,37.540,NaN,0,2024-08


In [194]:
na_m_returns = na_returns.groupby(
    ["gvkey", "iid", "data_ym"]
).last()

na_m_returns

datadate                      conm curcdd  ajexdi  \
gvkey  iid data_ym                                                       
1004   01  2014-01 2014-01-31                  AAR CORP    USD     1.0   
           2014-02 2014-02-28                  AAR CORP    USD     1.0   
           2014-03 2014-03-31                  AAR CORP    USD     1.0   
           2014-04 2014-04-30                  AAR CORP    USD     1.0   
           2014-05 2014-05-30                  AAR CORP    USD     1.0   
...                       ...                       ...    ...     ...   
351590 01  2024-04 2024-04-30  DAIMLER TRUCK HOLDING AG    USD     1.0   
           2024-05 2024-05-31  DAIMLER TRUCK HOLDING AG    USD     1.0   
           2024-06 2024-06-28  DAIMLER TRUCK HOLDING AG    USD     1.0   
           2024-07 2024-07-31  DAIMLER TRUCK HOLDING AG    USD     1.0   
           2024-08 2024-08-23  DAIMLER TRUCK HOLDING AG    USD     1.0   

                          cshoc   prccd      trfd tpci  
gvkey  iid data_ym                                      
1004   01  2014-01   39600000.0  26.650  1.623944    0  
           2014-02   39600000.0  28.900  1.623944    0  
           2014-03   39569000.0  25.950  1.623944    0  
           2014-04   39569000.0  25.900  1.628586    0  
           2014-05   39569000.0  24.300  1.628586    0  
...                         ...     ...       ...  ...  
351590 01  2024-04  822952000.0  46.864       NaN    0  
           2024-05  822952000.0  41.880       NaN    0  
           2024-06  822952000.0  38.622       NaN    0  
           2024-07  822952000.0  37.938       NaN    0  
           2024-08  822952000.0  38.059       NaN    0  

[619778 rows x 8 columns]

In [195]:
na_m_returns.isnull().sum()

datadate         0
conm             0
curcdd           0
ajexdi           0
cshoc         2507
prccd            0
trfd        213680
tpci             0
dtype: int64

In [196]:
trfd_allna = na_m_returns.isna().groupby(
    level=["gvkey", "iid"]
)["trfd"].all()

trfd_allna_idx = trfd_allna[trfd_allna].index
trfd_allna_idx

MultiIndex([(  1166, '02'),
            (  1712, '01'),
            (  1864, '01'),
            (  1932, '01'),
            (  2176, '01'),
            (  2176, '02'),
            (  2220, '01'),
            (  2250, '01'),
            (  2411, '01'),
            (  2578, '08'),
            ...
            (347007, '01'),
            (347328, '01'),
            (347471, '01'),
            (347708, '01'),
            (348615, '01'),
            (349485, '01'),
            (350366, '01'),
            (350952, '01'),
            (351491, '01'),
            (351590, '01')],
           names=['gvkey', 'iid'], length=2571)

In [197]:
na_m_returns.loc[
    np.isin(
        na_m_returns.index.droplevel("data_ym"), 
        trfd_allna_idx
    ), 
    "trfd"
] = 1

na_m_returns

datadate                      conm curcdd  ajexdi  \
gvkey  iid data_ym                                                       
1004   01  2014-01 2014-01-31                  AAR CORP    USD     1.0   
           2014-02 2014-02-28                  AAR CORP    USD     1.0   
           2014-03 2014-03-31                  AAR CORP    USD     1.0   
           2014-04 2014-04-30                  AAR CORP    USD     1.0   
           2014-05 2014-05-30                  AAR CORP    USD     1.0   
...                       ...                       ...    ...     ...   
351590 01  2024-04 2024-04-30  DAIMLER TRUCK HOLDING AG    USD     1.0   
           2024-05 2024-05-31  DAIMLER TRUCK HOLDING AG    USD     1.0   
           2024-06 2024-06-28  DAIMLER TRUCK HOLDING AG    USD     1.0   
           2024-07 2024-07-31  DAIMLER TRUCK HOLDING AG    USD     1.0   
           2024-08 2024-08-23  DAIMLER TRUCK HOLDING AG    USD     1.0   

                          cshoc   prccd      trfd tpci  
gvkey  iid data_ym                                      
1004   01  2014-01   39600000.0  26.650  1.623944    0  
           2014-02   39600000.0  28.900  1.623944    0  
           2014-03   39569000.0  25.950  1.623944    0  
           2014-04   39569000.0  25.900  1.628586    0  
           2014-05   39569000.0  24.300  1.628586    0  
...                         ...     ...       ...  ...  
351590 01  2024-04  822952000.0  46.864  1.000000    0  
           2024-05  822952000.0  41.880  1.000000    0  
           2024-06  822952000.0  38.622  1.000000    0  
           2024-07  822952000.0  37.938  1.000000    0  
           2024-08  822952000.0  38.059  1.000000    0  

[619778 rows x 8 columns]

In [198]:
na_m_returns.isnull().sum()

datadate       0
conm           0
curcdd         0
ajexdi         0
cshoc       2507
prccd          0
trfd           0
tpci           0
dtype: int64

In [199]:
na_m_returns = na_m_returns.dropna()

na_m_returns

datadate                      conm curcdd  ajexdi  \
gvkey  iid data_ym                                                       
1004   01  2014-01 2014-01-31                  AAR CORP    USD     1.0   
           2014-02 2014-02-28                  AAR CORP    USD     1.0   
           2014-03 2014-03-31                  AAR CORP    USD     1.0   
           2014-04 2014-04-30                  AAR CORP    USD     1.0   
           2014-05 2014-05-30                  AAR CORP    USD     1.0   
...                       ...                       ...    ...     ...   
351590 01  2024-04 2024-04-30  DAIMLER TRUCK HOLDING AG    USD     1.0   
           2024-05 2024-05-31  DAIMLER TRUCK HOLDING AG    USD     1.0   
           2024-06 2024-06-28  DAIMLER TRUCK HOLDING AG    USD     1.0   
           2024-07 2024-07-31  DAIMLER TRUCK HOLDING AG    USD     1.0   
           2024-08 2024-08-23  DAIMLER TRUCK HOLDING AG    USD     1.0   

                          cshoc   prccd      trfd tpci  
gvkey  iid data_ym                                      
1004   01  2014-01   39600000.0  26.650  1.623944    0  
           2014-02   39600000.0  28.900  1.623944    0  
           2014-03   39569000.0  25.950  1.623944    0  
           2014-04   39569000.0  25.900  1.628586    0  
           2014-05   39569000.0  24.300  1.628586    0  
...                         ...     ...       ...  ...  
351590 01  2024-04  822952000.0  46.864  1.000000    0  
           2024-05  822952000.0  41.880  1.000000    0  
           2024-06  822952000.0  38.622  1.000000    0  
           2024-07  822952000.0  37.938  1.000000    0  
           2024-08  822952000.0  38.059  1.000000    0  

[617271 rows x 8 columns]

In [202]:
na_m_returns = na_m_returns.drop(columns="tpci").rename(
    columns={
        "curcdd": "curcdm", 
        "cshoc": "cshom", 
        "prccd": "prccm", 
        "ajexdi": "ajexm", 
        "trfd": "trfm", 
    }
)

na_m_returns

datadate                      conm curcdm  ajexm  \
gvkey  iid data_ym                                                      
1004   01  2014-01 2014-01-31                  AAR CORP    USD    1.0   
           2014-02 2014-02-28                  AAR CORP    USD    1.0   
           2014-03 2014-03-31                  AAR CORP    USD    1.0   
           2014-04 2014-04-30                  AAR CORP    USD    1.0   
           2014-05 2014-05-30                  AAR CORP    USD    1.0   
...                       ...                       ...    ...    ...   
351590 01  2024-04 2024-04-30  DAIMLER TRUCK HOLDING AG    USD    1.0   
           2024-05 2024-05-31  DAIMLER TRUCK HOLDING AG    USD    1.0   
           2024-06 2024-06-28  DAIMLER TRUCK HOLDING AG    USD    1.0   
           2024-07 2024-07-31  DAIMLER TRUCK HOLDING AG    USD    1.0   
           2024-08 2024-08-23  DAIMLER TRUCK HOLDING AG    USD    1.0   

                          cshom   prccm      trfm  
gvkey  iid data_ym                                 
1004   01  2014-01   39600000.0  26.650  1.623944  
           2014-02   39600000.0  28.900  1.623944  
           2014-03   39569000.0  25.950  1.623944  
           2014-04   39569000.0  25.900  1.628586  
           2014-05   39569000.0  24.300  1.628586  
...                         ...     ...       ...  
351590 01  2024-04  822952000.0  46.864  1.000000  
           2024-05  822952000.0  41.880  1.000000  
           2024-06  822952000.0  38.622  1.000000  
           2024-07  822952000.0  37.938  1.000000  
           2024-08  822952000.0  38.059  1.000000  

[617271 rows x 7 columns]

In [203]:
na_m_returns = na_m_returns.reset_index()
na_m_returns.to_csv(
    "na_security_m_returns_2014to2024.csv", 
    index=False, header=True, 
)

na_m_returns

,gvkey,iid,data_ym,datadate,conm,curcdm,ajexm,cshom,prccm,trfm
0,1004,01,2014-01,2014-01-31,AAR CORP,USD,1.0,39600000.0,26.650,1.623944
1,1004,01,2014-02,2014-02-28,AAR CORP,USD,1.0,39600000.0,28.900,1.623944
2,1004,01,2014-03,2014-03-31,AAR CORP,USD,1.0,39569000.0,25.950,1.623944
3,1004,01,2014-04,2014-04-30,AAR CORP,USD,1.0,39569000.0,25.900,1.628586
4,1004,01,2014-05,2014-05-30,AAR CORP,USD,1.0,39569000.0,24.300,1.628586
...,...,...,...,...,...,...,...,...,...,...
617266,351590,01,2024-04,2024-04-30,DAIMLER TRUCK HOLDING AG,USD,1.0,822952000.0,46.864,1.000000
617267,351590,01,2024-05,2024-05-31,DAIMLER TRUCK HOLDING AG,USD,1.0,822952000.0,41.880,1.000000
617268,351590,01,2024-06,2024-06-28,DAIMLER TRUCK HOLDING AG,USD,1.0,822952000.0,38.622,1.000000
617269,351590,01,2024-07,2024-07-31,DAIMLER TRUCK HOLDING AG,USD,1.0,822952000.0,37.938,1.000000


In [24]:
na_m_returns = pd.read_csv(
    "na_security_m_returns_2014to2024.csv", 
    parse_dates=["datadate", "data_ym"], 
)
na_m_returns["data_ym"] = na_m_returns["data_ym"].dt.to_period('M')
na_m_returns = na_m_returns.set_index(["gvkey", "iid", "data_ym"])

na_m_returns

datadate                      conm curcdm  ajexm  \
gvkey  iid data_ym                                                      
1004   01  2014-01 2014-01-31                  AAR CORP    USD    1.0   
           2014-02 2014-02-28                  AAR CORP    USD    1.0   
           2014-03 2014-03-31                  AAR CORP    USD    1.0   
           2014-04 2014-04-30                  AAR CORP    USD    1.0   
           2014-05 2014-05-30                  AAR CORP    USD    1.0   
...                       ...                       ...    ...    ...   
351590 01  2024-04 2024-04-30  DAIMLER TRUCK HOLDING AG    USD    1.0   
           2024-05 2024-05-31  DAIMLER TRUCK HOLDING AG    USD    1.0   
           2024-06 2024-06-28  DAIMLER TRUCK HOLDING AG    USD    1.0   
           2024-07 2024-07-31  DAIMLER TRUCK HOLDING AG    USD    1.0   
           2024-08 2024-08-23  DAIMLER TRUCK HOLDING AG    USD    1.0   

                          cshom   prccm      trfm  
gvkey  iid data_ym                                 
1004   01  2014-01   39600000.0  26.650  1.623944  
           2014-02   39600000.0  28.900  1.623944  
           2014-03   39569000.0  25.950  1.623944  
           2014-04   39569000.0  25.900  1.628586  
           2014-05   39569000.0  24.300  1.628586  
...                         ...     ...       ...  
351590 01  2024-04  822952000.0  46.864  1.000000  
           2024-05  822952000.0  41.880  1.000000  
           2024-06  822952000.0  38.622  1.000000  
           2024-07  822952000.0  37.938  1.000000  
           2024-08  822952000.0  38.059  1.000000  

[617271 rows x 7 columns]

**Global**

In [48]:
global_returns_df = pd.read_csv(
    "global_security_returns_2014to2024.csv", 
    dtype={
        "tpci": "str"
    }, 
    parse_dates=["datadate"], 
)

global_returns_df

,gvkey,iid,datadate,conm,curcdd,ajexdi,cshoc,prccd,trfd,tpci
0,1166,01W,2014-01-01,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.950,1.242834,0
1,1166,01W,2014-01-02,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.855,1.242834,0
2,1166,01W,2014-01-03,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.880,1.242834,0
3,1166,01W,2014-01-06,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.575,1.242834,0
4,1166,01W,2014-01-07,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.535,1.242834,0
...,...,...,...,...,...,...,...,...,...,...
66927973,362779,01W,2024-08-15,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0
66927974,362779,01W,2024-08-16,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0
66927975,362779,01W,2024-08-19,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0
66927976,362779,01W,2024-08-20,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0


In [1763]:
global_returns_df.dtypes

gvkey                int64
iid                 object
datadate    datetime64[ns]
conm                object
curcdd              object
ajexdi             float64
cshoc              float64
prccd              float64
trfd               float64
tpci                object
dtype: object

In [56]:
global_returns_df.isnull().sum()

gvkey            0
iid              0
datadate         0
conm             0
curcdd        9037
ajexdi        9037
cshoc       765959
prccd         9037
trfd          2634
tpci             0
dtype: int64

In [192]:
global_returns_df[
    global_returns_df["prccd"].isna()
].isna().sum()

gvkey          0
iid            0
datadate       0
conm           0
curcdd      9037
ajexdi      9037
cshoc       9037
prccd       9037
trfd        2634
tpci           0
dtype: int64

We are only interested in the returns and market cap from ordinary stock.

In [87]:
global_returns = global_returns_df[
    (global_returns_df["tpci"] == "0")
]

global_returns

,gvkey,iid,datadate,conm,curcdd,ajexdi,cshoc,prccd,trfd,tpci
0,1166,01W,2014-01-01,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.950,1.242834,0
1,1166,01W,2014-01-02,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.855,1.242834,0
2,1166,01W,2014-01-03,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.880,1.242834,0
3,1166,01W,2014-01-06,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.575,1.242834,0
4,1166,01W,2014-01-07,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.535,1.242834,0
...,...,...,...,...,...,...,...,...,...,...
66927973,362779,01W,2024-08-15,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0
66927974,362779,01W,2024-08-16,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0
66927975,362779,01W,2024-08-19,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0
66927976,362779,01W,2024-08-20,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0


Sort by company, issue and then finally date.

In [88]:
global_returns = global_returns.sort_values(["gvkey", "iid", "datadate"])

global_returns

,gvkey,iid,datadate,conm,curcdd,ajexdi,cshoc,prccd,trfd,tpci
0,1166,01W,2014-01-01,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.950,1.242834,0
1,1166,01W,2014-01-02,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.855,1.242834,0
2,1166,01W,2014-01-03,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.880,1.242834,0
3,1166,01W,2014-01-06,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.575,1.242834,0
4,1166,01W,2014-01-07,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.535,1.242834,0
...,...,...,...,...,...,...,...,...,...,...
66927973,362779,01W,2024-08-15,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0
66927974,362779,01W,2024-08-16,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0
66927975,362779,01W,2024-08-19,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0
66927976,362779,01W,2024-08-20,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0


Add the year-month, which will be aggregated on shortly.

In [89]:
global_returns["data_ym"] = global_returns["datadate"].dt.to_period('M')

global_returns

,gvkey,iid,datadate,conm,curcdd,ajexdi,cshoc,prccd,trfd,tpci,data_ym
0,1166,01W,2014-01-01,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.950,1.242834,0,2014-01
1,1166,01W,2014-01-02,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.855,1.242834,0,2014-01
2,1166,01W,2014-01-03,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.880,1.242834,0,2014-01
3,1166,01W,2014-01-06,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.575,1.242834,0,2014-01
4,1166,01W,2014-01-07,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,23.535,1.242834,0,2014-01
...,...,...,...,...,...,...,...,...,...,...,...
66927973,362779,01W,2024-08-15,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0,2024-08
66927974,362779,01W,2024-08-16,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0,2024-08
66927975,362779,01W,2024-08-19,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0,2024-08
66927976,362779,01W,2024-08-20,NOVAMARINE SPA,EUR,1.0,12388500.0,3.580,1.145429,0,2024-08


In [96]:
global_returns = global_returns.groupby(
    ["gvkey", "iid", "data_ym"]
).last()

global_returns

datadate                        conm curcdd  ajexdi  \
gvkey  iid data_ym                                                         
1166   01W 2014-01 2014-01-31        ASM INTERNATIONAL NV    EUR     1.0   
           2014-02 2014-02-28        ASM INTERNATIONAL NV    EUR     1.0   
           2014-03 2014-03-31        ASM INTERNATIONAL NV    EUR     1.0   
           2014-04 2014-04-30        ASM INTERNATIONAL NV    EUR     1.0   
           2014-05 2014-05-30        ASM INTERNATIONAL NV    EUR     1.0   
...                       ...                         ...    ...     ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR     1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR     1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD     1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR     1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR     1.0   

                          cshoc     prccd      trfd tpci  
gvkey  iid data_ym                                        
1166   01W 2014-01   63076035.0    24.850  1.242834    0  
           2014-02   63076035.0    28.015  1.242834    0  
           2014-03   63076035.0    29.135  1.242834    0  
           2014-04   63076035.0    31.495  1.242834    0  
           2014-05   63076035.0    30.545  1.264115    0  
...                         ...       ...       ...  ...  
362705 01W 2024-08   26460000.0     2.820  1.147992    0  
       02W 2024-08   26460000.0     2.630  1.147992    0  
362758 01W 2024-08  154500000.0     0.220  1.147992    0  
362761 01W 2024-08  157393988.0  1010.750  1.147992    0  
362779 01W 2024-08   12388500.0     3.580  1.145429    0  

[2888634 rows x 8 columns]

Drop null values and finalise to the monthly format, assuming ordinary share 
focus.

In [107]:
global_returns = global_returns.dropna()
global_returns = global_returns.drop(columns="tpci").rename(
    columns={
        "curcdd": "curcdm", 
        "cshoc": "cshom", 
        "prccd": "prccm", 
        "ajexdi": "ajexm", 
        "trfd": "trfm", 
    }
)

global_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1166   01W 2014-01 2014-01-31        ASM INTERNATIONAL NV    EUR    1.0   
           2014-02 2014-02-28        ASM INTERNATIONAL NV    EUR    1.0   
           2014-03 2014-03-31        ASM INTERNATIONAL NV    EUR    1.0   
           2014-04 2014-04-30        ASM INTERNATIONAL NV    EUR    1.0   
           2014-05 2014-05-30        ASM INTERNATIONAL NV    EUR    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom     prccm      trfm  
gvkey  iid data_ym                                   
1166   01W 2014-01   63076035.0    24.850  1.242834  
           2014-02   63076035.0    28.015  1.242834  
           2014-03   63076035.0    29.135  1.242834  
           2014-04   63076035.0    31.495  1.242834  
           2014-05   63076035.0    30.545  1.264115  
...                         ...       ...       ...  
362705 01W 2024-08   26460000.0     2.820  1.147992  
       02W 2024-08   26460000.0     2.630  1.147992  
362758 01W 2024-08  154500000.0     0.220  1.147992  
362761 01W 2024-08  157393988.0  1010.750  1.147992  
362779 01W 2024-08   12388500.0     3.580  1.145429  

[2861222 rows x 7 columns]

Reset the index so that it is in a format readily exportable to csv, or 
equivalently assumed to be directly read in from csv.

In [109]:
global_returns = global_returns.reset_index()

global_returns

,gvkey,iid,data_ym,datadate,conm,curcdm,ajexm,cshom,prccm,trfm
0,1166,01W,2014-01,2014-01-31,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,24.850,1.242834
1,1166,01W,2014-02,2014-02-28,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,28.015,1.242834
2,1166,01W,2014-03,2014-03-31,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,29.135,1.242834
3,1166,01W,2014-04,2014-04-30,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,31.495,1.242834
4,1166,01W,2014-05,2014-05-30,ASM INTERNATIONAL NV,EUR,1.0,63076035.0,30.545,1.264115
...,...,...,...,...,...,...,...,...,...,...
2861217,362705,01W,2024-08,2024-08-21,MISITANO & STRACUZZI S P A,EUR,1.0,26460000.0,2.820,1.147992
2861218,362705,02W,2024-08,2024-08-21,MISITANO & STRACUZZI S P A,EUR,1.0,26460000.0,2.630,1.147992
2861219,362758,01W,2024-08,2024-08-21,NUREN GROUP LIMITED,AUD,1.0,154500000.0,0.220,1.147992
2861220,362761,01W,2024-08,2024-08-21,AKUMS DRUGS AND PHARMA,INR,1.0,157393988.0,1010.750,1.147992


In [110]:
global_returns.to_csv(
    "global_security_m_returns_2014to2024.csv", 
    index=False, header=True, 
)

In [25]:
global_returns = pd.read_csv(
    "global_security_m_returns_2014to2024.csv", 
    parse_dates=["datadate", "data_ym"], 
)
global_returns["data_ym"] = global_returns["data_ym"].dt.to_period('M')
global_returns = global_returns.set_index(["gvkey", "iid", "data_ym"])

global_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1166   01W 2014-01 2014-01-31        ASM INTERNATIONAL NV    EUR    1.0   
           2014-02 2014-02-28        ASM INTERNATIONAL NV    EUR    1.0   
           2014-03 2014-03-31        ASM INTERNATIONAL NV    EUR    1.0   
           2014-04 2014-04-30        ASM INTERNATIONAL NV    EUR    1.0   
           2014-05 2014-05-30        ASM INTERNATIONAL NV    EUR    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom     prccm      trfm  
gvkey  iid data_ym                                   
1166   01W 2014-01   63076035.0    24.850  1.242834  
           2014-02   63076035.0    28.015  1.242834  
           2014-03   63076035.0    29.135  1.242834  
           2014-04   63076035.0    31.495  1.242834  
           2014-05   63076035.0    30.545  1.264115  
...                         ...       ...       ...  
362705 01W 2024-08   26460000.0     2.820  1.147992  
       02W 2024-08   26460000.0     2.630  1.147992  
362758 01W 2024-08  154500000.0     0.220  1.147992  
362761 01W 2024-08  157393988.0  1010.750  1.147992  
362779 01W 2024-08   12388500.0     3.580  1.145429  

[2861222 rows x 7 columns]

In [26]:
global_m_returns = global_returns

global_m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1166   01W 2014-01 2014-01-31        ASM INTERNATIONAL NV    EUR    1.0   
           2014-02 2014-02-28        ASM INTERNATIONAL NV    EUR    1.0   
           2014-03 2014-03-31        ASM INTERNATIONAL NV    EUR    1.0   
           2014-04 2014-04-30        ASM INTERNATIONAL NV    EUR    1.0   
           2014-05 2014-05-30        ASM INTERNATIONAL NV    EUR    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom     prccm      trfm  
gvkey  iid data_ym                                   
1166   01W 2014-01   63076035.0    24.850  1.242834  
           2014-02   63076035.0    28.015  1.242834  
           2014-03   63076035.0    29.135  1.242834  
           2014-04   63076035.0    31.495  1.242834  
           2014-05   63076035.0    30.545  1.264115  
...                         ...       ...       ...  
362705 01W 2024-08   26460000.0     2.820  1.147992  
       02W 2024-08   26460000.0     2.630  1.147992  
362758 01W 2024-08  154500000.0     0.220  1.147992  
362761 01W 2024-08  157393988.0  1010.750  1.147992  
362779 01W 2024-08   12388500.0     3.580  1.145429  

[2861222 rows x 7 columns]

**Remarks**

*Notice that across both the NA and Global return data, we make up almost 17k 
out the ~19k companies in the emissions data*

Determine the number of companies that are in both the NA and Global datasets

In [1457]:
na_returns_uniq_gvkey = na_returns_df["gvkey"].unique()
global_returns_uniq_gvkey = global_returns_df["gvkey"].unique()

na_returns_uniq_gvkey[
    np.isin(na_returns_uniq_gvkey, global_returns_uniq_gvkey)
].size

/Users/varunchodanker/opt/anaconda3/lib/python3.9/site-packages/IPython/core/displayhook.py:275: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


972

### NA & Global - Data Preparation

In [205]:
na_m_returns

datadate                      conm curcdm  ajexm  \
gvkey  iid data_ym                                                      
1004   01  2014-01 2014-01-31                  AAR CORP    USD    1.0   
           2014-02 2014-02-28                  AAR CORP    USD    1.0   
           2014-03 2014-03-31                  AAR CORP    USD    1.0   
           2014-04 2014-04-30                  AAR CORP    USD    1.0   
           2014-05 2014-05-30                  AAR CORP    USD    1.0   
...                       ...                       ...    ...    ...   
351590 01  2024-04 2024-04-30  DAIMLER TRUCK HOLDING AG    USD    1.0   
           2024-05 2024-05-31  DAIMLER TRUCK HOLDING AG    USD    1.0   
           2024-06 2024-06-28  DAIMLER TRUCK HOLDING AG    USD    1.0   
           2024-07 2024-07-31  DAIMLER TRUCK HOLDING AG    USD    1.0   
           2024-08 2024-08-23  DAIMLER TRUCK HOLDING AG    USD    1.0   

                          cshom   prccm      trfm  
gvkey  iid data_ym                                 
1004   01  2014-01   39600000.0  26.650  1.623944  
           2014-02   39600000.0  28.900  1.623944  
           2014-03   39569000.0  25.950  1.623944  
           2014-04   39569000.0  25.900  1.628586  
           2014-05   39569000.0  24.300  1.628586  
...                         ...     ...       ...  
351590 01  2024-04  822952000.0  46.864  1.000000  
           2024-05  822952000.0  41.880  1.000000  
           2024-06  822952000.0  38.622  1.000000  
           2024-07  822952000.0  37.938  1.000000  
           2024-08  822952000.0  38.059  1.000000  

[617271 rows x 7 columns]

In [206]:
global_m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1166   01W 2014-01 2014-01-31        ASM INTERNATIONAL NV    EUR    1.0   
           2014-02 2014-02-28        ASM INTERNATIONAL NV    EUR    1.0   
           2014-03 2014-03-31        ASM INTERNATIONAL NV    EUR    1.0   
           2014-04 2014-04-30        ASM INTERNATIONAL NV    EUR    1.0   
           2014-05 2014-05-30        ASM INTERNATIONAL NV    EUR    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom     prccm      trfm  
gvkey  iid data_ym                                   
1166   01W 2014-01   63076035.0    24.850  1.242834  
           2014-02   63076035.0    28.015  1.242834  
           2014-03   63076035.0    29.135  1.242834  
           2014-04   63076035.0    31.495  1.242834  
           2014-05   63076035.0    30.545  1.264115  
...                         ...       ...       ...  
362705 01W 2024-08   26460000.0     2.820  1.147992  
       02W 2024-08   26460000.0     2.630  1.147992  
362758 01W 2024-08  154500000.0     0.220  1.147992  
362761 01W 2024-08  157393988.0  1010.750  1.147992  
362779 01W 2024-08   12388500.0     3.580  1.145429  

[2861222 rows x 7 columns]

In [27]:
m_returns = pd.concat(
    [na_m_returns.reset_index(), global_m_returns.reset_index()], 
    ignore_index=True, 
).groupby(
    ["gvkey", "iid", "data_ym"]
).first().sort_index()

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2014-01 2014-01-31                    AAR CORP    USD    1.0   
           2014-02 2014-02-28                    AAR CORP    USD    1.0   
           2014-03 2014-03-31                    AAR CORP    USD    1.0   
           2014-04 2014-04-30                    AAR CORP    USD    1.0   
           2014-05 2014-05-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom    prccm      trfm  
gvkey  iid data_ym                                  
1004   01  2014-01   39600000.0    26.65  1.623944  
           2014-02   39600000.0    28.90  1.623944  
           2014-03   39569000.0    25.95  1.623944  
           2014-04   39569000.0    25.90  1.628586  
           2014-05   39569000.0    24.30  1.628586  
...                         ...      ...       ...  
362705 01W 2024-08   26460000.0     2.82  1.147992  
       02W 2024-08   26460000.0     2.63  1.147992  
362758 01W 2024-08  154500000.0     0.22  1.147992  
362761 01W 2024-08  157393988.0  1010.75  1.147992  
362779 01W 2024-08   12388500.0     3.58  1.145429  

[3478493 rows x 7 columns]

**Add year-month**

Isolate the year-month from the data point dates, as the data points 
correspond to montly returns, and we are only interested in this 
level of granularity.

Missing values must be addressed - by imputating where possible first, and then 
finally dropping rows (which have null values).
- For imputation, we must determine which variables have missing values and 
decide on the appropriate corrective action for each.

We can see there are missing values in the close prices, adjustment factors and total return factors.

**Imputation**

In [856]:
""" TODO: nothing right now, but can look into what imputation is possible later """

""" if prices and 
adjustment factors are enough to calculate returns, as long as the total return 
factor is all null or all non-null for each issue. """

' TODO: nothing right now, but can look into what imputation is possible later '

**Drop Rows**

Check if the issues have distinct data for each year-month now.

In [28]:
sec_ym_entries = m_returns.groupby(level=["gvkey", "iid", "data_ym"]).size()

sec_ym_entries

gvkey   iid  data_ym
1004    01   2014-01    1
             2014-02    1
             2014-03    1
             2014-04    1
             2014-05    1
                       ..
362705  01W  2024-08    1
        02W  2024-08    1
362758  01W  2024-08    1
362761  01W  2024-08    1
362779  01W  2024-08    1
Length: 3478493, dtype: int64

In [29]:
sec_ym_entries[
    sec_ym_entries != 1
]

Series([], dtype: int64)

The empty series return indicates that this is indeed the case.

**Reindex & Sort**
- by `gvkey`, `iid` and then `data_ym` 
- we know `data_ym` is particularly appropriate 
for indexing now due to its uniqueness within security issues

**Calculate returns.**
- Utilising the year-month entries of each security issue

#### Can skip

Check the number of year-month data records for each issue

In [1127]:
na_returns_issue_yms = na_returns_df.reset_index(
).groupby(
    ["gvkey", "iid"]
).agg(
    {
        "data_ym": lambda vals: vals.count()
    }
)

na_returns_issue_yms

data_ym
gvkey  iid         
100001 91        32
100012 01        32
100013 90        32
100022 01         2
       02        16
...             ...
350952 90        31
351491 01        31
351590 01        18
       90        32
353444 90        25

[2172 rows x 1 columns]

We can see there are 2204 issues from <1900 companies with varying numbers of 
data points.

In [1128]:
na_returns_df.xs((100022, "01"), level=("gvkey", "iid"))

,datadate,conm,ajexm,curcdm,prccm,trfm,cshom
data_ym,,,,,,,
2022-07,2022-07-31,BAYER MOTOREN WERKE AG,1.0,USD,73.8,1.0,59404000.0
2024-02,2024-02-29,BAYER MOTOREN WERKE AG,1.0,USD,109.5,1.0,60844000.0


We also observe that the year-month entries of security issues can be 
non-contiguous when they are incomplete.

In [1129]:
na_returns_max_issue_yms = na_returns_issue_yms["data_ym"].max()
print(f"Maximum issue data points: {na_returns_max_issue_yms}")

na_returns_full_issue = na_returns_issue_yms[
    na_returns_issue_yms["data_ym"] == na_returns_max_issue_yms
]

na_returns_full_issue

Maximum issue data points: 32


,,data_ym
gvkey,iid,
100001,91,32
100012,01,32
100013,90,32
100022,90,32
100045,90,32
...,...,...
347328,90,32
347708,90,32
349485,90,32


We observe there are fewer issues with the maximum number of data points.

#### Continue

Calculate returns as per the Compustat manual. To do this we first need 
adjusted close.

In [30]:
m_returns["adjclose"] = (m_returns["prccm"] / m_returns["ajexm"]) * m_returns["trfm"]

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2014-01 2014-01-31                    AAR CORP    USD    1.0   
           2014-02 2014-02-28                    AAR CORP    USD    1.0   
           2014-03 2014-03-31                    AAR CORP    USD    1.0   
           2014-04 2014-04-30                    AAR CORP    USD    1.0   
           2014-05 2014-05-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom    prccm      trfm     adjclose  
gvkey  iid data_ym                                               
1004   01  2014-01   39600000.0    26.65  1.623944    43.278106  
           2014-02   39600000.0    28.90  1.623944    46.931980  
           2014-03   39569000.0    25.95  1.623944    42.141346  
           2014-04   39569000.0    25.90  1.628586    42.180366  
           2014-05   39569000.0    24.30  1.628586    39.574629  
...                         ...      ...       ...          ...  
362705 01W 2024-08   26460000.0     2.82  1.147992     3.237339  
       02W 2024-08   26460000.0     2.63  1.147992     3.019220  
362758 01W 2024-08  154500000.0     0.22  1.147992     0.252558  
362761 01W 2024-08  157393988.0  1010.75  1.147992  1160.333318  
362779 01W 2024-08   12388500.0     3.58  1.145429     4.100637  

[3478493 rows x 8 columns]

Exchange rates to the home currency (USD) are joined to allow computation of FX 
returns - that form a part of the overall return.

In [31]:
m_returns = m_returns.reset_index().merge(
    m_exrts, 
    how="left", 
    left_on=["curcdm", "data_ym"], 
    right_index=True, 
).set_index(
    ["gvkey", "iid", "data_ym"]
)
m_returns = m_returns.drop(columns="exratd_toGBP")
m_returns = m_returns.dropna()

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2014-01 2014-01-31                    AAR CORP    USD    1.0   
           2014-02 2014-02-28                    AAR CORP    USD    1.0   
           2014-03 2014-03-31                    AAR CORP    USD    1.0   
           2014-04 2014-04-30                    AAR CORP    USD    1.0   
           2014-05 2014-05-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom    prccm      trfm     adjclose  exratd_toUSD  
gvkey  iid data_ym                                                             
1004   01  2014-01   39600000.0    26.65  1.623944    43.278106      1.000000  
           2014-02   39600000.0    28.90  1.623944    46.931980      1.000000  
           2014-03   39569000.0    25.95  1.623944    42.141346      1.000000  
           2014-04   39569000.0    25.90  1.628586    42.180366      1.000000  
           2014-05   39569000.0    24.30  1.628586    39.574629      1.000000  
...                         ...      ...       ...          ...           ...  
362705 01W 2024-08   26460000.0     2.82  1.147992     3.237339      1.113108  
       02W 2024-08   26460000.0     2.63  1.147992     3.019220      1.113108  
362758 01W 2024-08  154500000.0     0.22  1.147992     0.252558      0.675052  
362761 01W 2024-08  157393988.0  1010.75  1.147992  1160.333318      0.011920  
362779 01W 2024-08   12388500.0     3.58  1.145429     4.100637      1.113108  

[3478493 rows x 9 columns]

From the adjusted closes and exchange rates, we can now compute the local and FX returns.

In [32]:
m_returns = pd.concat(
    [
        m_returns, 
        m_returns.groupby(
            level=["gvkey", "iid"]
        )[["adjclose", "exratd_toUSD"]].pct_change(
        ).rename(
            columns={
                "adjclose": "local_ret", 
                "exratd_toUSD": "USD_fxret", 
            }
        )
    ], 
    axis=1
)

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2014-01 2014-01-31                    AAR CORP    USD    1.0   
           2014-02 2014-02-28                    AAR CORP    USD    1.0   
           2014-03 2014-03-31                    AAR CORP    USD    1.0   
           2014-04 2014-04-30                    AAR CORP    USD    1.0   
           2014-05 2014-05-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom    prccm      trfm     adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                              
1004   01  2014-01   39600000.0    26.65  1.623944    43.278106      1.000000   
           2014-02   39600000.0    28.90  1.623944    46.931980      1.000000   
           2014-03   39569000.0    25.95  1.623944    42.141346      1.000000   
           2014-04   39569000.0    25.90  1.628586    42.180366      1.000000   
           2014-05   39569000.0    24.30  1.628586    39.574629      1.000000   
...                         ...      ...       ...          ...           ...   
362705 01W 2024-08   26460000.0     2.82  1.147992     3.237339      1.113108   
       02W 2024-08   26460000.0     2.63  1.147992     3.019220      1.113108   
362758 01W 2024-08  154500000.0     0.22  1.147992     0.252558      0.675052   
362761 01W 2024-08  157393988.0  1010.75  1.147992  1160.333318      0.011920   
362779 01W 2024-08   12388500.0     3.58  1.145429     4.100637      1.113108   

                    local_ret  USD_fxret  
gvkey  iid data_ym                        
1004   01  2014-01        NaN        NaN  
           2014-02   0.084428   0.000000  
           2014-03  -0.102076   0.000000  
           2014-04   0.000926   0.000000  
           2014-05  -0.061776   0.000000  
...                       ...        ...  
362705 01W 2024-08  -0.066225   0.028504  
       02W 2024-08        NaN        NaN  
362758 01W 2024-08        NaN        NaN  
362761 01W 2024-08        NaN        NaN  
362779 01W 2024-08        NaN        NaN  

[3478493 rows x 11 columns]

Combine local and FX returns to obtain USD returns. Note that these are 
raw returns with varying frequencies.

In [33]:
m_returns["USD_ret"] = (
    (1 + m_returns["local_ret"]) * (1 + m_returns["USD_fxret"]) - 1
)

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2014-01 2014-01-31                    AAR CORP    USD    1.0   
           2014-02 2014-02-28                    AAR CORP    USD    1.0   
           2014-03 2014-03-31                    AAR CORP    USD    1.0   
           2014-04 2014-04-30                    AAR CORP    USD    1.0   
           2014-05 2014-05-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom    prccm      trfm     adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                              
1004   01  2014-01   39600000.0    26.65  1.623944    43.278106      1.000000   
           2014-02   39600000.0    28.90  1.623944    46.931980      1.000000   
           2014-03   39569000.0    25.95  1.623944    42.141346      1.000000   
           2014-04   39569000.0    25.90  1.628586    42.180366      1.000000   
           2014-05   39569000.0    24.30  1.628586    39.574629      1.000000   
...                         ...      ...       ...          ...           ...   
362705 01W 2024-08   26460000.0     2.82  1.147992     3.237339      1.113108   
       02W 2024-08   26460000.0     2.63  1.147992     3.019220      1.113108   
362758 01W 2024-08  154500000.0     0.22  1.147992     0.252558      0.675052   
362761 01W 2024-08  157393988.0  1010.75  1.147992  1160.333318      0.011920   
362779 01W 2024-08   12388500.0     3.58  1.145429     4.100637      1.113108   

                    local_ret  USD_fxret   USD_ret  
gvkey  iid data_ym                                  
1004   01  2014-01        NaN        NaN       NaN  
           2014-02   0.084428   0.000000  0.084428  
           2014-03  -0.102076   0.000000 -0.102076  
           2014-04   0.000926   0.000000  0.000926  
           2014-05  -0.061776   0.000000 -0.061776  
...                       ...        ...       ...  
362705 01W 2024-08  -0.066225   0.028504 -0.039608  
       02W 2024-08        NaN        NaN       NaN  
362758 01W 2024-08        NaN        NaN       NaN  
362761 01W 2024-08        NaN        NaN       NaN  
362779 01W 2024-08        NaN        NaN       NaN  

[3478493 rows x 12 columns]

The differences in months of these raw returns can be used to standardise them 
to a common frequency (monthly).

In [34]:
""" Differences in months, that can be used to standardise returns to the 
same frequency - monthly """
m_returns["ret_mspan"] = m_returns.reset_index(
).groupby(
    ["gvkey", "iid"]
)["data_ym"].diff(
).apply(
    lambda x: x.n if pd.notnull(x) else np.nan
).values

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2014-01 2014-01-31                    AAR CORP    USD    1.0   
           2014-02 2014-02-28                    AAR CORP    USD    1.0   
           2014-03 2014-03-31                    AAR CORP    USD    1.0   
           2014-04 2014-04-30                    AAR CORP    USD    1.0   
           2014-05 2014-05-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom    prccm      trfm     adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                              
1004   01  2014-01   39600000.0    26.65  1.623944    43.278106      1.000000   
           2014-02   39600000.0    28.90  1.623944    46.931980      1.000000   
           2014-03   39569000.0    25.95  1.623944    42.141346      1.000000   
           2014-04   39569000.0    25.90  1.628586    42.180366      1.000000   
           2014-05   39569000.0    24.30  1.628586    39.574629      1.000000   
...                         ...      ...       ...          ...           ...   
362705 01W 2024-08   26460000.0     2.82  1.147992     3.237339      1.113108   
       02W 2024-08   26460000.0     2.63  1.147992     3.019220      1.113108   
362758 01W 2024-08  154500000.0     0.22  1.147992     0.252558      0.675052   
362761 01W 2024-08  157393988.0  1010.75  1.147992  1160.333318      0.011920   
362779 01W 2024-08   12388500.0     3.58  1.145429     4.100637      1.113108   

                    local_ret  USD_fxret   USD_ret  ret_mspan  
gvkey  iid data_ym                                             
1004   01  2014-01        NaN        NaN       NaN        NaN  
           2014-02   0.084428   0.000000  0.084428        1.0  
           2014-03  -0.102076   0.000000 -0.102076        1.0  
           2014-04   0.000926   0.000000  0.000926        1.0  
           2014-05  -0.061776   0.000000 -0.061776        1.0  
...                       ...        ...       ...        ...  
362705 01W 2024-08  -0.066225   0.028504 -0.039608        1.0  
       02W 2024-08        NaN        NaN       NaN        NaN  
362758 01W 2024-08        NaN        NaN       NaN        NaN  
362761 01W 2024-08        NaN        NaN       NaN        NaN  
362779 01W 2024-08        NaN        NaN       NaN        NaN  

[3478493 rows x 13 columns]

*Sanity check - there should be 0 return month frequencies that 
are non-null for the first year-month entries of issues*

In [35]:
pd.notnull(m_returns.groupby(
    level=["gvkey", "iid"]
).agg(
    {
        "ret_mspan": lambda vals: vals.iloc[0]
    }
)).sum()

ret_mspan    0
dtype: int64

Standardise the raw USD returns to the monthly frequency.

In [36]:
def compute_m_returns(df, rr_label, rspan_label="ret_mspan"):
    df[f"m_{rr_label}"] = (
        (1 + df[rr_label]) ** (1/df[rspan_label])
    ) - 1

compute_m_returns(m_returns, "USD_ret")
compute_m_returns(m_returns, "local_ret")

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2014-01 2014-01-31                    AAR CORP    USD    1.0   
           2014-02 2014-02-28                    AAR CORP    USD    1.0   
           2014-03 2014-03-31                    AAR CORP    USD    1.0   
           2014-04 2014-04-30                    AAR CORP    USD    1.0   
           2014-05 2014-05-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom    prccm      trfm     adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                              
1004   01  2014-01   39600000.0    26.65  1.623944    43.278106      1.000000   
           2014-02   39600000.0    28.90  1.623944    46.931980      1.000000   
           2014-03   39569000.0    25.95  1.623944    42.141346      1.000000   
           2014-04   39569000.0    25.90  1.628586    42.180366      1.000000   
           2014-05   39569000.0    24.30  1.628586    39.574629      1.000000   
...                         ...      ...       ...          ...           ...   
362705 01W 2024-08   26460000.0     2.82  1.147992     3.237339      1.113108   
       02W 2024-08   26460000.0     2.63  1.147992     3.019220      1.113108   
362758 01W 2024-08  154500000.0     0.22  1.147992     0.252558      0.675052   
362761 01W 2024-08  157393988.0  1010.75  1.147992  1160.333318      0.011920   
362779 01W 2024-08   12388500.0     3.58  1.145429     4.100637      1.113108   

                    local_ret  USD_fxret   USD_ret  ret_mspan  m_USD_ret  \
gvkey  iid data_ym                                                         
1004   01  2014-01        NaN        NaN       NaN        NaN        NaN   
           2014-02   0.084428   0.000000  0.084428        1.0   0.084428   
           2014-03  -0.102076   0.000000 -0.102076        1.0  -0.102076   
           2014-04   0.000926   0.000000  0.000926        1.0   0.000926   
           2014-05  -0.061776   0.000000 -0.061776        1.0  -0.061776   
...                       ...        ...       ...        ...        ...   
362705 01W 2024-08  -0.066225   0.028504 -0.039608        1.0  -0.039608   
       02W 2024-08        NaN        NaN       NaN        NaN        NaN   
362758 01W 2024-08        NaN        NaN       NaN        NaN        NaN   
362761 01W 2024-08        NaN        NaN       NaN        NaN        NaN   
362779 01W 2024-08        NaN        NaN       NaN        NaN        NaN   

                    m_local_ret  
gvkey  iid data_ym               
1004   01  2014-01          NaN  
           2014-02     0.084428  
           2014-03    -0.102076  
           2014-04     0.000926  
           2014-05    -0.061776  
...                         ...  
362705 01W 2024-08    -0.066225  
       02W 2024-08          NaN  
362758 01W 2024-08          NaN  
362761 01W 2024-08          NaN  
362779 01W 2024-08          NaN  

[3478493 rows x 15 columns]

*Notice that there are companies that have gone bankrupt (0 raw close price & 
0 adjusted close price), which leads to further null returns beyond just the 
first month for some companies (due to division by 0).*

In [37]:
m_returns = m_returns.copy()

At this point, the computation of monthly returns has been completed.
- w.r.t base currencies of USD and GBP.

**Calculate monthly market capitalisation**
- standardise the currency (USD)

First, per security

In [38]:
m_returns["USD_secval"] = m_returns["cshom"] * m_returns["prccm"] * m_returns["exratd_toUSD"]

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2014-01 2014-01-31                    AAR CORP    USD    1.0   
           2014-02 2014-02-28                    AAR CORP    USD    1.0   
           2014-03 2014-03-31                    AAR CORP    USD    1.0   
           2014-04 2014-04-30                    AAR CORP    USD    1.0   
           2014-05 2014-05-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom    prccm      trfm     adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                              
1004   01  2014-01   39600000.0    26.65  1.623944    43.278106      1.000000   
           2014-02   39600000.0    28.90  1.623944    46.931980      1.000000   
           2014-03   39569000.0    25.95  1.623944    42.141346      1.000000   
           2014-04   39569000.0    25.90  1.628586    42.180366      1.000000   
           2014-05   39569000.0    24.30  1.628586    39.574629      1.000000   
...                         ...      ...       ...          ...           ...   
362705 01W 2024-08   26460000.0     2.82  1.147992     3.237339      1.113108   
       02W 2024-08   26460000.0     2.63  1.147992     3.019220      1.113108   
362758 01W 2024-08  154500000.0     0.22  1.147992     0.252558      0.675052   
362761 01W 2024-08  157393988.0  1010.75  1.147992  1160.333318      0.011920   
362779 01W 2024-08   12388500.0     3.58  1.145429     4.100637      1.113108   

                    local_ret  USD_fxret   USD_ret  ret_mspan  m_USD_ret  \
gvkey  iid data_ym                                                         
1004   01  2014-01        NaN        NaN       NaN        NaN        NaN   
           2014-02   0.084428   0.000000  0.084428        1.0   0.084428   
           2014-03  -0.102076   0.000000 -0.102076        1.0  -0.102076   
           2014-04   0.000926   0.000000  0.000926        1.0   0.000926   
           2014-05  -0.061776   0.000000 -0.061776        1.0  -0.061776   
...                       ...        ...       ...        ...        ...   
362705 01W 2024-08  -0.066225   0.028504 -0.039608        1.0  -0.039608   
       02W 2024-08        NaN        NaN       NaN        NaN        NaN   
362758 01W 2024-08        NaN        NaN       NaN        NaN        NaN   
362761 01W 2024-08        NaN        NaN       NaN        NaN        NaN   
362779 01W 2024-08        NaN        NaN       NaN        NaN        NaN   

                    m_local_ret    USD_secval  
gvkey  iid data_ym                             
1004   01  2014-01          NaN  1.055340e+09  
           2014-02     0.084428  1.144440e+09  
           2014-03    -0.102076  1.026816e+09  
           2014-04     0.000926  1.024837e+09  
           2014-05    -0.061776  9.615267e+08  
...                         ...           ...  
362705 01W 2024-08    -0.066225  8.305700e+07  
       02W 2024-08          NaN  7.746096e+07  
362758 01W 2024-08          NaN  2.294501e+07  
362761 01W 2024-08          NaN  1.896297e+09  
362779 01W 2024-08          NaN  4.936726e+07  

[3478493 rows x 16 columns]

Now aggregate by gvkey and year-month to determine the total market value from 
all the securities.

In [39]:
m_returns = m_returns.join(
    m_returns.groupby(
        level=["gvkey", "data_ym"]
    )["USD_secval"].sum().rename("USD_mktval"), 
    on=["gvkey", "data_ym"]
)

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2014-01 2014-01-31                    AAR CORP    USD    1.0   
           2014-02 2014-02-28                    AAR CORP    USD    1.0   
           2014-03 2014-03-31                    AAR CORP    USD    1.0   
           2014-04 2014-04-30                    AAR CORP    USD    1.0   
           2014-05 2014-05-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom    prccm      trfm     adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                              
1004   01  2014-01   39600000.0    26.65  1.623944    43.278106      1.000000   
           2014-02   39600000.0    28.90  1.623944    46.931980      1.000000   
           2014-03   39569000.0    25.95  1.623944    42.141346      1.000000   
           2014-04   39569000.0    25.90  1.628586    42.180366      1.000000   
           2014-05   39569000.0    24.30  1.628586    39.574629      1.000000   
...                         ...      ...       ...          ...           ...   
362705 01W 2024-08   26460000.0     2.82  1.147992     3.237339      1.113108   
       02W 2024-08   26460000.0     2.63  1.147992     3.019220      1.113108   
362758 01W 2024-08  154500000.0     0.22  1.147992     0.252558      0.675052   
362761 01W 2024-08  157393988.0  1010.75  1.147992  1160.333318      0.011920   
362779 01W 2024-08   12388500.0     3.58  1.145429     4.100637      1.113108   

                    local_ret  USD_fxret   USD_ret  ret_mspan  m_USD_ret  \
gvkey  iid data_ym                                                         
1004   01  2014-01        NaN        NaN       NaN        NaN        NaN   
           2014-02   0.084428   0.000000  0.084428        1.0   0.084428   
           2014-03  -0.102076   0.000000 -0.102076        1.0  -0.102076   
           2014-04   0.000926   0.000000  0.000926        1.0   0.000926   
           2014-05  -0.061776   0.000000 -0.061776        1.0  -0.061776   
...                       ...        ...       ...        ...        ...   
362705 01W 2024-08  -0.066225   0.028504 -0.039608        1.0  -0.039608   
       02W 2024-08        NaN        NaN       NaN        NaN        NaN   
362758 01W 2024-08        NaN        NaN       NaN        NaN        NaN   
362761 01W 2024-08        NaN        NaN       NaN        NaN        NaN   
362779 01W 2024-08        NaN        NaN       NaN        NaN        NaN   

                    m_local_ret    USD_secval    USD_mktval  
gvkey  iid data_ym                                           
1004   01  2014-01          NaN  1.055340e+09  1.055340e+09  
           2014-02     0.084428  1.144440e+09  1.144440e+09  
           2014-03    -0.102076  1.026816e+09  1.026816e+09  
           2014-04     0.000926  1.024837e+09  1.024837e+09  
           2014-05    -0.061776  9.615267e+08  9.615267e+08  
...                         ...           ...           ...  
362705 01W 2024-08    -0.066225  8.305700e+07  1.605180e+08  
       02W 2024-08          NaN  7.746096e+07  1.605180e+08  
362758 01W 2024-08          NaN  2.294501e+07  2.294501e+07  
362761 01W 2024-08          NaN  1.896297e+09  1.896297e+09  
362779 01W 2024-08          NaN  4.936726e+07  4.936726e+07  

[3478493 rows x 17 columns]

Shift the calculated market values one forward, in each security, to prevent look-ahead bias in return predictions.

In [40]:
m_returns[
    "X_USD_mktval"
] = m_returns.groupby(
    level=["gvkey", "iid"]
)[
    "USD_mktval"
].shift()

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2014-01 2014-01-31                    AAR CORP    USD    1.0   
           2014-02 2014-02-28                    AAR CORP    USD    1.0   
           2014-03 2014-03-31                    AAR CORP    USD    1.0   
           2014-04 2014-04-30                    AAR CORP    USD    1.0   
           2014-05 2014-05-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom    prccm      trfm     adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                              
1004   01  2014-01   39600000.0    26.65  1.623944    43.278106      1.000000   
           2014-02   39600000.0    28.90  1.623944    46.931980      1.000000   
           2014-03   39569000.0    25.95  1.623944    42.141346      1.000000   
           2014-04   39569000.0    25.90  1.628586    42.180366      1.000000   
           2014-05   39569000.0    24.30  1.628586    39.574629      1.000000   
...                         ...      ...       ...          ...           ...   
362705 01W 2024-08   26460000.0     2.82  1.147992     3.237339      1.113108   
       02W 2024-08   26460000.0     2.63  1.147992     3.019220      1.113108   
362758 01W 2024-08  154500000.0     0.22  1.147992     0.252558      0.675052   
362761 01W 2024-08  157393988.0  1010.75  1.147992  1160.333318      0.011920   
362779 01W 2024-08   12388500.0     3.58  1.145429     4.100637      1.113108   

                    local_ret  USD_fxret   USD_ret  ret_mspan  m_USD_ret  \
gvkey  iid data_ym                                                         
1004   01  2014-01        NaN        NaN       NaN        NaN        NaN   
           2014-02   0.084428   0.000000  0.084428        1.0   0.084428   
           2014-03  -0.102076   0.000000 -0.102076        1.0  -0.102076   
           2014-04   0.000926   0.000000  0.000926        1.0   0.000926   
           2014-05  -0.061776   0.000000 -0.061776        1.0  -0.061776   
...                       ...        ...       ...        ...        ...   
362705 01W 2024-08  -0.066225   0.028504 -0.039608        1.0  -0.039608   
       02W 2024-08        NaN        NaN       NaN        NaN        NaN   
362758 01W 2024-08        NaN        NaN       NaN        NaN        NaN   
362761 01W 2024-08        NaN        NaN       NaN        NaN        NaN   
362779 01W 2024-08        NaN        NaN       NaN        NaN        NaN   

                    m_local_ret    USD_secval    USD_mktval  X_USD_mktval  
gvkey  iid data_ym                                                         
1004   01  2014-01          NaN  1.055340e+09  1.055340e+09           NaN  
           2014-02     0.084428  1.144440e+09  1.144440e+09  1.055340e+09  
           2014-03    -0.102076  1.026816e+09  1.026816e+09  1.144440e+09  
           2014-04     0.000926  1.024837e+09  1.024837e+09  1.026816e+09  
           2014-05    -0.061776  9.615267e+08  9.615267e+08  1.024837e+09  
...                         ...           ...           ...           ...  
362705 01W 2024-08    -0.066225  8.305700e+07  1.605180e+08  8.648243e+07  
       02W 2024-08          NaN  7.746096e+07  1.605180e+08           NaN  
362758 01W 2024-08          NaN  2.294501e+07  2.294501e+07           NaN  
362761 01W 2024-08          NaN  1.896297e+09  1.896297e+09           NaN  
362779 01W 2024-08          NaN  4.936726e+07  4.936726e+07           NaN  

[3478493 rows

**Finalise**

Returns from fractions to percentages.

In [41]:
m_returns["local_ret"] *= 100
m_returns["USD_fxret"] *= 100
# 
m_returns["USD_ret"] *= 100
# 
m_returns["m_USD_ret"] *= 100
m_returns["m_local_ret"] *= 100

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2014-01 2014-01-31                    AAR CORP    USD    1.0   
           2014-02 2014-02-28                    AAR CORP    USD    1.0   
           2014-03 2014-03-31                    AAR CORP    USD    1.0   
           2014-04 2014-04-30                    AAR CORP    USD    1.0   
           2014-05 2014-05-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom    prccm      trfm     adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                              
1004   01  2014-01   39600000.0    26.65  1.623944    43.278106      1.000000   
           2014-02   39600000.0    28.90  1.623944    46.931980      1.000000   
           2014-03   39569000.0    25.95  1.623944    42.141346      1.000000   
           2014-04   39569000.0    25.90  1.628586    42.180366      1.000000   
           2014-05   39569000.0    24.30  1.628586    39.574629      1.000000   
...                         ...      ...       ...          ...           ...   
362705 01W 2024-08   26460000.0     2.82  1.147992     3.237339      1.113108   
       02W 2024-08   26460000.0     2.63  1.147992     3.019220      1.113108   
362758 01W 2024-08  154500000.0     0.22  1.147992     0.252558      0.675052   
362761 01W 2024-08  157393988.0  1010.75  1.147992  1160.333318      0.011920   
362779 01W 2024-08   12388500.0     3.58  1.145429     4.100637      1.113108   

                    local_ret  USD_fxret    USD_ret  ret_mspan  m_USD_ret  \
gvkey  iid data_ym                                                          
1004   01  2014-01        NaN        NaN        NaN        NaN        NaN   
           2014-02   8.442777   0.000000   8.442777        1.0   8.442777   
           2014-03 -10.207612   0.000000 -10.207612        1.0 -10.207612   
           2014-04   0.092594   0.000000   0.092594        1.0   0.092594   
           2014-05  -6.177606   0.000000  -6.177606        1.0  -6.177606   
...                       ...        ...        ...        ...        ...   
362705 01W 2024-08  -6.622517   2.850443  -3.960845        1.0  -3.960845   
       02W 2024-08        NaN        NaN        NaN        NaN        NaN   
362758 01W 2024-08        NaN        NaN        NaN        NaN        NaN   
362761 01W 2024-08        NaN        NaN        NaN        NaN        NaN   
362779 01W 2024-08        NaN        NaN        NaN        NaN        NaN   

                    m_local_ret    USD_secval    USD_mktval  X_USD_mktval  
gvkey  iid data_ym                                                         
1004   01  2014-01          NaN  1.055340e+09  1.055340e+09           NaN  
           2014-02     8.442777  1.144440e+09  1.144440e+09  1.055340e+09  
           2014-03   -10.207612  1.026816e+09  1.026816e+09  1.144440e+09  
           2014-04     0.092594  1.024837e+09  1.024837e+09  1.026816e+09  
           2014-05    -6.177606  9.615267e+08  9.615267e+08  1.024837e+09  
...                         ...           ...           ...           ...  
362705 01W 2024-08    -6.622517  8.305700e+07  1.605180e+08  8.648243e+07  
       02W 2024-08          NaN  7.746096e+07  1.605180e+08           NaN  
362758 01W 2024-08          NaN  2.294501e+07  2.294501e+07           NaN  
362761 01W 2024-08          NaN  1.896297e+09  1.896297e+09           NaN  
362779 01W 2024-08          NaN  4.936726e+07  4.936726e+07           NaN  



Augment with year columns for linking with fundamentals and emissions data.

In [42]:
m_returns["datayear"] = m_returns.reset_index()["data_ym"].dt.year.values
m_returns["datayear-1"] = m_returns["datayear"] - 1

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2014-01 2014-01-31                    AAR CORP    USD    1.0   
           2014-02 2014-02-28                    AAR CORP    USD    1.0   
           2014-03 2014-03-31                    AAR CORP    USD    1.0   
           2014-04 2014-04-30                    AAR CORP    USD    1.0   
           2014-05 2014-05-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom    prccm      trfm     adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                              
1004   01  2014-01   39600000.0    26.65  1.623944    43.278106      1.000000   
           2014-02   39600000.0    28.90  1.623944    46.931980      1.000000   
           2014-03   39569000.0    25.95  1.623944    42.141346      1.000000   
           2014-04   39569000.0    25.90  1.628586    42.180366      1.000000   
           2014-05   39569000.0    24.30  1.628586    39.574629      1.000000   
...                         ...      ...       ...          ...           ...   
362705 01W 2024-08   26460000.0     2.82  1.147992     3.237339      1.113108   
       02W 2024-08   26460000.0     2.63  1.147992     3.019220      1.113108   
362758 01W 2024-08  154500000.0     0.22  1.147992     0.252558      0.675052   
362761 01W 2024-08  157393988.0  1010.75  1.147992  1160.333318      0.011920   
362779 01W 2024-08   12388500.0     3.58  1.145429     4.100637      1.113108   

                    local_ret  USD_fxret    USD_ret  ret_mspan  m_USD_ret  \
gvkey  iid data_ym                                                          
1004   01  2014-01        NaN        NaN        NaN        NaN        NaN   
           2014-02   8.442777   0.000000   8.442777        1.0   8.442777   
           2014-03 -10.207612   0.000000 -10.207612        1.0 -10.207612   
           2014-04   0.092594   0.000000   0.092594        1.0   0.092594   
           2014-05  -6.177606   0.000000  -6.177606        1.0  -6.177606   
...                       ...        ...        ...        ...        ...   
362705 01W 2024-08  -6.622517   2.850443  -3.960845        1.0  -3.960845   
       02W 2024-08        NaN        NaN        NaN        NaN        NaN   
362758 01W 2024-08        NaN        NaN        NaN        NaN        NaN   
362761 01W 2024-08        NaN        NaN        NaN        NaN        NaN   
362779 01W 2024-08        NaN        NaN        NaN        NaN        NaN   

                    m_local_ret    USD_secval    USD_mktval  X_USD_mktval  \
gvkey  iid data_ym                                                          
1004   01  2014-01          NaN  1.055340e+09  1.055340e+09           NaN   
           2014-02     8.442777  1.144440e+09  1.144440e+09  1.055340e+09   
           2014-03   -10.207612  1.026816e+09  1.026816e+09  1.144440e+09   
           2014-04     0.092594  1.024837e+09  1.024837e+09  1.026816e+09   
           2014-05    -6.177606  9.615267e+08  9.615267e+08  1.024837e+09   
...                         ...           ...           ...           ...   
362705 01W 2024-08    -6.622517  8.305700e+07  1.605180e+08  8.648243e+07   
       02W 2024-08          NaN  7.746096e+07  1.605180e+08           NaN   
362758 01W 2024-08          NaN  2.294501e+07  2.294501e+07           NaN   
362761 01W 2024-08          NaN  1.896297e+09  1.896297e+09           NaN   
362779 01W 2024-08          NaN  4.936726e+07  4.936726e+07      

Join market return in.

In [43]:
m_returns = m_returns.join(
    mkt_df, 
    how="left", 
    on="data_ym", 
)

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2014-01 2014-01-31                    AAR CORP    USD    1.0   
           2014-02 2014-02-28                    AAR CORP    USD    1.0   
           2014-03 2014-03-31                    AAR CORP    USD    1.0   
           2014-04 2014-04-30                    AAR CORP    USD    1.0   
           2014-05 2014-05-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom    prccm      trfm     adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                              
1004   01  2014-01   39600000.0    26.65  1.623944    43.278106      1.000000   
           2014-02   39600000.0    28.90  1.623944    46.931980      1.000000   
           2014-03   39569000.0    25.95  1.623944    42.141346      1.000000   
           2014-04   39569000.0    25.90  1.628586    42.180366      1.000000   
           2014-05   39569000.0    24.30  1.628586    39.574629      1.000000   
...                         ...      ...       ...          ...           ...   
362705 01W 2024-08   26460000.0     2.82  1.147992     3.237339      1.113108   
       02W 2024-08   26460000.0     2.63  1.147992     3.019220      1.113108   
362758 01W 2024-08  154500000.0     0.22  1.147992     0.252558      0.675052   
362761 01W 2024-08  157393988.0  1010.75  1.147992  1160.333318      0.011920   
362779 01W 2024-08   12388500.0     3.58  1.145429     4.100637      1.113108   

                    local_ret  ...    USD_ret  ret_mspan  m_USD_ret  \
gvkey  iid data_ym             ...                                    
1004   01  2014-01        NaN  ...        NaN        NaN        NaN   
           2014-02   8.442777  ...   8.442777        1.0   8.442777   
           2014-03 -10.207612  ... -10.207612        1.0 -10.207612   
           2014-04   0.092594  ...   0.092594        1.0   0.092594   
           2014-05  -6.177606  ...  -6.177606        1.0  -6.177606   
...                       ...  ...        ...        ...        ...   
362705 01W 2024-08  -6.622517  ...  -3.960845        1.0  -3.960845   
       02W 2024-08        NaN  ...        NaN        NaN        NaN   
362758 01W 2024-08        NaN  ...        NaN        NaN        NaN   
362761 01W 2024-08        NaN  ...        NaN        NaN        NaN   
362779 01W 2024-08        NaN  ...        NaN        NaN        NaN   

                    m_local_ret    USD_secval    USD_mktval  X_USD_mktval  \
gvkey  iid data_ym                                                          
1004   01  2014-01          NaN  1.055340e+09  1.055340e+09           NaN   
           2014-02     8.442777  1.144440e+09  1.144440e+09  1.055340e+09   
           2014-03   -10.207612  1.026816e+09  1.026816e+09  1.144440e+09   
           2014-04     0.092594  1.024837e+09  1.024837e+09  1.026816e+09   
           2014-05    -6.177606  9.615267e+08  9.615267e+08  1.024837e+09   
...                         ...           ...           ...           ...   
362705 01W 2024-08    -6.622517  8.305700e+07  1.605180e+08  8.648243e+07   
       02W 2024-08          NaN  7.746096e+07  1.605180e+08           NaN   
362758 01W 2024-08          NaN  2.294501e+07  2.294501e+07           NaN   
362761 01W 2024-08          NaN  1.896297e+09  1.896297e+09           NaN   
362779 01W 2024-08          NaN  4.936726e+07  4.936726e+07           NaN   

                    datayear  datayear-1  m_mktret  
gvkey  iid d

Compute rolling beta within issues.
- From rolling covariances and variances.

In [44]:
m_return_betas = m_returns.reset_index(
).groupby(
    ["gvkey", "iid"] # within issues
)[["m_USD_ret", "m_mktret"]].rolling(
    12
).cov().unstack()["m_mktret"].rename(
    columns={
        "m_USD_ret": "cov(i,m)", 
        "m_mktret": "var(m)", 
    }
).set_index(m_returns.index)

m_return_betas

cov(i,m)  var(m)
gvkey  iid data_ym                  
1004   01  2014-01       NaN     NaN
           2014-02       NaN     NaN
           2014-03       NaN     NaN
           2014-04       NaN     NaN
           2014-05       NaN     NaN
...                      ...     ...
362705 01W 2024-08       NaN     NaN
       02W 2024-08       NaN     NaN
362758 01W 2024-08       NaN     NaN
362761 01W 2024-08       NaN     NaN
362779 01W 2024-08       NaN     NaN

[3478493 rows x 2 columns]

Once again shift the calculated betas one forward, in each security, to prevent look-ahead bias.

In [45]:
m_return_betas["beta"] = m_return_betas["cov(i,m)"] / m_return_betas["var(m)"]
m_return_betas = m_return_betas.drop(columns=["cov(i,m)", "var(m)"])

m_return_betas["X_beta"] = m_return_betas.groupby(
    level=["gvkey", "iid"]
).shift()

m_return_betas

beta  X_beta
gvkey  iid data_ym              
1004   01  2014-01   NaN     NaN
           2014-02   NaN     NaN
           2014-03   NaN     NaN
           2014-04   NaN     NaN
           2014-05   NaN     NaN
...                  ...     ...
362705 01W 2024-08   NaN     NaN
       02W 2024-08   NaN     NaN
362758 01W 2024-08   NaN     NaN
362761 01W 2024-08   NaN     NaN
362779 01W 2024-08   NaN     NaN

[3478493 rows x 2 columns]

Beta is concatenated together with the returns, aligning with the index.

In [46]:
m_returns = pd.concat([m_returns, m_return_betas], axis=1)

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2014-01 2014-01-31                    AAR CORP    USD    1.0   
           2014-02 2014-02-28                    AAR CORP    USD    1.0   
           2014-03 2014-03-31                    AAR CORP    USD    1.0   
           2014-04 2014-04-30                    AAR CORP    USD    1.0   
           2014-05 2014-05-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom    prccm      trfm     adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                              
1004   01  2014-01   39600000.0    26.65  1.623944    43.278106      1.000000   
           2014-02   39600000.0    28.90  1.623944    46.931980      1.000000   
           2014-03   39569000.0    25.95  1.623944    42.141346      1.000000   
           2014-04   39569000.0    25.90  1.628586    42.180366      1.000000   
           2014-05   39569000.0    24.30  1.628586    39.574629      1.000000   
...                         ...      ...       ...          ...           ...   
362705 01W 2024-08   26460000.0     2.82  1.147992     3.237339      1.113108   
       02W 2024-08   26460000.0     2.63  1.147992     3.019220      1.113108   
362758 01W 2024-08  154500000.0     0.22  1.147992     0.252558      0.675052   
362761 01W 2024-08  157393988.0  1010.75  1.147992  1160.333318      0.011920   
362779 01W 2024-08   12388500.0     3.58  1.145429     4.100637      1.113108   

                    local_ret  ...  m_USD_ret  m_local_ret    USD_secval  \
gvkey  iid data_ym             ...                                         
1004   01  2014-01        NaN  ...        NaN          NaN  1.055340e+09   
           2014-02   8.442777  ...   8.442777     8.442777  1.144440e+09   
           2014-03 -10.207612  ... -10.207612   -10.207612  1.026816e+09   
           2014-04   0.092594  ...   0.092594     0.092594  1.024837e+09   
           2014-05  -6.177606  ...  -6.177606    -6.177606  9.615267e+08   
...                       ...  ...        ...          ...           ...   
362705 01W 2024-08  -6.622517  ...  -3.960845    -6.622517  8.305700e+07   
       02W 2024-08        NaN  ...        NaN          NaN  7.746096e+07   
362758 01W 2024-08        NaN  ...        NaN          NaN  2.294501e+07   
362761 01W 2024-08        NaN  ...        NaN          NaN  1.896297e+09   
362779 01W 2024-08        NaN  ...        NaN          NaN  4.936726e+07   

                      USD_mktval  X_USD_mktval  datayear  datayear-1  \
gvkey  iid data_ym                                                     
1004   01  2014-01  1.055340e+09           NaN      2014        2013   
           2014-02  1.144440e+09  1.055340e+09      2014        2013   
           2014-03  1.026816e+09  1.144440e+09      2014        2013   
           2014-04  1.024837e+09  1.026816e+09      2014        2013   
           2014-05  9.615267e+08  1.024837e+09      2014        2013   
...                          ...           ...       ...         ...   
362705 01W 2024-08  1.605180e+08  8.648243e+07      2024        2023   
       02W 2024-08  1.605180e+08           NaN      2024        2023   
362758 01W 2024-08  2.294501e+07           NaN      2024        2023   
362761 01W 2024-08  1.896297e+09           NaN      2024        2023   
362779 01W 2024-08  4.936726e+07           NaN      2024        2023   

                    m_mktret  beta  X_beta  
gvkey  iid data_ym  

Finally, drop all rows with null values.

In [47]:
m_returns = m_returns.dropna()

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2015-02 2015-02-27                    AAR CORP    USD    1.0   
           2015-03 2015-03-31                    AAR CORP    USD    1.0   
           2015-04 2015-04-30                    AAR CORP    USD    1.0   
           2015-05 2015-05-29                    AAR CORP    USD    1.0   
           2015-06 2015-06-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
361486 01W 2024-03 2024-03-29  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2024-04 2024-04-30  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2024-05 2024-05-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2024-06 2024-06-28  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2024-07 2024-07-31  FABRICA DE SCULE RASNOV SA    RON    1.0   

                         cshom  prccm      trfm   adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                         
1004   01  2015-02  39791000.0  29.40  1.642484  48.289041      1.000000   
           2015-03  39791000.0  30.70  1.642484  50.424271      1.000000   
           2015-04  39661000.0  30.24  1.646443  49.788431      1.000000   
           2015-05  39661000.0  29.54  1.646443  48.635921      1.000000   
           2015-06  39661000.0  31.87  1.646443  52.472133      1.000000   
...                        ...    ...       ...        ...           ...   
361486 01W 2024-03   5650039.0   3.60  1.102500   3.969000      0.217178   
           2024-04   5650039.0   3.60  1.102500   3.969000      0.214867   
           2024-05   5650039.0   9.50  1.102500  10.473750      0.217907   
           2024-06   5650039.0  12.20  1.102500  13.450500      0.215265   
           2024-07   5650039.0  11.40  1.102500  12.568500      0.217489   

                     local_ret  ...   m_USD_ret  m_local_ret    USD_secval  \
gvkey  iid data_ym              ...                                          
1004   01  2015-02    2.581996  ...    2.581996     2.581996  1.169855e+09   
           2015-03    4.421769  ...    4.421769     4.421769  1.221584e+09   
           2015-04   -1.260980  ...   -1.260980    -1.260980  1.199349e+09   
           2015-05   -2.314815  ...   -2.314815    -2.314815  1.171586e+09   
           2015-06    7.887610  ...    7.887610     7.887610  1.263996e+09   
...                        ...  ...         ...          ...           ...   
361486 01W 2024-03    0.000000  ...   -0.152682     0.000000  4.417436e+06   
           2024-04    0.000000  ...   -1.064424     0.000000  4.370415e+06   
           2024-05  163.888889  ...  167.622508   163.888889  1.169622e+07   
           2024-06   28.421053  ...   26.864346    28.421053  1.483833e+07   
           2024-07   -6.557377  ...   -5.591926    -6.557377  1.400858e+07   

                      USD_mktval  X_USD_mktval  datayear  datayear-1  \
gvkey  iid data_ym                                                     
1004   01  2015-02  1.169855e+09  1.140410e+09      2015        2014   
           2015-03  1.221584e+09  1.169855e+09      2015        2014   
           2015-04  1.199349e+09  1.221584e+09      2015        2014   
           2015-05  1.171586e+09  1.199349e+09      2015        2014   
           2015-06  1.263996e+09  1.171586e+09      2015        2014   
...                          ...           ...       ...         ...   
361486 01W 2024-03  4.417436e+06  4.424191e+06      2024        2023   
           2024-04  4.370415e+06  4.417436e+06      2024        2023   
           2024-05  1.169622e+07  4.370415e+06      2024        2023   
           2024-06  1.483833e+07  1.169622e+07      2024        2023   
           2024-07  1.400858e+07  1.483833e+07      2024        2023   

                    m_mktret      beta    X_beta  
gvkey  iid data_ym                                
10

#### Optional QC

For this whole dataframe, the following should be noted:
- rolling beta computation causes many null values at the start of security entries
  - shifting to prevent look-ahead pushes in a further null
- the first security entry has null returns
- shifting market value to prevent look-ahead once again pushes in a null

In [48]:
m_returns.describe()

,ajexm,cshom,prccm,trfm,adjclose,exratd_toUSD,local_ret,USD_fxret,USD_ret,ret_mspan,m_USD_ret,m_local_ret,USD_secval,USD_mktval,X_USD_mktval,datayear,datayear-1,m_mktret,beta,X_beta
count,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06,2.964769e+06
mean,1.187213e+00,2.219655e+09,2.599763e+03,6.353686e+00,1.375497e+04,4.968647e-01,5.488095e+01,-8.894104e-02,5.481879e+01,1.006416e+00,4.717179e+01,4.723473e+01,5.649694e+09,2.026250e+10,2.014140e+10,2.019528e+03,2.018528e+03,7.647547e-01,-2.856569e+00,-2.902776e+00
std,2.063503e+00,8.532751e+10,2.737965e+04,3.741609e+03,1.832237e+06,5.038307e-01,4.102491e+04,1.606003e+01,4.102612e+04,5.420143e-01,4.010576e+04,4.010452e+04,2.849917e+10,1.132465e+11,1.122033e+11,2.718561e+00,2.718561e+00,4.378971e+00,3.248307e+03,3.247685e+03
min,5.000000e-08,0.000000e+00,1.000000e-06,1.000000e+00,1.000000e-06,3.929460e-05,-9.999996e+01,-9.969494e+01,-9.999996e+01,1.000000e+00,-9.999996e+01,-9.999996e+01,0.000000e+00,1.755000e+00,3.231000e+00,2.015000e+03,2.014000e+03,-1.318113e+01,-1.813294e+06,-1.813294e+06
25%,1.000000e+00,3.917199e+07,5.000000e+00,1.030423e+00,6.612383e+00,3.396126e-02,-5.387970e+00,-9.282515e-01,-5.786780e+00,1.000000e+00,-5.781262e+00,-5.383734e+00,1.993802e+08,2.388426e+08,2.388584e+08,2.017000e+03,2.016000e+03,-2.043487e+00,2.741334e-01,2.724270e-01
50%,1.000000e+00,1.719770e+08,2.038000e+01,1.164562e+00,2.668000e+01,1.551776e-01,0.000000e+00,0.000000e+00,-1.934878e-02,1.000000e+00,-1.934101e-02,0.000000e+00,7.938009e+08,1.255716e+09,1.253500e+09,2.020000e+03,2.019000e+03,1.239566e+00,8.358552e-01,8.371212e-01
75%,1.000000e+00,8.065043e+08,1.156000e+02,1.576426e+00,1.751996e+02,1.000000e+00,5.714286e+00,6.462257e-01,5.948656e+00,1.000000e+00,5.941818e+00,5.711588e+00,2.826342e+09,5.490539e+09,5.478155e+09,2.022000e+03,2.021000e+03,3.189458e+00,1.502049e+00,1.505038e+00
max,2.900519e+02,1.333047e+13,5.808000e+06,5.310840e+06,6.918929e+08,3.378142e+00,5.999990e+07,2.467480e+04,5.999990e+07,1.090000e+02,5.999990e+07,5.999990e+07,3.405393e+12,5.942845e+12,5.942845e+12,2.024000e+03,2.023000e+03,1.222895e+01,6.467876e+05,6.467876e+05


We also observe extreme outliers for return, market value & beta.

In [1199]:
# na_returns_df.xs((140044, "01"))
# na_returns_df.xs((311798, "01"))

Thus, we create an instance will the null values and outlier rows filtered out.
- at the 0.1% level

In [257]:
# na_returns = na_returns[
#     (na_returns["m_USD_ret"].quantile(0.001) < na_returns["m_USD_ret"])
#     & (na_returns["m_USD_ret"] < na_returns["m_USD_ret"].quantile(0.999))
#     & (na_returns["beta"].quantile(0.001) < na_returns["beta"])
#     & (na_returns["beta"] < na_returns["beta"].quantile(0.999))
#     & (na_returns["USD_mktval"].quantile(0.001) < na_returns["USD_mktval"])
#     & (na_returns["USD_mktval"] < na_returns["USD_mktval"].quantile(0.999))
# ]

m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2014-01 2014-01-31                    AAR CORP    USD    1.0   
           2014-02 2014-02-28                    AAR CORP    USD    1.0   
           2014-03 2014-03-31                    AAR CORP    USD    1.0   
           2014-04 2014-04-30                    AAR CORP    USD    1.0   
           2014-05 2014-05-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
362705 01W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
       02W 2024-08 2024-08-21  MISITANO & STRACUZZI S P A    EUR    1.0   
362758 01W 2024-08 2024-08-21         NUREN GROUP LIMITED    AUD    1.0   
362761 01W 2024-08 2024-08-21      AKUMS DRUGS AND PHARMA    INR    1.0   
362779 01W 2024-08 2024-08-21              NOVAMARINE SPA    EUR    1.0   

                          cshom    prccm      trfm     adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                              
1004   01  2014-01   39600000.0    26.65  1.623944    43.278106      1.000000   
           2014-02   39600000.0    28.90  1.623944    46.931980      1.000000   
           2014-03   39569000.0    25.95  1.623944    42.141346      1.000000   
           2014-04   39569000.0    25.90  1.628586    42.180366      1.000000   
           2014-05   39569000.0    24.30  1.628586    39.574629      1.000000   
...                         ...      ...       ...          ...           ...   
362705 01W 2024-08   26460000.0     2.82  1.147992     3.237339      1.113108   
       02W 2024-08   26460000.0     2.63  1.147992     3.019220      1.113108   
362758 01W 2024-08  154500000.0     0.22  1.147992     0.252558      0.675052   
362761 01W 2024-08  157393988.0  1010.75  1.147992  1160.333318      0.011920   
362779 01W 2024-08   12388500.0     3.58  1.145429     4.100637      1.113108   

                    local_ret  ...  m_USD_ret  m_local_ret    USD_secval  \
gvkey  iid data_ym             ...                                         
1004   01  2014-01        NaN  ...        NaN          NaN  1.055340e+09   
           2014-02   8.442777  ...   8.442777     8.442777  1.144440e+09   
           2014-03 -10.207612  ... -10.207612   -10.207612  1.026816e+09   
           2014-04   0.092594  ...   0.092594     0.092594  1.024837e+09   
           2014-05  -6.177606  ...  -6.177606    -6.177606  9.615267e+08   
...                       ...  ...        ...          ...           ...   
362705 01W 2024-08  -6.622517  ...  -3.960845    -6.622517  8.305700e+07   
       02W 2024-08        NaN  ...        NaN          NaN  7.746096e+07   
362758 01W 2024-08        NaN  ...        NaN          NaN  2.294501e+07   
362761 01W 2024-08        NaN  ...        NaN          NaN  1.896297e+09   
362779 01W 2024-08        NaN  ...        NaN          NaN  4.936726e+07   

                      USD_mktval  X_USD_mktval  datayear  datayear-1  \
gvkey  iid data_ym                                                     
1004   01  2014-01  1.055340e+09           NaN      2014        2013   
           2014-02  1.144440e+09  1.055340e+09      2014        2013   
           2014-03  1.026816e+09  1.144440e+09      2014        2013   
           2014-04  1.024837e+09  1.026816e+09      2014        2013   
           2014-05  9.615267e+08  1.024837e+09      2014        2013   
...                          ...           ...       ...         ...   
362705 01W 2024-08  1.605180e+08  8.648243e+07      2024        2023   
       02W 2024-08  1.605180e+08           NaN      2024        2023   
362758 01W 2024-08  2.294501e+07           NaN      2024        2023   
362761 01W 2024-08  1.896297e+09           NaN      2024        2023   
362779 01W 2024-08  4.936726e+07           NaN      2024        2023   

                    m_mktret  beta  X_beta  
gvkey  iid data_ym  

In [258]:
m_returns.describe()

,ajexm,cshom,prccm,trfm,adjclose,exratd_toUSD,local_ret,USD_fxret,USD_ret,ret_mspan,m_USD_ret,m_local_ret,USD_secval,USD_mktval,X_USD_mktval,datayear,datayear-1,m_mktret,beta,X_beta
count,3.478493e+06,3.478493e+06,3.478493e+06,3.478493e+06,3.478493e+06,3.478493e+06,3.439982e+06,3.439982e+06,3.439982e+06,3.439982e+06,3.439982e+06,3.439982e+06,3.478493e+06,3.478493e+06,3.439982e+06,3.478493e+06,3.478493e+06,3.427583e+06,3.001259e+06,2.992752e+06
mean,1.223254e+00,2.196598e+09,2.579741e+03,9.992550e+02,9.076072e+04,5.046947e-01,5.432607e+01,-1.176851e-01,5.424129e+01,1.008295e+00,4.467695e+01,4.476162e+01,5.538351e+09,1.997734e+10,1.989513e+10,2.019071e+03,2.018071e+03,7.347236e-01,-2.787099e+00,-2.854971e+00
std,2.604588e+00,8.583865e+10,2.796112e+04,5.349894e+05,4.186256e+07,5.096464e-01,3.858370e+04,1.492885e+01,3.858506e+04,5.631355e-01,3.745016e+04,3.744877e+04,2.816118e+10,1.130512e+11,1.121710e+11,3.032324e+00,3.032324e+00,4.226920e+00,3.228652e+03,3.232489e+03
min,5.000000e-08,0.000000e+00,1.000000e-06,1.000000e+00,1.000000e-06,3.929460e-05,-9.999996e+01,-9.969494e+01,-9.999996e+01,1.000000e+00,-9.999996e+01,-9.999996e+01,0.000000e+00,1.755000e+00,1.755000e+00,2.014000e+03,2.013000e+03,-1.318113e+01,-1.813294e+06,-1.813294e+06
25%,1.000000e+00,3.836600e+07,5.060000e+00,1.021916e+00,6.551132e+00,3.565410e-02,-5.373832e+00,-9.401676e-01,-5.787336e+00,1.000000e+00,-5.778898e+00,-5.365912e+00,1.966070e+08,2.357381e+08,2.361839e+08,2.017000e+03,2.016000e+03,-1.883493e+00,2.713983e-01,2.724096e-01
50%,1.000000e+00,1.672452e+08,2.014000e+01,1.142766e+00,2.550274e+01,1.572043e-01,0.000000e+00,0.000000e+00,-4.504465e-02,1.000000e+00,-4.496231e-02,0.000000e+00,7.838229e+08,1.221833e+09,1.222880e+09,2.019000e+03,2.018000e+03,1.115131e+00,8.354287e-01,8.361603e-01
75%,1.000000e+00,7.791466e+08,1.079500e+02,1.533917e+00,1.601946e+02,1.000000e+00,5.686126e+00,6.394226e-01,5.870456e+00,1.000000e+00,5.861166e+00,5.676968e+00,2.750985e+09,5.306462e+09,5.309057e+09,2.022000e+03,2.021000e+03,3.122349e+00,1.503432e+00,1.503695e+00
max,1.112137e+03,1.333047e+13,5.808000e+06,2.911432e+08,2.824089e+10,3.555697e+00,5.999990e+07,2.467480e+04,5.999990e+07,1.190000e+02,5.999990e+07,5.999990e+07,3.448906e+12,5.942845e+12,5.942845e+12,2.024000e+03,2.023000e+03,1.222895e+01,6.467876e+05,6.467876e+05


: 

## Fundamentals Data

### NA & Global - Loading and Preliminary Inspection

**NA**

In [49]:
na_fundamentals_df = pd.read_csv(
    "na_fundamentals_2014to2024.csv", 
    parse_dates=["datadate"]
).rename(columns={"fyear": "fiscalyear"})

print(na_fundamentals_df.dtypes)
print(na_fundamentals_df.isnull().sum())
na_fundamentals_df

gvkey                  int64
datadate      datetime64[ns]
fiscalyear           float64
indfmt                object
consol                object
popsrc                object
datafmt               object
conm                  object
curcd                 object
at                   float64
ceq                  float64
oiadp                float64
revt                 float64
costat                object
dtype: object
gvkey            0
datadate         0
fiscalyear     102
indfmt           0
consol           0
popsrc           0
datafmt          0
conm             0
curcd          102
at             906
ceq           1018
oiadp         8322
revt           968
costat           0
dtype: int64


,gvkey,datadate,fiscalyear,indfmt,consol,popsrc,datafmt,conm,curcd,at,ceq,oiadp,revt,costat
0,1004,2015-05-31,2014.0,INDL,C,D,STD,AAR CORP,USD,1515.000,845.100,-8.600,1594.300,A
1,1004,2016-05-31,2015.0,INDL,C,D,STD,AAR CORP,USD,1442.100,865.800,66.100,1662.600,A
2,1004,2017-05-31,2016.0,INDL,C,D,STD,AAR CORP,USD,1504.100,914.200,77.200,1767.600,A
3,1004,2018-05-31,2017.0,INDL,C,D,STD,AAR CORP,USD,1524.700,936.300,86.000,1748.300,A
4,1004,2019-05-31,2018.0,INDL,C,D,STD,AAR CORP,USD,1517.200,905.900,110.700,2051.800,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55142,353444,2021-12-31,2021.0,INDL,C,D,STD,HALEON PLC,USD,46650.099,35687.306,2816.529,12924.885,A
55143,353444,2022-12-31,2022.0,INDL,C,D,STD,HALEON PLC,USD,41948.594,19677.222,2913.448,13082.804,A
55144,353444,2023-12-31,2023.0,INDL,C,D,STD,HALEON PLC,USD,43379.259,21152.723,3181.953,14396.488,A
55145,356128,2022-12-31,2022.0,INDL,C,D,STD,JOINT STOCK COMPANY KASPI KZ,USD,11072.847,1771.010,2160.348,2746.982,A


In [50]:
na_fiscalyear_null_mask = pd.isnull(na_fundamentals_df["fiscalyear"])

# fiscal year is the previous year to the reporting period end date if the 
# month is May or earlier, otherwise the fiscal year is taken as the same 
# year as the reporting period end date 
na_fundamentals_df.loc[
    na_fiscalyear_null_mask, "fiscalyear"
] = na_fundamentals_df.loc[
    na_fiscalyear_null_mask, "datadate"
].dt.year - (
    na_fundamentals_df.loc[
        na_fiscalyear_null_mask, "datadate"
    ].dt.month < 6
)
# now that there are no nulls, ensure that it is integer typed for 
# compatibility with the other datasets
na_fundamentals_df["fiscalyear"] = na_fundamentals_df["fiscalyear"].astype(int)

# review
print(na_fundamentals_df.dtypes)
print(na_fundamentals_df.isnull().sum())
na_fundamentals_df

gvkey                  int64
datadate      datetime64[ns]
fiscalyear             int64
indfmt                object
consol                object
popsrc                object
datafmt               object
conm                  object
curcd                 object
at                   float64
ceq                  float64
oiadp                float64
revt                 float64
costat                object
dtype: object
gvkey            0
datadate         0
fiscalyear       0
indfmt           0
consol           0
popsrc           0
datafmt          0
conm             0
curcd          102
at             906
ceq           1018
oiadp         8322
revt           968
costat           0
dtype: int64


,gvkey,datadate,fiscalyear,indfmt,consol,popsrc,datafmt,conm,curcd,at,ceq,oiadp,revt,costat
0,1004,2015-05-31,2014,INDL,C,D,STD,AAR CORP,USD,1515.000,845.100,-8.600,1594.300,A
1,1004,2016-05-31,2015,INDL,C,D,STD,AAR CORP,USD,1442.100,865.800,66.100,1662.600,A
2,1004,2017-05-31,2016,INDL,C,D,STD,AAR CORP,USD,1504.100,914.200,77.200,1767.600,A
3,1004,2018-05-31,2017,INDL,C,D,STD,AAR CORP,USD,1524.700,936.300,86.000,1748.300,A
4,1004,2019-05-31,2018,INDL,C,D,STD,AAR CORP,USD,1517.200,905.900,110.700,2051.800,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55142,353444,2021-12-31,2021,INDL,C,D,STD,HALEON PLC,USD,46650.099,35687.306,2816.529,12924.885,A
55143,353444,2022-12-31,2022,INDL,C,D,STD,HALEON PLC,USD,41948.594,19677.222,2913.448,13082.804,A
55144,353444,2023-12-31,2023,INDL,C,D,STD,HALEON PLC,USD,43379.259,21152.723,3181.953,14396.488,A
55145,356128,2022-12-31,2022,INDL,C,D,STD,JOINT STOCK COMPANY KASPI KZ,USD,11072.847,1771.010,2160.348,2746.982,A


Companies, identified by gvkeys, can have multiple entries per fiscal year in the 
raw downloaded data. 

This is e.g. because of different 
reporting formats (`indfmt`).

 - Consolidate information into single gvkey-fiscalyear entries.

This will reduce the number of null values.

In [51]:
na_fundamentals_df = na_fundamentals_df.groupby(
    ["gvkey", "fiscalyear"]
).first()
na_fundamentals_df = na_fundamentals_df.sort_index()

# review
print(na_fundamentals_df.dtypes)
print(na_fundamentals_df.isnull().sum())
na_fundamentals_df

datadate    datetime64[ns]
indfmt              object
consol              object
popsrc              object
datafmt             object
conm                object
curcd               object
at                 float64
ceq                float64
oiadp              float64
revt               float64
costat              object
dtype: object
datadate       0
indfmt         0
consol         0
popsrc         0
datafmt        0
conm           0
curcd         99
at           902
ceq         1001
oiadp        961
revt         961
costat         0
dtype: int64


datadate indfmt consol popsrc datafmt  \
gvkey  fiscalyear                                           
1004   2014       2015-05-31   INDL      C      D     STD   
       2015       2016-05-31   INDL      C      D     STD   
       2016       2017-05-31   INDL      C      D     STD   
       2017       2018-05-31   INDL      C      D     STD   
       2018       2019-05-31   INDL      C      D     STD   
...                      ...    ...    ...    ...     ...   
353444 2021       2021-12-31   INDL      C      D     STD   
       2022       2022-12-31   INDL      C      D     STD   
       2023       2023-12-31   INDL      C      D     STD   
356128 2022       2022-12-31   INDL      C      D     STD   
       2023       2023-12-31   INDL      C      D     STD   

                                           conm curcd         at        ceq  \
gvkey  fiscalyear                                                             
1004   2014                            AAR CORP   USD   1515.000    845.100   
       2015                            AAR CORP   USD   1442.100    865.800   
       2016                            AAR CORP   USD   1504.100    914.200   
       2017                            AAR CORP   USD   1524.700    936.300   
       2018                            AAR CORP   USD   1517.200    905.900   
...                                         ...   ...        ...        ...   
353444 2021                          HALEON PLC   USD  46650.099  35687.306   
       2022                          HALEON PLC   USD  41948.594  19677.222   
       2023                          HALEON PLC   USD  43379.259  21152.723   
356128 2022        JOINT STOCK COMPANY KASPI KZ   USD  11072.847   1771.010   
       2023        JOINT STOCK COMPANY KASPI KZ   USD  15046.521   2377.919   

                      oiadp       revt costat  
gvkey  fiscalyear                              
1004   2014          -8.600   1594.300      A  
       2015          66.100   1662.600      A  
       2016          77.200   1767.600      A  
       2017          86.000   1748.300      A  
       2018         110.700   2051.800      A  
...                     ...        ...    ...  
353444 2021        2816.529  12924.885      A  
       2022        2913.448  13082.804      A  
       2023        3181.953  14396.488      A  
356128 2022        2160.348   2746.982      A  
       2023        3308.446   4220.413      A  

[47786 rows x 12 columns]

Finally, drop the rows which still have any null values.

In [52]:
na_fundamentals_df = na_fundamentals_df.dropna()

na_fundamentals_df

datadate indfmt consol popsrc datafmt  \
gvkey  fiscalyear                                           
1004   2014       2015-05-31   INDL      C      D     STD   
       2015       2016-05-31   INDL      C      D     STD   
       2016       2017-05-31   INDL      C      D     STD   
       2017       2018-05-31   INDL      C      D     STD   
       2018       2019-05-31   INDL      C      D     STD   
...                      ...    ...    ...    ...     ...   
353444 2021       2021-12-31   INDL      C      D     STD   
       2022       2022-12-31   INDL      C      D     STD   
       2023       2023-12-31   INDL      C      D     STD   
356128 2022       2022-12-31   INDL      C      D     STD   
       2023       2023-12-31   INDL      C      D     STD   

                                           conm curcd         at        ceq  \
gvkey  fiscalyear                                                             
1004   2014                            AAR CORP   USD   1515.000    845.100   
       2015                            AAR CORP   USD   1442.100    865.800   
       2016                            AAR CORP   USD   1504.100    914.200   
       2017                            AAR CORP   USD   1524.700    936.300   
       2018                            AAR CORP   USD   1517.200    905.900   
...                                         ...   ...        ...        ...   
353444 2021                          HALEON PLC   USD  46650.099  35687.306   
       2022                          HALEON PLC   USD  41948.594  19677.222   
       2023                          HALEON PLC   USD  43379.259  21152.723   
356128 2022        JOINT STOCK COMPANY KASPI KZ   USD  11072.847   1771.010   
       2023        JOINT STOCK COMPANY KASPI KZ   USD  15046.521   2377.919   

                      oiadp       revt costat  
gvkey  fiscalyear                              
1004   2014          -8.600   1594.300      A  
       2015          66.100   1662.600      A  
       2016          77.200   1767.600      A  
       2017          86.000   1748.300      A  
       2018         110.700   2051.800      A  
...                     ...        ...    ...  
353444 2021        2816.529  12924.885      A  
       2022        2913.448  13082.804      A  
       2023        3181.953  14396.488      A  
356128 2022        2160.348   2746.982      A  
       2023        3308.446   4220.413      A  

[46727 rows x 12 columns]

**Global**

In [53]:
global_fundamentals_df = pd.read_csv(
    "global_fundamentals_2014to2024.csv", 
    parse_dates=["datadate"]
).rename(columns={"fyear": "fiscalyear"})

global_fundamentals_df

,gvkey,curcd,fiscalyear,datadate,at,ceq,oiadp,revt,conm
0,1166,EUR,2014.0,2014-12-31,1826.933,1690.200,68.098,545.604,ASM INTERNATIONAL NV
1,1166,EUR,2015.0,2015-12-31,2075.977,1948.379,101.776,669.621,ASM INTERNATIONAL NV
2,1166,EUR,2016.0,2016-12-31,2148.263,2015.856,63.470,597.930,ASM INTERNATIONAL NV
3,1166,EUR,2017.0,2017-12-31,2177.202,2011.512,95.495,737.401,ASM INTERNATIONAL NV
4,1166,EUR,2018.0,2018-12-31,1847.972,1641.551,113.390,818.081,ASM INTERNATIONAL NV
...,...,...,...,...,...,...,...,...,...
172550,362282,INR,2018.0,2019-03-31,41.185,-32.446,20.168,81.469,DIENSTEN TECH LIMITED
172551,362282,INR,2019.0,2020-03-31,33.762,-22.880,17.603,56.199,DIENSTEN TECH LIMITED
172552,362282,INR,2020.0,2021-03-31,37.035,-11.173,20.016,52.320,DIENSTEN TECH LIMITED
172553,362282,INR,2021.0,2022-03-31,20.411,13.999,1.870,7.241,DIENSTEN TECH LIMITED


Same processing as for NA.

In [54]:
global_fiscalyear_null_mask = pd.isnull(global_fundamentals_df["fiscalyear"])

# fiscal year is the previous year to the reporting period end date if the 
# month is May or earlier, otherwise the fiscal year is taken as the same 
# year as the reporting period end date 
global_fundamentals_df.loc[
    global_fiscalyear_null_mask, "fiscalyear"
] = global_fundamentals_df.loc[
    global_fiscalyear_null_mask, "datadate"
].dt.year - (
    global_fundamentals_df.loc[
        global_fiscalyear_null_mask, "datadate"
    ].dt.month < 6
)
# now that there are no nulls, ensure that it is integer typed for 
# compatibility with the other datasets
global_fundamentals_df["fiscalyear"] = global_fundamentals_df["fiscalyear"].astype(int)

global_fundamentals_df = global_fundamentals_df.groupby(
    ["gvkey", "fiscalyear"]
).first()
global_fundamentals_df = global_fundamentals_df.sort_index()
global_fundamentals_df = global_fundamentals_df.dropna()

global_fundamentals_df

curcd   datadate        at       ceq    oiadp     revt  \
gvkey  fiscalyear                                                          
1166   2014         EUR 2014-12-31  1826.933  1690.200   68.098  545.604   
       2015         EUR 2015-12-31  2075.977  1948.379  101.776  669.621   
       2016         EUR 2016-12-31  2148.263  2015.856   63.470  597.930   
       2017         EUR 2017-12-31  2177.202  2011.512   95.495  737.401   
       2018         EUR 2018-12-31  1847.972  1641.551  113.390  818.081   
...                 ...        ...       ...       ...      ...      ...   
362282 2018         INR 2019-03-31    41.185   -32.446   20.168   81.469   
       2019         INR 2020-03-31    33.762   -22.880   17.603   56.199   
       2020         INR 2021-03-31    37.035   -11.173   20.016   52.320   
       2021         INR 2022-03-31    20.411    13.999    1.870    7.241   
       2022         INR 2023-03-31   150.482    42.505    3.899  375.269   

                                    conm  
gvkey  fiscalyear                         
1166   2014         ASM INTERNATIONAL NV  
       2015         ASM INTERNATIONAL NV  
       2016         ASM INTERNATIONAL NV  
       2017         ASM INTERNATIONAL NV  
       2018         ASM INTERNATIONAL NV  
...                                  ...  
362282 2018        DIENSTEN TECH LIMITED  
       2019        DIENSTEN TECH LIMITED  
       2020        DIENSTEN TECH LIMITED  
       2021        DIENSTEN TECH LIMITED  
       2022        DIENSTEN TECH LIMITED  

[171215 rows x 7 columns]

### NA & Global - Data Preparation

Combine both datasets into one authorative fundamentals data source.

In [55]:
fundamentals_df = pd.concat(
    [na_fundamentals_df.reset_index(), global_fundamentals_df.reset_index()], 
    ignore_index=True, 
).groupby(
    ["gvkey", "fiscalyear"]
).first().sort_index().drop(columns=["indfmt", "consol", "popsrc", "datafmt", "costat"])

fundamentals_df

datadate                   conm curcd        at      ceq  \
gvkey  fiscalyear                                                              
1004   2014       2015-05-31               AAR CORP   USD  1515.000  845.100   
       2015       2016-05-31               AAR CORP   USD  1442.100  865.800   
       2016       2017-05-31               AAR CORP   USD  1504.100  914.200   
       2017       2018-05-31               AAR CORP   USD  1524.700  936.300   
       2018       2019-05-31               AAR CORP   USD  1517.200  905.900   
...                      ...                    ...   ...       ...      ...   
362282 2018       2019-03-31  DIENSTEN TECH LIMITED   INR    41.185  -32.446   
       2019       2020-03-31  DIENSTEN TECH LIMITED   INR    33.762  -22.880   
       2020       2021-03-31  DIENSTEN TECH LIMITED   INR    37.035  -11.173   
       2021       2022-03-31  DIENSTEN TECH LIMITED   INR    20.411   13.999   
       2022       2023-03-31  DIENSTEN TECH LIMITED   INR   150.482   42.505   

                     oiadp      revt  
gvkey  fiscalyear                     
1004   2014         -8.600  1594.300  
       2015         66.100  1662.600  
       2016         77.200  1767.600  
       2017         86.000  1748.300  
       2018        110.700  2051.800  
...                    ...       ...  
362282 2018         20.168    81.469  
       2019         17.603    56.199  
       2020         20.016    52.320  
       2021          1.870     7.241  
       2022          3.899   375.269  

[213880 rows x 7 columns]

Join with the exchange rates to USD in order to standardise the values to USD.
- Last for balance sheet
- Average for income statement

In [56]:
m_exrts_flat = m_exrts.reset_index().drop(columns="exratd_toGBP")
m_exrts_flat["datayear"] = m_exrts_flat["data_ym"].dt.year

m_exrts_flat

,curd,data_ym,exratd_toUSD,datayear
0,AED,2014-01,0.272257,2014
1,AED,2014-02,0.272249,2014
2,AED,2014-03,0.272248,2014
3,AED,2014-04,0.272251,2014
4,AED,2014-05,0.272243,2014
...,...,...,...,...
20782,ZWL,2024-04,0.002944,2024
20783,ZWL,2024-05,0.002991,2024
20784,ZWL,2024-06,0.002970,2024
20785,ZWL,2024-07,0.003018,2024


In [57]:
y_bs_exrts = m_exrts_flat.drop(columns="data_ym").groupby(
    ["curd", "datayear"]
).last()[["exratd_toUSD"]].rename(
    columns={"exratd_toUSD": "bs_toUSD"}
)

y_bs_exrts

bs_toUSD
curd datayear          
AED  2014      0.272257
     2015      0.272272
     2016      0.272215
     2017      0.272230
     2018      0.272253
...                 ...
ZWL  2020      0.003209
     2021      0.003183
     2022      0.002832
     2023      0.002994
     2024      0.003072

[1798 rows x 1 columns]

In [58]:
y_is_exrts = m_exrts_flat.drop(columns="data_ym").groupby(
    ["curd", "datayear"]
).mean()[["exratd_toUSD"]].rename(columns={"exratd_toUSD": "is_toUSD"})

y_is_exrts

is_toUSD
curd datayear          
AED  2014      0.272244
     2015      0.272258
     2016      0.272263
     2017      0.272265
     2018      0.272249
...                 ...
ZWL  2020      0.003038
     2021      0.003230
     2022      0.002893
     2023      0.002933
     2024      0.002990

[1798 rows x 1 columns]

In [59]:
y_fs_exrts = pd.concat(
    [y_bs_exrts, y_is_exrts], 
    axis=1
)

y_fs_exrts

bs_toUSD  is_toUSD
curd datayear                    
AED  2014      0.272257  0.272244
     2015      0.272272  0.272258
     2016      0.272215  0.272263
     2017      0.272230  0.272265
     2018      0.272253  0.272249
...                 ...       ...
ZWL  2020      0.003209  0.003038
     2021      0.003183  0.003230
     2022      0.002832  0.002893
     2023      0.002994  0.002933
     2024      0.003072  0.002990

[1798 rows x 2 columns]

Now join the computed exchange rates by aligning on fiscal year.

In [60]:
fundamentals_df = fundamentals_df.reset_index().merge(
    y_fs_exrts, 
    how="left", 
    left_on=["curcd", "fiscalyear"], 
    right_index=True, 
).set_index(
    ["gvkey", "fiscalyear"]
)

fundamentals_df

datadate                   conm curcd        at      ceq  \
gvkey  fiscalyear                                                              
1004   2014       2015-05-31               AAR CORP   USD  1515.000  845.100   
       2015       2016-05-31               AAR CORP   USD  1442.100  865.800   
       2016       2017-05-31               AAR CORP   USD  1504.100  914.200   
       2017       2018-05-31               AAR CORP   USD  1524.700  936.300   
       2018       2019-05-31               AAR CORP   USD  1517.200  905.900   
...                      ...                    ...   ...       ...      ...   
362282 2018       2019-03-31  DIENSTEN TECH LIMITED   INR    41.185  -32.446   
       2019       2020-03-31  DIENSTEN TECH LIMITED   INR    33.762  -22.880   
       2020       2021-03-31  DIENSTEN TECH LIMITED   INR    37.035  -11.173   
       2021       2022-03-31  DIENSTEN TECH LIMITED   INR    20.411   13.999   
       2022       2023-03-31  DIENSTEN TECH LIMITED   INR   150.482   42.505   

                     oiadp      revt  bs_toUSD  is_toUSD  
gvkey  fiscalyear                                         
1004   2014         -8.600  1594.300  1.000000  1.000000  
       2015         66.100  1662.600  1.000000  1.000000  
       2016         77.200  1767.600  1.000000  1.000000  
       2017         86.000  1748.300  1.000000  1.000000  
       2018        110.700  2051.800  1.000000  1.000000  
...                    ...       ...       ...       ...  
362282 2018         20.168    81.469  0.014368  0.014634  
       2019         17.603    56.199  0.014051  0.014216  
       2020         20.016    52.320  0.013694  0.013500  
       2021          1.870     7.241  0.013422  0.013529  
       2022          3.899   375.269  0.012083  0.012695  

[213880 rows x 9 columns]

Apply the exchange rates.

In [61]:
fundamentals_df["at"] *= fundamentals_df["bs_toUSD"]
fundamentals_df["ceq"] *= fundamentals_df["bs_toUSD"]

fundamentals_df["oiadp"] *= fundamentals_df["is_toUSD"]
fundamentals_df["revt"] *= fundamentals_df["is_toUSD"]

In [62]:
fundamentals_df

datadate                   conm curcd           at  \
gvkey  fiscalyear                                                        
1004   2014       2015-05-31               AAR CORP   USD  1515.000000   
       2015       2016-05-31               AAR CORP   USD  1442.100000   
       2016       2017-05-31               AAR CORP   USD  1504.100000   
       2017       2018-05-31               AAR CORP   USD  1524.700000   
       2018       2019-05-31               AAR CORP   USD  1517.200000   
...                      ...                    ...   ...          ...   
362282 2018       2019-03-31  DIENSTEN TECH LIMITED   INR     0.591730   
       2019       2020-03-31  DIENSTEN TECH LIMITED   INR     0.474373   
       2020       2021-03-31  DIENSTEN TECH LIMITED   INR     0.507151   
       2021       2022-03-31  DIENSTEN TECH LIMITED   INR     0.273953   
       2022       2023-03-31  DIENSTEN TECH LIMITED   INR     1.818302   

                          ceq       oiadp         revt  bs_toUSD  is_toUSD  
gvkey  fiscalyear                                                           
1004   2014        845.100000   -8.600000  1594.300000  1.000000  1.000000  
       2015        865.800000   66.100000  1662.600000  1.000000  1.000000  
       2016        914.200000   77.200000  1767.600000  1.000000  1.000000  
       2017        936.300000   86.000000  1748.300000  1.000000  1.000000  
       2018        905.900000  110.700000  2051.800000  1.000000  1.000000  
...                       ...         ...          ...       ...       ...  
362282 2018         -0.466171    0.295137     1.192211  0.014368  0.014634  
       2019         -0.321476    0.250241     0.798915  0.014051  0.014216  
       2020         -0.153001    0.270213     0.706313  0.013694  0.013500  
       2021          0.187893    0.025298     0.097960  0.013422  0.013529  
       2022          0.513596    0.049500     4.764219  0.012083  0.012695  

[213880 rows x 9 columns]

Compute investment by percentage change in assets per annum.

In [63]:
fundamentals_df["investment"] = fundamentals_df.groupby(
    level="gvkey"
)["at"].pct_change()

# standardise to per annum
fundamentals_df["reporting_gap"] = fundamentals_df.reset_index().groupby(
    "gvkey"
)["fiscalyear"].diff().set_axis(fundamentals_df.index)
fundamentals_df["investment"] = (1 + fundamentals_df["investment"]) ** (1/fundamentals_df["reporting_gap"]) - 1

fundamentals_df

datadate                   conm curcd           at  \
gvkey  fiscalyear                                                        
1004   2014       2015-05-31               AAR CORP   USD  1515.000000   
       2015       2016-05-31               AAR CORP   USD  1442.100000   
       2016       2017-05-31               AAR CORP   USD  1504.100000   
       2017       2018-05-31               AAR CORP   USD  1524.700000   
       2018       2019-05-31               AAR CORP   USD  1517.200000   
...                      ...                    ...   ...          ...   
362282 2018       2019-03-31  DIENSTEN TECH LIMITED   INR     0.591730   
       2019       2020-03-31  DIENSTEN TECH LIMITED   INR     0.474373   
       2020       2021-03-31  DIENSTEN TECH LIMITED   INR     0.507151   
       2021       2022-03-31  DIENSTEN TECH LIMITED   INR     0.273953   
       2022       2023-03-31  DIENSTEN TECH LIMITED   INR     1.818302   

                          ceq       oiadp         revt  bs_toUSD  is_toUSD  \
gvkey  fiscalyear                                                            
1004   2014        845.100000   -8.600000  1594.300000  1.000000  1.000000   
       2015        865.800000   66.100000  1662.600000  1.000000  1.000000   
       2016        914.200000   77.200000  1767.600000  1.000000  1.000000   
       2017        936.300000   86.000000  1748.300000  1.000000  1.000000   
       2018        905.900000  110.700000  2051.800000  1.000000  1.000000   
...                       ...         ...          ...       ...       ...   
362282 2018         -0.466171    0.295137     1.192211  0.014368  0.014634   
       2019         -0.321476    0.250241     0.798915  0.014051  0.014216   
       2020         -0.153001    0.270213     0.706313  0.013694  0.013500   
       2021          0.187893    0.025298     0.097960  0.013422  0.013529   
       2022          0.513596    0.049500     4.764219  0.012083  0.012695   

                   investment  reporting_gap  
gvkey  fiscalyear                             
1004   2014               NaN            NaN  
       2015         -0.048119            1.0  
       2016          0.042993            1.0  
       2017          0.013696            1.0  
       2018         -0.004919            1.0  
...                       ...            ...  
362282 2018         -0.439908            1.0  
       2019         -0.198328            1.0  
       2020          0.069097            1.0  
       2021         -0.459819            1.0  
       2022          5.637264            1.0  

[213880 rows x 11 columns]

In [64]:
len(fundamentals_df.dropna())

188416

Compute operating profitability.

In [65]:
fundamentals_df["OP"] = fundamentals_df["oiadp"] / fundamentals_df["ceq"]
# np.clip(np.nan_to_num(
#     , 
#     posinf=0, 
#     neginf=0
# ), a_min=0, a_max=None)

fundamentals_df

datadate                   conm curcd           at  \
gvkey  fiscalyear                                                        
1004   2014       2015-05-31               AAR CORP   USD  1515.000000   
       2015       2016-05-31               AAR CORP   USD  1442.100000   
       2016       2017-05-31               AAR CORP   USD  1504.100000   
       2017       2018-05-31               AAR CORP   USD  1524.700000   
       2018       2019-05-31               AAR CORP   USD  1517.200000   
...                      ...                    ...   ...          ...   
362282 2018       2019-03-31  DIENSTEN TECH LIMITED   INR     0.591730   
       2019       2020-03-31  DIENSTEN TECH LIMITED   INR     0.474373   
       2020       2021-03-31  DIENSTEN TECH LIMITED   INR     0.507151   
       2021       2022-03-31  DIENSTEN TECH LIMITED   INR     0.273953   
       2022       2023-03-31  DIENSTEN TECH LIMITED   INR     1.818302   

                          ceq       oiadp         revt  bs_toUSD  is_toUSD  \
gvkey  fiscalyear                                                            
1004   2014        845.100000   -8.600000  1594.300000  1.000000  1.000000   
       2015        865.800000   66.100000  1662.600000  1.000000  1.000000   
       2016        914.200000   77.200000  1767.600000  1.000000  1.000000   
       2017        936.300000   86.000000  1748.300000  1.000000  1.000000   
       2018        905.900000  110.700000  2051.800000  1.000000  1.000000   
...                       ...         ...          ...       ...       ...   
362282 2018         -0.466171    0.295137     1.192211  0.014368  0.014634   
       2019         -0.321476    0.250241     0.798915  0.014051  0.014216   
       2020         -0.153001    0.270213     0.706313  0.013694  0.013500   
       2021          0.187893    0.025298     0.097960  0.013422  0.013529   
       2022          0.513596    0.049500     4.764219  0.012083  0.012695   

                   investment  reporting_gap        OP  
gvkey  fiscalyear                                       
1004   2014               NaN            NaN -0.010176  
       2015         -0.048119            1.0  0.076346  
       2016          0.042993            1.0  0.084445  
       2017          0.013696            1.0  0.091851  
       2018         -0.004919            1.0  0.122199  
...                       ...            ...       ...  
362282 2018         -0.439908            1.0 -0.633108  
       2019         -0.198328            1.0 -0.778414  
       2020          0.069097            1.0 -1.766085  
       2021         -0.459819            1.0  0.134643  
       2022          5.637264            1.0  0.096379  

[213880 rows x 12 columns]

Finally, drop null rows so that the table is ready for linking.
- including (pos/neg) infinite values from dividing by 0 in opm

In [66]:
with pd.option_context('mode.use_inf_as_null', True):
    fundamentals = fundamentals_df.dropna()

fundamentals

datadate                   conm curcd           at  \
gvkey  fiscalyear                                                        
1004   2015       2016-05-31               AAR CORP   USD  1442.100000   
       2016       2017-05-31               AAR CORP   USD  1504.100000   
       2017       2018-05-31               AAR CORP   USD  1524.700000   
       2018       2019-05-31               AAR CORP   USD  1517.200000   
       2019       2020-05-31               AAR CORP   USD  2079.000000   
...                      ...                    ...   ...          ...   
362282 2018       2019-03-31  DIENSTEN TECH LIMITED   INR     0.591730   
       2019       2020-03-31  DIENSTEN TECH LIMITED   INR     0.474373   
       2020       2021-03-31  DIENSTEN TECH LIMITED   INR     0.507151   
       2021       2022-03-31  DIENSTEN TECH LIMITED   INR     0.273953   
       2022       2023-03-31  DIENSTEN TECH LIMITED   INR     1.818302   

                          ceq       oiadp         revt  bs_toUSD  is_toUSD  \
gvkey  fiscalyear                                                            
1004   2015        865.800000   66.100000  1662.600000  1.000000  1.000000   
       2016        914.200000   77.200000  1767.600000  1.000000  1.000000   
       2017        936.300000   86.000000  1748.300000  1.000000  1.000000   
       2018        905.900000  110.700000  2051.800000  1.000000  1.000000   
       2019        902.600000  106.400000  2089.300000  1.000000  1.000000   
...                       ...         ...          ...       ...       ...   
362282 2018         -0.466171    0.295137     1.192211  0.014368  0.014634   
       2019         -0.321476    0.250241     0.798915  0.014051  0.014216   
       2020         -0.153001    0.270213     0.706313  0.013694  0.013500   
       2021          0.187893    0.025298     0.097960  0.013422  0.013529   
       2022          0.513596    0.049500     4.764219  0.012083  0.012695   

                   investment  reporting_gap        OP  
gvkey  fiscalyear                                       
1004   2015         -0.048119            1.0  0.076346  
       2016          0.042993            1.0  0.084445  
       2017          0.013696            1.0  0.091851  
       2018         -0.004919            1.0  0.122199  
       2019          0.370287            1.0  0.117882  
...                       ...            ...       ...  
362282 2018         -0.439908            1.0 -0.633108  
       2019         -0.198328            1.0 -0.778414  
       2020          0.069097            1.0 -1.766085  
       2021         -0.459819            1.0  0.134643  
       2022          5.637264            1.0  0.096379  

[188092 rows x 12 columns]

In [67]:
fundamentals = fundamentals[
    ["revt", "at", "ceq", "OP", "investment"]
].rename(columns={"ceq": "USD_ceq", "at": "USD_at", "revt": "USD_revt"})

fundamentals

USD_revt       USD_at     USD_ceq        OP  investment
gvkey  fiscalyear                                                            
1004   2015        1662.600000  1442.100000  865.800000  0.076346   -0.048119
       2016        1767.600000  1504.100000  914.200000  0.084445    0.042993
       2017        1748.300000  1524.700000  936.300000  0.091851    0.013696
       2018        2051.800000  1517.200000  905.900000  0.122199   -0.004919
       2019        2089.300000  2079.000000  902.600000  0.117882    0.370287
...                        ...          ...         ...       ...         ...
362282 2018           1.192211     0.591730   -0.466171 -0.633108   -0.439908
       2019           0.798915     0.474373   -0.321476 -0.778414   -0.198328
       2020           0.706313     0.507151   -0.153001 -1.766085    0.069097
       2021           0.097960     0.273953    0.187893  0.134643   -0.459819
       2022           4.764219     1.818302    0.513596  0.096379    5.637264

[188092 rows x 5 columns]

In [240]:
fundamentals

USD_revt       USD_at     USD_ceq        OP  investment
gvkey  fiscalyear                                                            
1004   2015        1662.600000  1442.100000  865.800000  0.076346   -0.048119
       2016        1767.600000  1504.100000  914.200000  0.084445    0.042993
       2017        1748.300000  1524.700000  936.300000  0.091851    0.013696
       2018        2051.800000  1517.200000  905.900000  0.122199   -0.004919
       2019        2089.300000  2079.000000  902.600000  0.117882    0.370287
...                        ...          ...         ...       ...         ...
362282 2018           1.192211     0.591730   -0.466171 -0.633108   -0.439908
       2019           0.798915     0.474373   -0.321476 -0.778414   -0.198328
       2020           0.706313     0.507151   -0.153001 -1.766085    0.069097
       2021           0.097960     0.273953    0.187893  0.134643   -0.459819
       2022           4.764219     1.818302    0.513596  0.096379    5.637264

[188092 rows x 5 columns]

snippets

In [68]:
(fundamentals["OP"] == 0).sum()

18

## Carbon Earnings at Risk Forecast

In [69]:
cerf_df = pd.read_csv("cerf_2017to2023.csv")
cerf_df = cerf_df.dropna(subset="gvkey").copy()
cerf_df["gvkey"] = cerf_df["gvkey"].astype(int)

cerf_df

,institutionid,fiscalyear,periodenddate,scenariolevel,forecastyear,di_327652,di_327651,di_327644_text,di_327645_text,gvkey,companyname,country
0,100631,2017,2017-12-31,High,2030.0,0.100000,0.130000,0.0,0.0,111819,"Orrstown Financial Services, Inc.",United States
2,100631,2017,2017-12-31,High,2050.0,0.160000,0.200000,0.0,0.0,111819,"Orrstown Financial Services, Inc.",United States
4,100631,2017,2017-12-31,Medium,2040.0,0.070000,0.090000,0.0,0.0,111819,"Orrstown Financial Services, Inc.",United States
6,100631,2017,2017-12-31,Low,2030.0,0.030000,0.040000,0.0,0.0,111819,"Orrstown Financial Services, Inc.",United States
8,100631,2017,2017-12-31,Medium,2025.0,0.030000,0.040000,0.0,0.0,111819,"Orrstown Financial Services, Inc.",United States
...,...,...,...,...,...,...,...,...,...,...,...,...
2252678,111628694,2023,2023-03-31,High,2050.0,2.171778,2.293580,0.0,0.0,357018,NIIT Learning Systems Limited,India
2252679,111628694,2023,2023-03-31,Low,2025.0,0.371646,0.392490,0.0,0.0,357018,NIIT Learning Systems Limited,India
2252680,111628694,2023,2023-03-31,High,2030.0,1.012907,1.069715,0.0,0.0,357018,NIIT Learning Systems Limited,India
2252681,111628694,2023,2023-03-31,Low,2020.0,0.180156,0.190260,0.0,0.0,357018,NIIT Learning Systems Limited,India


In [70]:
cerf_df = cerf_df.drop(
    columns=[
        "institutionid", "periodenddate", 
        "di_327651", "di_327644_text", #EDIT metrics because they have more null entries
    ]
).dropna().rename(columns={
    "di_327652": "EAR", 
    "di_327645_text": "EAR_flag"
})
# 
cerf_df["forecastyear"] = cerf_df["forecastyear"].astype(int)
cerf_df["EAR_flag"] = cerf_df["EAR_flag"].astype(int)

cerf_df

,fiscalyear,scenariolevel,forecastyear,EAR,EAR_flag,gvkey,companyname,country
0,2017,High,2030,0.100000,0,111819,"Orrstown Financial Services, Inc.",United States
2,2017,High,2050,0.160000,0,111819,"Orrstown Financial Services, Inc.",United States
4,2017,Medium,2040,0.070000,0,111819,"Orrstown Financial Services, Inc.",United States
6,2017,Low,2030,0.030000,0,111819,"Orrstown Financial Services, Inc.",United States
8,2017,Medium,2025,0.030000,0,111819,"Orrstown Financial Services, Inc.",United States
...,...,...,...,...,...,...,...,...
2252678,2023,High,2050,2.171778,0,357018,NIIT Learning Systems Limited,India
2252679,2023,Low,2025,0.371646,0,357018,NIIT Learning Systems Limited,India
2252680,2023,High,2030,1.012907,0,357018,NIIT Learning Systems Limited,India
2252681,2023,Low,2020,0.180156,0,357018,NIIT Learning Systems Limited,India


In [71]:
cerf_df = cerf_df.groupby(
    ["gvkey", "fiscalyear", "forecastyear", "scenariolevel"]
).agg({
    "EAR": "mean", 
    "EAR_flag": "max", 
    "companyname": "first", 
    "country": "first", 
}).drop(columns=["companyname", "country"])

cerf_df

EAR  EAR_flag
gvkey  fiscalyear forecastyear scenariolevel                    
100001 2017       2020         High           0.280000         0
                               Low            0.050000         0
                               Medium         0.100000         0
                  2025         High           3.390000         0
                               Low            0.380000         0
...                                                ...       ...
358176 2023       2040         Low            1.329855         0
                               Medium         2.338599         0
                  2050         High           3.906918         0
                               Low            1.551523         0
                               Medium         2.944448         0

[1038270 rows x 2 columns]

Create a dictionary that stores the dataframes for each scenario
- rerun here is safe

In [80]:
cer_dict = {
    (scenario, horizon): 
        cerf_df.xs((scenario, horizon), level=["scenariolevel", "forecastyear"])
    for scenario in ["Low", "Medium", "High"]
    for horizon in [2020, 2025, 2030, 2040, 2050]
}

print(cer_dict)

{('Low', 2020):                         EAR  EAR_flag
gvkey  fiscalyear                    
100001 2017        0.050000         0
       2018        0.000000         0
       2019        0.000000         0
       2020        0.662374         0
       2021        3.867465         0
...                     ...       ...
356115 2023        0.032048         0
356457 2023        0.692557         0
356706 2023        0.037502         0
357018 2023        0.180156         0
358176 2023        0.323772         0

[69218 rows x 2 columns], ('Low', 2025):                          EAR  EAR_flag
gvkey  fiscalyear                     
100001 2017         0.380000         0
       2018         1.838300         0
       2019         1.635270         0
       2020         4.442814         0
       2021        55.660053         1
...                      ...       ...
356115 2023         0.066445         0
356457 2023         1.345458         0
356706 2023         0.077584         0
357018 2023        

In [70]:
cer_2050h = cerf_df.xs((2050, "High"), level=["forecastyear", "scenariolevel"])

cer_2050h.describe()

,EAR,EAR_flag
count,6.921800e+04,69218.000000
mean,1.189737e+03,0.414242
std,1.628919e+05,0.492594
min,0.000000e+00,0.000000
25%,1.996025e+00,0.000000
50%,6.690037e+00,0.000000
75%,2.343524e+01,1.000000
max,4.268570e+07,1.000000


In [71]:
cer_2025l = cerf_df.xs((2025, "Low"), level=["forecastyear", "scenariolevel"])

cer_2025l.describe()

,EAR,EAR_flag
count,6.921800e+04,69218.000000
mean,6.216563e+01,0.096969
std,9.643774e+03,0.295918
min,0.000000e+00,0.000000
25%,8.081497e-02,0.000000
50%,3.779510e-01,0.000000
75%,1.460876e+00,0.000000
max,2.530776e+06,1.000000


In [122]:
cer_2025h = cerf_df.xs((2025, "High"), level=["forecastyear", "scenariolevel"])

cer_2025h.describe()

,EAR,EAR_flag
count,6.921800e+04,69218.000000
mean,3.547000e+02,0.204065
std,5.048406e+04,0.403020
min,0.000000e+00,0.000000
25%,5.381125e-01,0.000000
50%,1.884808e+00,0.000000
75%,6.706911e+00,0.000000
max,1.324092e+07,1.000000


In [130]:
cer_2030l = cerf_df.xs((2030, "Low"), level=["forecastyear", "scenariolevel"])
cer_2030h = cerf_df.xs((2030, "High"), level=["forecastyear", "scenariolevel"])

## Linking

First the two lagged, annual frequency datasets for emissions and fundamentals.

In [162]:
emissions

di_319413     di_319414      di_319415        country
gvkey  fiscalyear                                                          
1004   2016        56034.630253  28700.181128  263831.791850  United States
       2017        59349.114332  33489.111842  311638.624770  United States
       2018        54842.261413  30334.505186  211206.593710  United States
       2019        62932.147241  32039.420175  217486.646950  United States
       2020        62592.179000  32015.494000  192435.070000  United States
...                         ...           ...            ...            ...
362761 2021         5542.197413   4629.001659   34334.587431          India
       2022         7202.802447   4349.187602   34489.242775          India
       2023         6651.660029   4016.397443   31850.202650          India
362779 2020          105.640229    119.005359    3249.154936          Italy
       2021          162.790078    201.931404    5448.875156          Italy

[147049 rows x 4 columns]

In [163]:
m_returns

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2015-02 2015-02-27                    AAR CORP    USD    1.0   
           2015-03 2015-03-31                    AAR CORP    USD    1.0   
           2015-04 2015-04-30                    AAR CORP    USD    1.0   
           2015-05 2015-05-29                    AAR CORP    USD    1.0   
           2015-06 2015-06-30                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
361486 01W 2024-03 2024-03-29  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2024-04 2024-04-30  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2024-05 2024-05-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2024-06 2024-06-28  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2024-07 2024-07-31  FABRICA DE SCULE RASNOV SA    RON    1.0   

                         cshom  prccm      trfm   adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                         
1004   01  2015-02  39791000.0  29.40  1.642484  48.289041      1.000000   
           2015-03  39791000.0  30.70  1.642484  50.424271      1.000000   
           2015-04  39661000.0  30.24  1.646443  49.788431      1.000000   
           2015-05  39661000.0  29.54  1.646443  48.635921      1.000000   
           2015-06  39661000.0  31.87  1.646443  52.472133      1.000000   
...                        ...    ...       ...        ...           ...   
361486 01W 2024-03   5650039.0   3.60  1.102500   3.969000      0.217178   
           2024-04   5650039.0   3.60  1.102500   3.969000      0.214867   
           2024-05   5650039.0   9.50  1.102500  10.473750      0.217907   
           2024-06   5650039.0  12.20  1.102500  13.450500      0.215265   
           2024-07   5650039.0  11.40  1.102500  12.568500      0.217489   

                     local_ret  ...   m_USD_ret  m_local_ret    USD_secval  \
gvkey  iid data_ym              ...                                          
1004   01  2015-02    2.581996  ...    2.581996     2.581996  1.169855e+09   
           2015-03    4.421769  ...    4.421769     4.421769  1.221584e+09   
           2015-04   -1.260980  ...   -1.260980    -1.260980  1.199349e+09   
           2015-05   -2.314815  ...   -2.314815    -2.314815  1.171586e+09   
           2015-06    7.887610  ...    7.887610     7.887610  1.263996e+09   
...                        ...  ...         ...          ...           ...   
361486 01W 2024-03    0.000000  ...   -0.152682     0.000000  4.417436e+06   
           2024-04    0.000000  ...   -1.064424     0.000000  4.370415e+06   
           2024-05  163.888889  ...  167.622508   163.888889  1.169622e+07   
           2024-06   28.421053  ...   26.864346    28.421053  1.483833e+07   
           2024-07   -6.557377  ...   -5.591926    -6.557377  1.400858e+07   

                      USD_mktval  X_USD_mktval  datayear  datayear-1  \
gvkey  iid data_ym                                                     
1004   01  2015-02  1.169855e+09  1.140410e+09      2015        2014   
           2015-03  1.221584e+09  1.169855e+09      2015        2014   
           2015-04  1.199349e+09  1.221584e+09      2015        2014   
           2015-05  1.171586e+09  1.199349e+09      2015        2014   
           2015-06  1.263996e+09  1.171586e+09      2015        2014   
...                          ...           ...       ...         ...   
361486 01W 2024-03  4.417436e+06  4.424191e+06      2024        2023   
           2024-04  4.370415e+06  4.417436e+06      2024        2023   
           2024-05  1.169622e+07  4.370415e+06      2024        2023   
           2024-06  1.483833e+07  1.169622e+07      2024        2023   
           2024-07  1.400858e+07  1.483833e+07      2024        2023   

                    m_mktret      beta    X_beta  
gvkey  iid data_ym                                
10

In [164]:
fundamentals

USD_ceq       opm  investment
gvkey  fiscalyear                                  
1004   2015        865.800000  0.039757   -0.048119
       2016        914.200000  0.043675    0.042993
       2017        936.300000  0.049191    0.013696
       2018        905.900000  0.053953   -0.004919
       2019        902.600000  0.050926    0.370287
...                       ...       ...         ...
362282 2018         -0.466171  0.247554   -0.439908
       2019         -0.321476  0.313226   -0.198328
       2020         -0.153001  0.382569    0.069097
       2021          0.187893  0.258252   -0.459819
       2022          0.513596  0.010390    5.637264

[185987 rows x 3 columns]

In [172]:
""" test - can skip """
emissions.join(fundamentals).dropna().groupby(
    level="fiscalyear"
).size()

fiscalyear
2015     5726
2016    12673
2017    13487
2018    14829
2019    15314
2020    18323
2021    18258
2022    19099
2023     7503
dtype: int64

Start building the regression dataframe...

In [81]:
reg_df = m_returns.join(
    emissions.join(fundamentals).dropna(), 
    on=["gvkey", "datayear-1"], 
).dropna()

reg_df

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2017-01 2017-01-31                    AAR CORP    USD    1.0   
           2017-02 2017-02-28                    AAR CORP    USD    1.0   
           2017-03 2017-03-31                    AAR CORP    USD    1.0   
           2017-04 2017-04-28                    AAR CORP    USD    1.0   
           2017-05 2017-05-31                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
361486 01W 2023-08 2023-08-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-09 2023-09-29  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-10 2023-10-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-11 2023-11-30  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-12 2023-12-29  FABRICA DE SCULE RASNOV SA    RON    1.0   

                         cshom  prccm      trfm   adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                         
1004   01  2017-01  34329000.0  31.99  1.680476  53.758429      1.000000   
           2017-02  34329000.0  34.42  1.680476  57.841985      1.000000   
           2017-03  34324000.0  33.63  1.680476  56.514409      1.000000   
           2017-04  34324000.0  35.99  1.683981  60.606474      1.000000   
           2017-05  34324000.0  34.94  1.683981  58.838294      1.000000   
...                        ...    ...       ...        ...           ...   
361486 01W 2023-08   5650039.0   3.60  1.102500   3.969000      0.219514   
           2023-09   5650039.0   3.60  1.102500   3.969000      0.212655   
           2023-10   5650039.0   3.60  1.102500   3.969000      0.212630   
           2023-11   5650039.0   3.60  1.102500   3.969000      0.219513   
           2023-12   5650039.0   3.60  1.102500   3.969000      0.222218   

                    local_ret  ...    X_beta     di_319413     di_319414  \
gvkey  iid data_ym             ...                                         
1004   01  2017-01  -2.981392  ...  1.686367  56034.630253  28700.181128   
           2017-02   7.596124  ...  0.200928  56034.630253  28700.181128   
           2017-03  -2.295177  ...  0.159154  56034.630253  28700.181128   
           2017-04   7.240745  ... -0.586214  56034.630253  28700.181128   
           2017-05  -2.917477  ... -0.552977  56034.630253  28700.181128   
...                       ...  ...       ...           ...           ...   
361486 01W 2023-08   0.000000  ...  0.349470     28.918321     60.365321   
           2023-09   0.000000  ...  0.390276     28.918321     60.365321   
           2023-10   0.000000  ...  0.382048     28.918321     60.365321   
           2023-11   0.000000  ...  0.388536     28.918321     60.365321   
           2023-12   0.000000  ...  0.356243     28.918321     60.365321   

                        di_319415        country     USD_revt       USD_at  \
gvkey  iid data_ym                                                           
1004   01  2017-01  263831.791850  United States  1767.600000  1504.100000   
           2017-02  263831.791850  United States  1767.600000  1504.100000   
           2017-03  263831.791850  United States  1767.600000  1504.100000   
           2017-04  263831.791850  United States  1767.600000  1504.100000   
           2017-05  263831.791850  United States  1767.600000  1504.100000   
...                           ...            ...          ...          ...   
361486 01W 2023-08     376.564905        Romania     1.888745     6.403583   
           2023-09     376.564905        Romania     1.888745     6.403583   
           2023-10     376.564905        Romania     1.888745     6.403583   
           2023-11     376.564905        Romania     1.888745     6.403583   
           2023-12     376.564905        Romania     1.888745     6.403583   

                      USD_ceq        OP  investment 

Risk free rates taken as IMF SDR interest rates

In [82]:
rf_rates = pd.read_csv("riskfreerates.csv")
rf_rates = rf_rates.drop(columns=["From", "Financial Quarter"])
rf_rates["To"] = pd.to_datetime(rf_rates["To"]).dt.to_period('M')
rf_rates = rf_rates.rename(columns={
    "To": "data_ym", 
    "SDR Interest Rate": "rf_rate"
})
rf_rates = rf_rates.groupby(
    "data_ym"
).last()

rf_rates["rf_rate"] = ((1 + (rf_rates["rf_rate"] / 100))**(1/12) - 1) * 100

rf_rates

,rf_rate
data_ym,
2016-01,0.004915
2016-02,0.004332
2016-03,0.004166
2016-04,0.004166
2016-05,0.004166
...,...
2024-04,0.335972
2024-05,0.335008
2024-06,0.328499


Append risk free rates

In [83]:
reg_df = reg_df.join(
    rf_rates, 
    on=["data_ym"]
).dropna()

reg_df

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2017-01 2017-01-31                    AAR CORP    USD    1.0   
           2017-02 2017-02-28                    AAR CORP    USD    1.0   
           2017-03 2017-03-31                    AAR CORP    USD    1.0   
           2017-04 2017-04-28                    AAR CORP    USD    1.0   
           2017-05 2017-05-31                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
361486 01W 2023-08 2023-08-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-09 2023-09-29  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-10 2023-10-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-11 2023-11-30  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-12 2023-12-29  FABRICA DE SCULE RASNOV SA    RON    1.0   

                         cshom  prccm      trfm   adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                         
1004   01  2017-01  34329000.0  31.99  1.680476  53.758429      1.000000   
           2017-02  34329000.0  34.42  1.680476  57.841985      1.000000   
           2017-03  34324000.0  33.63  1.680476  56.514409      1.000000   
           2017-04  34324000.0  35.99  1.683981  60.606474      1.000000   
           2017-05  34324000.0  34.94  1.683981  58.838294      1.000000   
...                        ...    ...       ...        ...           ...   
361486 01W 2023-08   5650039.0   3.60  1.102500   3.969000      0.219514   
           2023-09   5650039.0   3.60  1.102500   3.969000      0.212655   
           2023-10   5650039.0   3.60  1.102500   3.969000      0.212630   
           2023-11   5650039.0   3.60  1.102500   3.969000      0.219513   
           2023-12   5650039.0   3.60  1.102500   3.969000      0.222218   

                    local_ret  ...     di_319413     di_319414      di_319415  \
gvkey  iid data_ym             ...                                              
1004   01  2017-01  -2.981392  ...  56034.630253  28700.181128  263831.791850   
           2017-02   7.596124  ...  56034.630253  28700.181128  263831.791850   
           2017-03  -2.295177  ...  56034.630253  28700.181128  263831.791850   
           2017-04   7.240745  ...  56034.630253  28700.181128  263831.791850   
           2017-05  -2.917477  ...  56034.630253  28700.181128  263831.791850   
...                       ...  ...           ...           ...            ...   
361486 01W 2023-08   0.000000  ...     28.918321     60.365321     376.564905   
           2023-09   0.000000  ...     28.918321     60.365321     376.564905   
           2023-10   0.000000  ...     28.918321     60.365321     376.564905   
           2023-11   0.000000  ...     28.918321     60.365321     376.564905   
           2023-12   0.000000  ...     28.918321     60.365321     376.564905   

                          country     USD_revt       USD_at    USD_ceq  \
gvkey  iid data_ym                                                       
1004   01  2017-01  United States  1767.600000  1504.100000  914.20000   
           2017-02  United States  1767.600000  1504.100000  914.20000   
           2017-03  United States  1767.600000  1504.100000  914.20000   
           2017-04  United States  1767.600000  1504.100000  914.20000   
           2017-05  United States  1767.600000  1504.100000  914.20000   
...                           ...          ...          ...        ...   
361486 01W 2023-08        Romania     1.888745     6.403583    5.18879   
           2023-09        Romania     1.888745     6.403583    5.18879   
           2023-10        Romania     1.888745     6.403583    5.18879   
           2023-11        Romania     1.888745     6.403583    5.18879   
           2023-12        Romania     1.888745     6.403583    5.18879   

                          OP  investmen

In [84]:
reg_df["m_excess_USD_ret"] = reg_df["m_USD_ret"] - reg_df["rf_rate"]
reg_df["m_excess_local_ret"] = reg_df["m_local_ret"] - reg_df["rf_rate"]

reg_df

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2017-01 2017-01-31                    AAR CORP    USD    1.0   
           2017-02 2017-02-28                    AAR CORP    USD    1.0   
           2017-03 2017-03-31                    AAR CORP    USD    1.0   
           2017-04 2017-04-28                    AAR CORP    USD    1.0   
           2017-05 2017-05-31                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
361486 01W 2023-08 2023-08-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-09 2023-09-29  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-10 2023-10-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-11 2023-11-30  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-12 2023-12-29  FABRICA DE SCULE RASNOV SA    RON    1.0   

                         cshom  prccm      trfm   adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                         
1004   01  2017-01  34329000.0  31.99  1.680476  53.758429      1.000000   
           2017-02  34329000.0  34.42  1.680476  57.841985      1.000000   
           2017-03  34324000.0  33.63  1.680476  56.514409      1.000000   
           2017-04  34324000.0  35.99  1.683981  60.606474      1.000000   
           2017-05  34324000.0  34.94  1.683981  58.838294      1.000000   
...                        ...    ...       ...        ...           ...   
361486 01W 2023-08   5650039.0   3.60  1.102500   3.969000      0.219514   
           2023-09   5650039.0   3.60  1.102500   3.969000      0.212655   
           2023-10   5650039.0   3.60  1.102500   3.969000      0.212630   
           2023-11   5650039.0   3.60  1.102500   3.969000      0.219513   
           2023-12   5650039.0   3.60  1.102500   3.969000      0.222218   

                    local_ret  ...      di_319415        country     USD_revt  \
gvkey  iid data_ym             ...                                              
1004   01  2017-01  -2.981392  ...  263831.791850  United States  1767.600000   
           2017-02   7.596124  ...  263831.791850  United States  1767.600000   
           2017-03  -2.295177  ...  263831.791850  United States  1767.600000   
           2017-04   7.240745  ...  263831.791850  United States  1767.600000   
           2017-05  -2.917477  ...  263831.791850  United States  1767.600000   
...                       ...  ...            ...            ...          ...   
361486 01W 2023-08   0.000000  ...     376.564905        Romania     1.888745   
           2023-09   0.000000  ...     376.564905        Romania     1.888745   
           2023-10   0.000000  ...     376.564905        Romania     1.888745   
           2023-11   0.000000  ...     376.564905        Romania     1.888745   
           2023-12   0.000000  ...     376.564905        Romania     1.888745   

                         USD_at    USD_ceq        OP  investment   rf_rate  \
gvkey  iid data_ym                                                           
1004   01  2017-01  1504.100000  914.20000  0.084445    0.042993  0.026877   
           2017-02  1504.100000  914.20000  0.084445    0.042993  0.023885   
           2017-03  1504.100000  914.20000  0.084445    0.042993  0.030781   
           2017-04  1504.100000  914.20000  0.084445    0.042993  0.038751   
           2017-05  1504.100000  914.20000  0.084445    0.042993  0.044226   
...                         ...        ...       ...         ...       ...   
361486 01W 2023-08     6.403583    5.18879  0.060654   -0.057218  0.331232   
           2023-09     6.403583    5.18879  0.060654   -0.057218  0.339023   
           2023-10     6.403583    5.18879  0.060654   -0.057218  0.343277   
           2023-11     6.403583    5.18879  0.060654   -0.057218  0.340388   
           2023-12     6.403583    5.18879  0.060654   -0.057218  

Then financial calculations, ratios, renaming, etc

In [85]:
# further GHG metrics
reg_df["GHGS1"] = reg_df["di_319413"]
reg_df["GHGS2"] = reg_df["di_319414"]
reg_df["GHGS3"] = reg_df["di_319415"]
reg_df["GHG"] = reg_df["GHGS1"] + reg_df["GHGS2"] + reg_df["GHGS3"]
reg_df["GHG_REVT"] = reg_df["GHG"] / reg_df["USD_revt"]
reg_df["GHG_AT"] = reg_df["GHG"] / reg_df["USD_at"]

# book to market
reg_df["BM"] = reg_df["USD_ceq"] / reg_df["X_USD_mktval"]

with pd.option_context('mode.use_inf_as_null', True):
    reg_df = reg_df.dropna()
reg_df

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2017-01 2017-01-31                    AAR CORP    USD    1.0   
           2017-02 2017-02-28                    AAR CORP    USD    1.0   
           2017-03 2017-03-31                    AAR CORP    USD    1.0   
           2017-04 2017-04-28                    AAR CORP    USD    1.0   
           2017-05 2017-05-31                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
361486 01W 2023-08 2023-08-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-09 2023-09-29  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-10 2023-10-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-11 2023-11-30  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-12 2023-12-29  FABRICA DE SCULE RASNOV SA    RON    1.0   

                         cshom  prccm      trfm   adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                         
1004   01  2017-01  34329000.0  31.99  1.680476  53.758429      1.000000   
           2017-02  34329000.0  34.42  1.680476  57.841985      1.000000   
           2017-03  34324000.0  33.63  1.680476  56.514409      1.000000   
           2017-04  34324000.0  35.99  1.683981  60.606474      1.000000   
           2017-05  34324000.0  34.94  1.683981  58.838294      1.000000   
...                        ...    ...       ...        ...           ...   
361486 01W 2023-08   5650039.0   3.60  1.102500   3.969000      0.219514   
           2023-09   5650039.0   3.60  1.102500   3.969000      0.212655   
           2023-10   5650039.0   3.60  1.102500   3.969000      0.212630   
           2023-11   5650039.0   3.60  1.102500   3.969000      0.219513   
           2023-12   5650039.0   3.60  1.102500   3.969000      0.222218   

                    local_ret  ...   rf_rate  m_excess_USD_ret  \
gvkey  iid data_ym             ...                               
1004   01  2017-01  -2.981392  ...  0.026877         -3.008269   
           2017-02   7.596124  ...  0.023885          7.572239   
           2017-03  -2.295177  ...  0.030781         -2.325958   
           2017-04   7.240745  ...  0.038751          7.201995   
           2017-05  -2.917477  ...  0.044226         -2.961703   
...                       ...  ...       ...               ...   
361486 01W 2023-08   0.000000  ...  0.331232         -2.053267   
           2023-09   0.000000  ...  0.339023         -3.463745   
           2023-10   0.000000  ...  0.343277         -0.354996   
           2023-11   0.000000  ...  0.340388          2.896823   
           2023-12   0.000000  ...  0.335650          0.896566   

                    m_excess_local_ret         GHGS1         GHGS2  \
gvkey  iid data_ym                                                   
1004   01  2017-01           -3.008269  56034.630253  28700.181128   
           2017-02            7.572239  56034.630253  28700.181128   
           2017-03           -2.325958  56034.630253  28700.181128   
           2017-04            7.201995  56034.630253  28700.181128   
           2017-05           -2.961703  56034.630253  28700.181128   
...                                ...           ...           ...   
361486 01W 2023-08           -0.331232     28.918321     60.365321   
           2023-09           -0.339023     28.918321     60.365321   
           2023-10           -0.343277     28.918321     60.365321   
           2023-11           -0.340388     28.918321     60.365321   
           2023-12           -0.335650     28.918321     60.365321   

                            GHGS3            GHG    GHG_REVT      GHG_AT  \
gvkey  iid data_ym                                                         
1004   01  2017-01  263831.791850  348566.603231  197.197671  231.744301   
           2017-02  263831.791850  348566.603231  197.1976

In [86]:
reg_df = reg_df.drop(columns="beta").rename(
    columns={
        "X_beta": "beta", 
        "X_USD_mktval": "ME", 
    }
)

reg_df

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2017-01 2017-01-31                    AAR CORP    USD    1.0   
           2017-02 2017-02-28                    AAR CORP    USD    1.0   
           2017-03 2017-03-31                    AAR CORP    USD    1.0   
           2017-04 2017-04-28                    AAR CORP    USD    1.0   
           2017-05 2017-05-31                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
361486 01W 2023-08 2023-08-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-09 2023-09-29  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-10 2023-10-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-11 2023-11-30  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-12 2023-12-29  FABRICA DE SCULE RASNOV SA    RON    1.0   

                         cshom  prccm      trfm   adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                         
1004   01  2017-01  34329000.0  31.99  1.680476  53.758429      1.000000   
           2017-02  34329000.0  34.42  1.680476  57.841985      1.000000   
           2017-03  34324000.0  33.63  1.680476  56.514409      1.000000   
           2017-04  34324000.0  35.99  1.683981  60.606474      1.000000   
           2017-05  34324000.0  34.94  1.683981  58.838294      1.000000   
...                        ...    ...       ...        ...           ...   
361486 01W 2023-08   5650039.0   3.60  1.102500   3.969000      0.219514   
           2023-09   5650039.0   3.60  1.102500   3.969000      0.212655   
           2023-10   5650039.0   3.60  1.102500   3.969000      0.212630   
           2023-11   5650039.0   3.60  1.102500   3.969000      0.219513   
           2023-12   5650039.0   3.60  1.102500   3.969000      0.222218   

                    local_ret  ...   rf_rate  m_excess_USD_ret  \
gvkey  iid data_ym             ...                               
1004   01  2017-01  -2.981392  ...  0.026877         -3.008269   
           2017-02   7.596124  ...  0.023885          7.572239   
           2017-03  -2.295177  ...  0.030781         -2.325958   
           2017-04   7.240745  ...  0.038751          7.201995   
           2017-05  -2.917477  ...  0.044226         -2.961703   
...                       ...  ...       ...               ...   
361486 01W 2023-08   0.000000  ...  0.331232         -2.053267   
           2023-09   0.000000  ...  0.339023         -3.463745   
           2023-10   0.000000  ...  0.343277         -0.354996   
           2023-11   0.000000  ...  0.340388          2.896823   
           2023-12   0.000000  ...  0.335650          0.896566   

                    m_excess_local_ret         GHGS1         GHGS2  \
gvkey  iid data_ym                                                   
1004   01  2017-01           -3.008269  56034.630253  28700.181128   
           2017-02            7.572239  56034.630253  28700.181128   
           2017-03           -2.325958  56034.630253  28700.181128   
           2017-04            7.201995  56034.630253  28700.181128   
           2017-05           -2.961703  56034.630253  28700.181128   
...                                ...           ...           ...   
361486 01W 2023-08           -0.331232     28.918321     60.365321   
           2023-09           -0.339023     28.918321     60.365321   
           2023-10           -0.343277     28.918321     60.365321   
           2023-11           -0.340388     28.918321     60.365321   
           2023-12           -0.335650     28.918321     60.365321   

                            GHGS3            GHG    GHG_REVT      GHG_AT  \
gvkey  iid data_ym                                                         
1004   01  2017-01  263831.791850  348566.603231  197.197671  231.744301   
           2017-02  263831.791850  348566.603231  197.1976

In [87]:
def drop_outlier_rows(df, drop_outliers, level=0.001, high_only=False):
    keep_idx = pd.Series(True, index=df.index)
    for drop_col in drop_outliers:
        if high_only:
            keep_idx &= (df[drop_col] < df[drop_col].quantile(1 - level))
        else:
            keep_idx &= (
                (df[drop_col].quantile(level) < df[drop_col])
                & (df[drop_col] < df[drop_col].quantile(1 - level))
            )
    return df[keep_idx]

## Fama-Macbeth Regression

In [93]:
""" Create all the dataframes """
ghg_reg_df = reg_df.copy()

ghg_reg_df

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2017-01 2017-01-31                    AAR CORP    USD    1.0   
           2017-02 2017-02-28                    AAR CORP    USD    1.0   
           2017-03 2017-03-31                    AAR CORP    USD    1.0   
           2017-04 2017-04-28                    AAR CORP    USD    1.0   
           2017-05 2017-05-31                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
361486 01W 2023-08 2023-08-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-09 2023-09-29  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-10 2023-10-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-11 2023-11-30  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-12 2023-12-29  FABRICA DE SCULE RASNOV SA    RON    1.0   

                         cshom  prccm      trfm   adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                         
1004   01  2017-01  34329000.0  31.99  1.680476  53.758429      1.000000   
           2017-02  34329000.0  34.42  1.680476  57.841985      1.000000   
           2017-03  34324000.0  33.63  1.680476  56.514409      1.000000   
           2017-04  34324000.0  35.99  1.683981  60.606474      1.000000   
           2017-05  34324000.0  34.94  1.683981  58.838294      1.000000   
...                        ...    ...       ...        ...           ...   
361486 01W 2023-08   5650039.0   3.60  1.102500   3.969000      0.219514   
           2023-09   5650039.0   3.60  1.102500   3.969000      0.212655   
           2023-10   5650039.0   3.60  1.102500   3.969000      0.212630   
           2023-11   5650039.0   3.60  1.102500   3.969000      0.219513   
           2023-12   5650039.0   3.60  1.102500   3.969000      0.222218   

                    local_ret  ...   rf_rate  m_excess_USD_ret  \
gvkey  iid data_ym             ...                               
1004   01  2017-01  -2.981392  ...  0.026877         -3.008269   
           2017-02   7.596124  ...  0.023885          7.572239   
           2017-03  -2.295177  ...  0.030781         -2.325958   
           2017-04   7.240745  ...  0.038751          7.201995   
           2017-05  -2.917477  ...  0.044226         -2.961703   
...                       ...  ...       ...               ...   
361486 01W 2023-08   0.000000  ...  0.331232         -2.053267   
           2023-09   0.000000  ...  0.339023         -3.463745   
           2023-10   0.000000  ...  0.343277         -0.354996   
           2023-11   0.000000  ...  0.340388          2.896823   
           2023-12   0.000000  ...  0.335650          0.896566   

                    m_excess_local_ret         GHGS1         GHGS2  \
gvkey  iid data_ym                                                   
1004   01  2017-01           -3.008269  56034.630253  28700.181128   
           2017-02            7.572239  56034.630253  28700.181128   
           2017-03           -2.325958  56034.630253  28700.181128   
           2017-04            7.201995  56034.630253  28700.181128   
           2017-05           -2.961703  56034.630253  28700.181128   
...                                ...           ...           ...   
361486 01W 2023-08           -0.331232     28.918321     60.365321   
           2023-09           -0.339023     28.918321     60.365321   
           2023-10           -0.343277     28.918321     60.365321   
           2023-11           -0.340388     28.918321     60.365321   
           2023-12           -0.335650     28.918321     60.365321   

                            GHGS3            GHG    GHG_REVT      GHG_AT  \
gvkey  iid data_ym                                                         
1004   01  2017-01  263831.791850  348566.603231  197.197671  231.744301   
           2017-02  263831.791850  348566.603231  197.1976

In [95]:
cer_reg_dfs = {
    (scenario, horizon):
        ghg_reg_df.join(
            df, 
            on=["gvkey", "datayear-1"]
        ).dropna()
    for (scenario, horizon), df in cer_dict.items()
}

cer_reg_dfs

{('Low',
  2020):                      datadate                   conm curcdm  ajexm  \
 gvkey  iid data_ym                                                   
 100001 01  2018-01 2018-01-31               ACCOR SA    USD    1.0   
            2018-02 2018-02-28               ACCOR SA    USD    1.0   
            2018-03 2018-03-29               ACCOR SA    USD    1.0   
            2018-04 2018-04-30               ACCOR SA    USD    1.0   
            2018-05 2018-05-31               ACCOR SA    USD    1.0   
 ...                       ...                    ...    ...    ...   
 355280 01W 2024-07 2024-07-31    ITMAX SYSTEM BERHAD    MYR    1.0   
 356706 01W 2024-06 2024-06-28  LOTTOMATICA GROUP SPA    EUR    1.0   
            2024-07 2024-07-31  LOTTOMATICA GROUP SPA    EUR    1.0   
        02W 2024-06 2024-06-28  LOTTOMATICA GROUP SPA    EUR    1.0   
            2024-07 2024-07-31  LOTTOMATICA GROUP SPA    EUR    1.0   
 
                            cshom  prccm      trfm   adjcl

In [125]:
def perform_fm_reg(df, cer_flag=False):
    df = drop_outlier_rows(
        df, 
        [
            "beta", "ME", "BM", "OP", "investment", 
            # "GHGS1", "GHGS2", "GHGS3", 
            "m_USD_ret"
        ]
    ).copy()

    """ TOGGLE """
    if cer_flag:
        df = drop_outlier_rows(
            df, 
            [
                "EAR"
            ], 
            level=0.05
        ).copy()

    """ Notice that we restrict our focus to normal functioning companies which 
    have positive, non-zero market capitalisations, book values of equity and 
    revenues. 

    We do not exclude the possibility of negative operating profitability however, 
    because we deem this is within the scope of normal functioning companies.
    """
    df = df[
        (df["ME"] > 0) &
        (df["BM"] > 0) &
        (df["USD_revt"] > 0)
    ].copy()

    """  """
    df["ln_ME"] = np.log(df["ME"])
    df["ln_GHG"] = np.log(df["GHG"])

    df["ln_BM"] = np.log(df["BM"])
    df["ln_GHG_REVT"] = np.log(df["GHG_REVT"])
    # df["ln_GHG_AT"] = np.log(df["GHG_AT"])

    with pd.option_context('mode.use_inf_as_null', True):
        df = df.dropna()
    
    """  """
    reg_res = df.groupby(
        level="data_ym"
    ).apply(
        lambda ym_df: smf.gls(
        formula="m_excess_USD_ret ~ beta + ln_ME + ln_BM + OP + investment + " + ("EAR" if cer_flag else "ln_GHG"), 
        data=ym_df
        ).fit(
        ).params
        # .summary()
        # .params
        # .rsquared
        # .rsquared_adj
    )

    """  """
    print("- coefficients")
    print(reg_res.mean())
    print("- t-statistics")
    print(
        reg_res.mean() / (reg_res.std() / np.sqrt(len(reg_res)))
    )
    print("- stdevs")
    print(reg_res.std())



"""  """    

'  '

In [126]:
perform_fm_reg(ghg_reg_df)

- coefficients
Intercept     2.145926
beta          0.237956
ln_ME        -0.052023
ln_BM         0.109194
OP            0.344749
investment   -0.060099
ln_GHG        0.073931
dtype: float64
- t-statistics
Intercept     1.868382
beta          1.597315
ln_ME        -1.260174
ln_BM         1.243736
OP            4.460354
investment   -0.494948
ln_GHG        2.796540
dtype: float64
- stdevs
Intercept     11.656486
beta           1.511909
ln_ME          0.418970
ln_BM          0.891026
OP             0.784427
investment     1.232329
ln_GHG         0.268302
dtype: float64


In [116]:
for scenario in ["Low", "Medium", "High"]:
    for horizon in [2020, 2025, 2030, 2040, 2050]:
        print(f">>> {scenario} {horizon}")
        perform_fm_reg(
            cer_reg_dfs[scenario, horizon], 
            cer_flag=True
        )

>>> Low 2020
- coefficients
Intercept     1.100198
beta          0.333103
ln_ME         0.003317
ln_BM         0.090228
OP            0.742038
investment   -0.035539
EAR           0.261416
dtype: float64
- t-statistics
Intercept     0.666021
beta          1.736043
ln_ME         0.062398
ln_BM         0.656314
OP            2.077038
investment   -0.188482
EAR           1.945404
dtype: float64
- stdevs
Intercept     14.682375
beta           1.705422
ln_ME          0.472466
ln_BM          1.221927
OP             3.175378
investment     1.675923
EAR            1.194364
dtype: float64
>>> Low 2025
- coefficients
Intercept     1.271984
beta          0.304258
ln_ME        -0.009990
ln_BM         0.077597
OP            0.610222
investment   -0.006175
EAR           0.025188
dtype: float64
- t-statistics
Intercept     0.779257
beta          1.584135
ln_ME        -0.186737
ln_BM         0.570939
OP            1.808076
investment   -0.037813
EAR           1.769587
dtype: float64
- stdevs
Intercept

In [658]:
fm_reg_df.describe()

,ajexm,cshom,prccm,trfm,adjclose,exratd_toUSD,local_ret,USD_fxret,USD_ret,ret_mspan,...,GHG,GHG_REVT,GHG_AT,BM,EAR,EAR_flag,ln_ME,ln_GHG,ln_BM,ln_GHG_REVT
count,813165.000000,8.131650e+05,8.131650e+05,813165.000000,8.131650e+05,813165.000000,813165.000000,813165.000000,813165.000000,813165.00000,...,8.131650e+05,813165.000000,813165.000000,8.131650e+05,813165.000000,813165.000000,813165.000000,813165.000000,813165.000000,813165.000000
mean,1.157771,2.318023e+09,3.194712e+03,1.746299,4.779147e+03,0.411112,0.668595,-0.126240,0.568341,1.00419,...,1.327595e+06,322.493402,219.367775,5.622569e-07,0.607153,0.000405,21.563285,12.031111,-15.018527,5.244419
std,1.558995,1.109353e+11,2.891994e+04,26.214167,4.421459e+04,0.481179,11.860473,2.484969,12.272911,0.31311,...,6.559314e+06,1804.775299,425.563783,8.378977e-07,0.673848,0.020110,1.940284,1.997077,1.137731,0.950997
min,0.000206,0.000000e+00,5.000000e-04,1.000000,5.083333e-04,0.000040,-97.706940,-55.401016,-97.450385,1.00000,...,1.782000e+00,0.020015,0.000462,1.476235e-09,0.020011,0.000000,16.113416,0.577736,-20.333771,-3.911257
25%,1.000000,5.746100e+07,7.500000e+00,1.040204,9.254848e+00,0.032924,-5.583756,-1.161087,-6.069447,1.00000,...,4.340038e+04,98.904747,49.067107,1.412563e-07,0.126834,0.000000,20.141724,10.678224,-15.772690,4.594157
50%,1.000000,2.544000e+08,2.735000e+01,1.161967,3.409976e+01,0.144165,0.000000,0.000000,-0.104828,1.00000,...,1.565240e+05,196.353873,122.829650,3.013348e-07,0.337923,0.000000,21.508905,11.960965,-15.015044,5.279919
75%,1.000000,1.017405e+09,1.775000e+02,1.565487,2.392893e+02,1.000000,5.891557,0.878276,6.155118,1.00000,...,6.235608e+05,351.364209,261.417721,6.601154e-07,0.842860,0.000000,22.761648,13.343202,-14.230851,5.861823
max,100.000000,1.296054e+13,2.650000e+06,3009.997196,2.102751e+06,3.336582,2121.311475,818.073529,2125.523396,88.00000,...,3.779907e+08,298461.169575,29239.356035,1.533983e-05,3.082000,1.000000,28.029744,19.750380,-11.085058,12.606395


In [460]:
# fm_reg_df = fm_reg_df[
#     ["beta", "ln_ME", "BM", "OP", "investment", "ln_GHG", "m_USD_ret"]
# ]

fm_reg_df

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2017-01 2017-01-31                    AAR CORP    USD    1.0   
           2017-02 2017-02-28                    AAR CORP    USD    1.0   
           2017-03 2017-03-31                    AAR CORP    USD    1.0   
           2017-04 2017-04-28                    AAR CORP    USD    1.0   
           2017-05 2017-05-31                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
361486 01W 2023-08 2023-08-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-09 2023-09-29  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-10 2023-10-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-11 2023-11-30  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-12 2023-12-29  FABRICA DE SCULE RASNOV SA    RON    1.0   

                         cshom  prccm      trfm   adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                         
1004   01  2017-01  34329000.0  31.99  1.680476  53.758429      1.000000   
           2017-02  34329000.0  34.42  1.680476  57.841985      1.000000   
           2017-03  34324000.0  33.63  1.680476  56.514409      1.000000   
           2017-04  34324000.0  35.99  1.683981  60.606474      1.000000   
           2017-05  34324000.0  34.94  1.683981  58.838294      1.000000   
...                        ...    ...       ...        ...           ...   
361486 01W 2023-08   5650039.0   3.60  1.102500   3.969000      0.219514   
           2023-09   5650039.0   3.60  1.102500   3.969000      0.212655   
           2023-10   5650039.0   3.60  1.102500   3.969000      0.212630   
           2023-11   5650039.0   3.60  1.102500   3.969000      0.219513   
           2023-12   5650039.0   3.60  1.102500   3.969000      0.222218   

                    local_ret  ...         GHGS1         GHGS2          GHGS3  \
gvkey  iid data_ym             ...                                              
1004   01  2017-01  -2.981392  ...  56034.630253  28700.181128  263831.791850   
           2017-02   7.596124  ...  56034.630253  28700.181128  263831.791850   
           2017-03  -2.295177  ...  56034.630253  28700.181128  263831.791850   
           2017-04   7.240745  ...  56034.630253  28700.181128  263831.791850   
           2017-05  -2.917477  ...  56034.630253  28700.181128  263831.791850   
...                       ...  ...           ...           ...            ...   
361486 01W 2023-08   0.000000  ...     28.918321     60.365321     376.564905   
           2023-09   0.000000  ...     28.918321     60.365321     376.564905   
           2023-10   0.000000  ...     28.918321     60.365321     376.564905   
           2023-11   0.000000  ...     28.918321     60.365321     376.564905   
           2023-12   0.000000  ...     28.918321     60.365321     376.564905   

                              GHG    GHG_REVT      GHG_AT            BM  \
gvkey  iid data_ym                                                        
1004   01  2017-01  348566.603231  197.197671  231.744301  8.057654e-07   
           2017-02  348566.603231  197.197671  231.744301  8.324647e-07   
           2017-03  348566.603231  197.197671  231.744301  7.736939e-07   
           2017-04  348566.603231  197.197671  231.744301  7.919841e-07   
           2017-05  348566.603231  197.197671  231.744301  7.400507e-07   
...                           ...         ...         ...           ...   
361486 01W 2023-08     465.848548  246.644520   72.748111  1.142104e-06   
           2023-09     465.848548  246.644520   72.748111  1.162116e-06   
           2023-10     465.848548  246.644520   72.748111  1.199600e-06   
           2023-11     465.848548  246.644520   72.748111  1.199740e-06   
           2023-12     465.848548  246.644520   72.748111  1.162120e-06   

                        ln

Ready for regression...

datadate                        conm curcdm  ajexm  \
gvkey  iid data_ym                                                        
1004   01  2017-01 2017-01-31                    AAR CORP    USD    1.0   
           2017-02 2017-02-28                    AAR CORP    USD    1.0   
           2017-03 2017-03-31                    AAR CORP    USD    1.0   
           2017-04 2017-04-28                    AAR CORP    USD    1.0   
           2017-05 2017-05-31                    AAR CORP    USD    1.0   
...                       ...                         ...    ...    ...   
361486 01W 2023-08 2023-08-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-09 2023-09-29  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-10 2023-10-31  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-11 2023-11-30  FABRICA DE SCULE RASNOV SA    RON    1.0   
           2023-12 2023-12-29  FABRICA DE SCULE RASNOV SA    RON    1.0   

                         cshom  prccm      trfm   adjclose  exratd_toUSD  \
gvkey  iid data_ym                                                         
1004   01  2017-01  34329000.0  31.99  1.680476  53.758429      1.000000   
           2017-02  34329000.0  34.42  1.680476  57.841985      1.000000   
           2017-03  34324000.0  33.63  1.680476  56.514409      1.000000   
           2017-04  34324000.0  35.99  1.683981  60.606474      1.000000   
           2017-05  34324000.0  34.94  1.683981  58.838294      1.000000   
...                        ...    ...       ...        ...           ...   
361486 01W 2023-08   5650039.0   3.60  1.102500   3.969000      0.219514   
           2023-09   5650039.0   3.60  1.102500   3.969000      0.212655   
           2023-10   5650039.0   3.60  1.102500   3.969000      0.212630   
           2023-11   5650039.0   3.60  1.102500   3.969000      0.219513   
           2023-12   5650039.0   3.60  1.102500   3.969000      0.222218   

                    local_ret  ...    GHG_REVT      GHG_AT            BM  \
gvkey  iid data_ym             ...                                         
1004   01  2017-01  -2.981392  ...  197.197671  231.744301  8.057654e-07   
           2017-02   7.596124  ...  197.197671  231.744301  8.324647e-07   
           2017-03  -2.295177  ...  197.197671  231.744301  7.736939e-07   
           2017-04   7.240745  ...  197.197671  231.744301  7.919841e-07   
           2017-05  -2.917477  ...  197.197671  231.744301  7.400507e-07   
...                       ...  ...         ...         ...           ...   
361486 01W 2023-08   0.000000  ...  246.644520   72.748111  1.142104e-06   
           2023-09   0.000000  ...  246.644520   72.748111  1.162116e-06   
           2023-10   0.000000  ...  246.644520   72.748111  1.199600e-06   
           2023-11   0.000000  ...  246.644520   72.748111  1.199740e-06   
           2023-12   0.000000  ...  246.644520   72.748111  1.162120e-06   

                        ln_ME     ln_GHG      ln_BM  ln_GHG_REVT   rf_rate  \
gvkey  iid data_ym                                                           
1004   01  2017-01  20.849523  12.761585 -14.031473     5.284207  0.026877   
           2017-02  20.816924  12.761585 -13.998875     5.284207  0.023885   
           2017-03  20.890139  12.761585 -14.072089     5.284207  0.030781   
           2017-04  20.866774  12.761585 -14.048725     5.284207  0.038751   
           2017-05  20.934596  12.761585 -14.116547     5.284207  0.044226   
...                       ...        ...        ...          ...       ...   
361486 01W 2023-08  15.329139   6.143861 -13.682639     5.507948  0.331232   
           2023-09  15.311769   6.143861 -13.665268     5.507948  0.339023   
           2023-10  15.280023   6.143861 -13.633523     5.507948  0.343277   
           2023-11  15.279906   6.143861 -13.633405     5.507948  0.340388   
           2023-12  15.311765   6.143861 -13.665265     5.507948  0.335650   

                    m_excess_USD_ret  m_excess_local

,Intercept,beta,ln_ME,BM,OP,investment,ln_GHG
data_ym,,,,,,,
2016-01,-2.092071,-2.843224,-0.350397,2.932313e+04,1.155040,-3.518653,0.315729
2016-02,7.857908,0.235727,-0.630638,8.339679e+05,0.717559,-1.261690,0.461758
2016-03,19.445363,2.460236,-0.730252,1.172683e+06,-1.301388,-1.698184,0.332570
2016-04,8.501775,0.771122,-0.589118,1.317701e+06,-1.421597,-2.624359,0.479929
2016-05,-2.494601,-1.022125,0.398654,-6.873462e+05,1.063975,1.179337,-0.533246
...,...,...,...,...,...,...,...
2024-03,-4.432205,0.205344,0.147478,2.743985e+05,0.222127,0.048258,0.195432
2024-04,-7.734911,-1.852889,0.072215,1.195741e+05,1.281437,1.991481,0.437139
2024-05,-9.305826,0.739660,0.406008,1.111642e+06,0.171310,1.790647,0.058704


In [227]:
# dir(res["2016-01"])

In [351]:
# res["ln_GHG"].rolling(window=6, min_periods=1).mean().plot()

In [343]:
res.mean()

Intercept         0.956935
beta              0.226488
ln_ME            -0.080619
BM            61587.983689
OP                0.338677
investment       -0.058005
ln_GHG            0.085944
dtype: float64

t values: https://www.statology.org/t-score-p-value-calculator/

In [338]:
res.mean() / (res.std() / np.sqrt(len(res)))

Intercept    -3.032521
beta          1.134454
ln_ME         1.701190
BM            1.398562
OP            7.303926
investment   -1.238582
ln_GHG        2.326360
dtype: float64

In [451]:
fm_reg_df.std()

/var/folders/y5/ty9zs0px6gbg3f5k7p4jqh4m0000gn/T/ipykernel_1285/2696475453.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  fm_reg_df.std()


datadate        820 days 22:27:12.215353496
ajexm                              1.549552
cshom                    87743525282.625046
prccm                          28094.575787
trfm                              20.484626
adjclose                       62646.058135
exratd_toUSD                       0.500299
local_ret                         53.089962
USD_fxret                          8.781603
USD_ret                           55.338814
ret_mspan                          0.273404
m_USD_ret                         12.349158
m_local_ret                       11.988111
USD_secval               25014059984.318691
USD_mktval              102448084198.340576
ME                      101286856447.572601
datayear                            2.24251
datayear-1                          2.24251
m_mktret                           4.588039
beta                               1.176957
di_319413                   12598717.127909
di_319414                    1821916.855848
di_319415                    499

In [653]:
len(res) - 1

78

## Checks on Duplicates

In [ ]:
""" Now determine the number of unique companies in the 
different years
"""
print(f"All years: {np.unique(df['fiscalyear'])}")

for year in np.unique(df['fiscalyear']):
    print(f"{year} => {np.unique(df[df['fiscalyear'] == year]['gvkey']).size}")

NameError: name 'df' is not defined

In [ ]:
"""
All years: [2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015
 2016 2017 2018 2019 2020 2021 2022 2023]
2002 => 1763
2003 => 2000
2004 => 2885
2005 => 3880
2006 => 4170
2007 => 4307
2008 => 4269
2009 => 4563
2010 => 4723
2011 => 4833
2012 => 4868
2013 => 5757
2014 => 6154
2015 => 6235
2016 => 13882
2017 => 14786
2018 => 16979
2019 => 17378
2020 => 23353
2021 => 23386
2022 => 24053
2023 => 9725
"""

In [ ]:
# unique non-nan in returns
returns_gvkeys = np.unique(returns_df[~np.isnan(returns_df["gvkey"])]["gvkey"])

print(returns_gvkeys.size)

24110


In [ ]:
env_ret_common_gvkeys = np.intersect1d(
    env_gvkeys, 
    returns_gvkeys, 
    assume_unique=True
)

print(env_ret_common_gvkeys.size)

24110


In [ ]:
""" Check the missing gvkeys to see what country they are from """
missing_gvkeys = np.setdiff1d(env_gvkeys, env_ret_common_gvkeys, assume_unique=True)

print(missing_gvkeys.size)

7400


In [ ]:
""" Create representatives and break down their distribution """
df_missing_reprs_idx = [(df["gvkey"] == gvkey).idxmax() for gvkey in missing_gvkeys]
missing_dist = df.iloc[df_missing_reprs_idx][["gvkey", "country"]].groupby(
    "country"
).count().reset_index().sort_values('gvkey', ascending=False)

In [ ]:
missing_dist

,country,gvkey
79,United States,5220
10,Canada,803
78,United Kingdom,271
13,China,113
24,France,111
...,...,...
58,Peru,1
57,Panama,1
56,Pakistan,1
48,Marshall Islands,1


In [ ]:
missing_dist[missing_dist["country"] != "United States"]["gvkey"].sum()

2180

In [15]:
company_dups = df[df.duplicated('gvkey', keep=False) == True].sort_values(by="gvkey")

In [26]:
last_iid = None
last_fyears = set()
for i, row in company_dups.iterrows():
    current_iid = row["institutionid"]
    current_fyear = row["fiscalyear"]
    if last_iid is not None and current_iid == last_iid:
        if current_fyear in last_fyears:
            raise Exception(f"Fyear clash!, iid: {current_iid}, prev_fyears: {last_fyears}, clash: {current_fyear}")
        else:
            last_fyears.add(current_fyear)
    else:
        last_iid = current_iid
        last_fyears.clear()
        last_fyears.add(current_fyear)

print("No issues...")

Exception: Fyear clash!, iid: 4415462, prev_fyears: {2022}, clash: 2022

In [31]:
company_dups.head(20)

,periodid,institutionid,reportedcurrencyisocode,tcprimarysectorid,fiscalyear,periodenddate,di_319380,di_319381,di_319382,di_319383,...,streetaddress3,streetaddress4,zipcode,yearfounded,monthfounded,dayfounded,officephonevalue,otherphonevalue,officefaxvalue,webpage
5,30D218CF-2E2A-46B4-AF72-CADF593290E8,4074603,USD,713A00,2022,01/01/2023,0.285,0.021,2.079,0.153,...,NaN,NaN,76011,1961.0,NaN,NaN,972 595 5000,NaN,NaN,www.sixflags.com
2503,1989BCA5-218F-4857-A3CE-1F2114D67F8E,4074603,USD,713A00,2023,31/12/2023,0.302,0.021,2.098,0.147,...,NaN,NaN,76011,1961.0,NaN,NaN,972 595 5000,NaN,NaN,www.sixflags.com
4988,5B249039-D1E2-43D2-825B-DDD233FE2FE0,4996548,USD,561300,2023,31/12/2023,0.015,0.005,0.125,0.040,...,NaN,NaN,75024,2007.0,NaN,NaN,972 692 2400,NaN,NaN,bgsf.com
40,5F205052-5493-41C8-9C8C-1E0FE1FC5DC0,4996548,USD,561300,2022,01/01/2023,0.014,0.005,0.122,0.041,...,NaN,NaN,75024,2007.0,NaN,NaN,972 692 2400,NaN,NaN,bgsf.com
1,01C53196-7DD9-42C7-9D3B-F152BFB3A364,4054841,USD,445000A,2022,01/01/2023,3.061,0.003,186.279,0.203,...,NaN,NaN,1506 MA,1867.0,NaN,NaN,31 88 659 9111,NaN,NaN,www.aholddelhaize.com
2462,793C84DE-3E9D-4CE0-8768-647BA85F9272,4054841,USD,445000A,2023,31/12/2023,3.183,0.003,181.084,0.189,...,NaN,NaN,1506 MA,1867.0,NaN,NaN,31 88 659 9111,NaN,NaN,www.aholddelhaize.com
110,A706AF20-7252-4C95-AA58-2481363942C5,10175068,USD,722000,2022,02/01/2023,0.095,0.053,0.696,0.389,...,NaN,NaN,33309,2011.0,NaN,NaN,954-618-2000,NaN,NaN,www.burgerfi.com
6071,96731D0E-B576-41E4-8386-DAD27811DF60,10175068,USD,722000,2023,01/01/2024,0.092,0.054,0.625,0.368,...,NaN,NaN,33309,2011.0,NaN,NaN,954-618-2000,NaN,NaN,www.burgerfi.com
99,0446986D-7007-4B72-AA5B-49AA5ADB4CF2,28295169,USD,541512,2022,01/01/2023,0.042,0.020,0.362,0.170,...,NaN,NaN,55425,2016.0,NaN,NaN,952 851 5200,NaN,NaN,www.skywatertechnology.com
6011,9AABC54A-2E82-4C6A-99E2-489EF08AE342,28295169,USD,541512,2023,31/12/2023,0.051,0.018,0.420,0.147,...,NaN,NaN,55425,2016.0,NaN,NaN,952 851 5200,NaN,NaN,www.skywatertechnology.com


In [32]:
company_dups[company_dups["institutionid"] == 4415462]

,periodid,institutionid,reportedcurrencyisocode,tcprimarysectorid,fiscalyear,periodenddate,di_319380,di_319381,di_319382,di_319383,...,streetaddress3,streetaddress4,zipcode,yearfounded,monthfounded,dayfounded,officephonevalue,otherphonevalue,officefaxvalue,webpage
23,5282F3CE-9CFA-4830-9116-3B36EFDE5089,4415462,USD,52A000,2022,01/01/2023,0.001,0.001,0.054,0.037,...,NaN,NaN,050059,1993.0,NaN,NaN,7 727 244 5484,NaN,7 727 244 5480,www.homecredit.kz
24,5282F3CE-9CFA-4830-9116-3B36EFDE5089,4415462,USD,52A000,2022,01/01/2023,0.001,0.001,0.054,0.037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


array(['United States', 'Netherlands', 'United Kingdom', 'Canada',
       'Kazakhstan', 'Belarus', 'Australia', 'Belgium', 'Austria',
       'Finland', 'Ireland', nan, 'Singapore', 'France', 'Denmark',
       'Japan', 'Israel', 'Italy', 'South Africa', 'Thailand', 'Germany',
       'China', 'Hong Kong', 'Luxembourg', 'India', 'Switzerland',
       'Malaysia', 'South Korea', 'Kenya', 'New Zealand', 'Spain',
       'Pakistan', 'Saudi Arabia', 'Sweden', 'British Virgin Islands',
       'Kuwait', 'Turkey', 'Philippines', 'Mauritius', 'Bangladesh',
       'Cayman Islands', 'Botswana', 'Egypt', 'Malta', 'Malawi',
       'Jamaica', 'Bermuda', 'Colombia', 'Mexico', 'Norway', 'Brazil',
       'Bahrain', 'Morocco', 'Indonesia', 'Romania', 'Russia',
       'Ivory Coast', 'Tunisia', 'Greece', 'Vietnam', 'Taiwan', 'Nigeria',
       'Oman', 'Qatar', 'Portugal', 'United Arab Emirates', 'Jersey',
       'Poland', 'Bulgaria', 'Chile', 'Reunion', 'Ghana', 'Monaco',
       'Bahamas', 'Guernsey'], dtype=o